# FitchGroup Codeathon '25 - Drive Sustainability Using AI - Group 1

### Problem Understanding

#### What are we predicting?

We're predicting greenhouse gas emissions for companies that don't publicly report them, using two targets:

- **Scope 1 Emissions** - Direct emissions from sources the company owns/controls

    Examples: burning fuel in company vehicles, emissions from company-owned factories, on-site manufacturing processes

- **Scope 2 Emissions** - Indirect emissions from purchased energy

    Examples: electricity bought from the grid, purchased steam/heating/cooling

#### Why does this matter?
Many companies don't report their emissions, making it hard for investors, regulators, and sustainability analysts to assess climate risk. Our models will estimate emissions for these non-reporting companies.

### Key Hypotheses We Can Form

#### Hypothesis 1: Sector Mix Drives Emissions

Different industries have vastly different emission intensities.

Specific predictions:

- Manufacturing, mining, energy → HIGH Scope 1 (they burn fuel, run machinery)
- Utilities, heavy electricity users → HIGH Scope 2 (they buy lots of power)
- Services, software, consulting → LOW both (mostly office work)

Features to test this:

- Percentage in high-emission sectors
- Dominant sector
- Sector diversity

#### Hypothesis 2: Revenue as a Scaling Factor

Bigger companies (by revenue) generally emit more, but emission intensity (emissions per dollar of revenue) varies by sector.

Specific predictions:

- 2x revenue in manufacturing ≈ 2x emissions
- 2x revenue in consulting ≠ 2x emissions (office space doesn't scale linearly)

Features to test this:

- Raw revenue
- Log(revenue) - to handle the wide range
- Interaction: revenue × sector exposure

#### Hypothesis 3: Geography Matters for Scope 2
Scope 2 depends heavily on where you buy electricity because grid carbon intensity varies by country.

Specific predictions:

Same company, same electricity use:

- In coal-heavy grid (e.g., Poland, China) → HIGH Scope 2
- In renewable-heavy grid (e.g., Norway, Iceland) → LOW Scope 2

Features to test this:

- Country/region codes
- Target encoding: average Scope 2 by country

#### Hypothesis 4: Environmental Score Reflects Emissions Management

Companies with better environmental scores likely have emissions reduction programs.

Specific predictions:
- Lower environmental_score (1 = best) → LOWER emissions
- Higher environmental_score (5 = worst) → HIGHER emissions
- This effect is stronger for Scope 1 (more controllable) than Scope 2

Features to test this:
- Environmental score (raw and transformed)
- Environmental activities (positive vs negative adjustments)

#### Hypothesis 5: Scope 1 vs Scope 2 Have Different Drivers
The two targets are fundamentally different and need different features.

Scope 1 (Direct) is driven by:

- Manufacturing intensity
- Fuel combustion activities
- Industrial processes
- Company's own operations

Scope 2 (Indirect) is driven by:

- Electricity consumption
- Grid carbon intensity (geography)
- Office/facility scale
- Purchased energy

Implication: 

We might need:

- Separate models for Scope 1 vs Scope 2, OR
- Multi-output model that learns different weights for each target

#### Hypothesis 6: Revenue Concentration Affects Emission Predictability
Companies focused on one sector are more predictable than diversified conglomerates.

Specific predictions:

- Specialized company (HHI = 10,000): Emissions closely match sector average
- Diversified company (HHI = 2,500): Emissions are averaged across sectors, harder to predict

Features to test this:

- Herfindahl index
- Entropy
- Number of sectors

#### Hypothesis 7: Environmental Activities Are Leading Indicators
Companies actively managing environmental impact will have different emissions than expected from sector alone.

Specific predictions:

- Positive activities (renewable energy, efficiency programs) → LOWER emissions than sector average
- Negative activities (pollution incidents, violations) → HIGHER emissions

Features to test this:

- Count of positive vs negative activities
- Net environmental score adjustment
- Presence of any activities (vs none)

#### Hypothesis 8: SDG Commitments Signal Intent

Companies committed to climate-related SDGs are working on emissions reduction.

Specific predictions:

- Committed to SDG 7 (Clean Energy) or SDG 13 (Climate Action) → LOWER emissions
- More SDG commitments overall → more mature sustainability programs → LOWER emissions

Features to test this:

- Binary flags for climate SDGs
- Total number of SDG commitments

#### Hypothesis 9: Scale sets the baseline

Larger organizations should emit disproportionately more, but only up to a point.
    
Specific predictions:

- log transformed revenue and regional revenue residuals will correlate positively with both scopes, revealing giants and over-performers within each country.

Features to test this:

- `log_revenue`, `log_rev_country_resid`, `logrev_x_highintensity`.

#### Hypothesis 10: Sector mix drives structural emissions

Firms with higher revenue share in carbon-heavy industries should emit above peers even when similar in size.
    
Specific predictions:

- sector concentration, heavy-intensity share, and manufacturing prevalence will show strong positive correlations.

Features to test this:

- `sect_high_intensity_pct`, `sect_C_pct`, `sect_hhi`, `sect_entropy`.

#### Hypothesis 11: Governance gaps signal unmanaged carbon risk

When governance scores outpace environmental scores, execution lags strategy, leading to higher emissions than expected.

Specific predictions:

- governance-environment score gaps correlate with increased scopes, while absolute governance scores correlate negatively.

Features to test this:

- `gov_env_gap`, `governance_score`, `governance_score_squared`.

#### Hypothesis 12: Environmental action mix is a leading indicator

Companies running more positive programs (renewables, efficiency) should under-shoot sector averages, while negative events push emissions higher.

Specific predictions:

- counts/ratios of positive vs. negative activities, net adjustments, and "has activity" flags explain residual variance after sectors.

Features to test this:

- `env_positive_cnt`, `env_positive_ratio`, `env_adj_per_event`, `has_env_activity`.

#### Hypothesis 13: SDG climate commitments amplify or mitigate intensity

Companies that actively pledge SDGs tied to climate and energy (7/12/13) should curb emissions relative to peers with similar industrial exposure.

Specific predictions:

- SDG flags/cluster counts interact with sector intensity to reduce emissions relative to peers.

Features to test this:

- `sdg_climate_focus`, `climate_focus_x_intensity`, `sdg_focus_x_envgap`.

#### Hypothesis 14: Social and governance balance matters

When social and overall ESG momentum outpaces environmental execution, the imbalance should show up as excess emissions unless offset by scale-adjusted investments.

Specific predictions:

- larger positive gaps correlate with higher emissions unless offset by scale-adjusted investments.

Features to test this:

- `env_overall_gap`, `social_env_gap`, `logrev_x_envgap`.

#### Hypothesis 15: PPP-Adjusted Revenue Reflects True Emission Scale

Raw revenue alone may not accurately reflect operational scale across countries with different cost levels. Adjusting revenue by Purchasing Power Parities (PPP) normalizes for local price differences, giving a better estimate of real production and energy use.

Specific predictions:

- Companies in countries with high nominal revenue but low PPP → actual emissions may be lower than raw revenue suggests

- Companies in countries with lower nominal revenue but high PPP → actual emissions may be higher than raw revenue suggests

Features to test:

- revenue / PPP → PPP-adjusted revenue

- Interaction with region (Western Europe, North America) → captures regional energy mix differences

- Interaction with sector → aligns scaled revenue with industry-specific emission intensities

#### Hypothesis 16: Exponentially-Weighted SDG Impact Signals Stronger Emission Reduction Behavior

Companies that commit to high-impact SDGs—and especially those mapped to exponentially larger impact weights—are far more likely to invest in cleaner operations, energy efficiency, and reporting discipline. The exponential scaling captures the idea that not all SDGs contribute equally: a commitment to Climate Action (SDG 13) should matter far more than Peace & Justice (SDG 16).

Specific predictions:

- Entities with high exponential SDG impact scores → LOWER Scope 1 and Scope 2 emissions (strong sustainability alignment)

- Entities with low or flat impact profiles → HIGHER emissions (minimal environmental commitment)

- A single high-impact SDG can outweigh several low-impact ones because exponential weights amplify true environmental focus

Features to test:

- Sum of exponential impact indexes per entity (overall Environmental Impact Score)

- Average exponential impact index (normalizes by number of SDGs)

- Maximum exponential impact index the entity has (captures “strongest” SDG commitment)

- Standard deviation of exponential impact indexes (captures whether commitments are concentrated on high-impact environmental goals)

#### Hypothesis 17: Environmental Reporting Intensity Signals Company Size
Statement: 
Entities with more environmental activities recorded (count or diversity) also have higher Scope 1 emissions and higher Scope 2 emissions.
because:

* large companies are more regulated

* large companies must report ESG activities

* large companies have sustainability departments

Enviornment Activities Adjustment auditors focus on large emitters

Features to test this:
- `env_activities_count`

#### Hypothesis 18: Environmental Activity Diversity Predicts Emissions
The company who engage in a wider range of environmental-related activities are likely operating multiple processes, facilities, or business units, each contributing to greenhouse gas emissions

Predictions:

* greater environmental activity diversity => higher direct emissions (Scope 1) and higher electricity-related emissions (Scope 2).

Features to test this:

* `env_activity_types`


#### Hypothesis 19: Environmental Reporting Only Predicts Emissions When Combined With Company Size
Large companies will operate more facilities, consume more energy, and emit more greenhouse gasses. Hints to these companies are more likely to track environmental activities, more disclose environmental actions, and will be required to report sustainability efforts publicly.

Predictions:

* High Revenue + High environment count → High Scope_1 and High Scope_2

Features to test:

* Revenue
* Environment count

## Import Libraries

In [1]:
# Import Libraries & Requirements
!pip install -r ../requirements.txt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os

## Load Data Set & Inspection

In [2]:
# Load CSV Data Set
environmental_activities_df = pd.read_csv('../data/environmental_activities.csv')
revenue_distribution_by_sector_df = pd.read_csv('../data/revenue_distribution_by_sector.csv')
sustainable_development_goals_df = pd.read_csv('../data/sustainable_development_goals.csv')
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')

### Train Data Set

In [3]:
train_df.head()

,entity_id,region_code,region_name,country_code,country_name,revenue,overall_score,environmental_score,social_score,governance_score,target_scope_1,target_scope_2
0,1782,WEU,Western Europe,GB,United Kingdom of Great Britain and Northern I...,3.528060e+08,2.988,3.900,1.750,2.833,60.0,0.0
1,3918,NAM,Northern America,US,United States of America,1.513700e+09,2.770,3.004,2.942,2.143,265.0,0.0
2,10299,WEU,Western Europe,FR,France,1.560000e+09,2.501,2.979,2.560,1.571,1136.0,0.0
3,2324,NAM,Northern America,US,United States of America,1.238511e+10,3.207,3.776,3.000,2.429,1468.0,0.0
4,1206,WEU,Western Europe,ES,Spain,2.980000e+09,1.998,2.138,1.785,2.000,1802.0,0.0


In [4]:
train_df.shape

(429, 12)

In [5]:
train_df.columns

Index(['entity_id', 'region_code', 'region_name', 'country_code',
       'country_name', 'revenue', 'overall_score', 'environmental_score',
       'social_score', 'governance_score', 'target_scope_1', 'target_scope_2'],
      dtype='object')

In [6]:
train_df.dtypes

entity_id                int64
region_code             object
region_name             object
country_code            object
country_name            object
revenue                float64
overall_score          float64
environmental_score    float64
social_score           float64
governance_score       float64
target_scope_1         float64
target_scope_2         float64
dtype: object

### Test Data Set

In [7]:
test_df.head()

,entity_id,region_code,region_name,country_code,country_name,revenue,overall_score,environmental_score,social_score,governance_score
0,1076,WEU,Western Europe,NL,Netherlands,1.670000e+09,3.170,3.940,2.692,2.357
1,2067,WEU,Western Europe,GB,United Kingdom of Great Britain and Northern I...,5.880000e+08,2.976,4.000,2.014,2.286
2,910,WEU,Western Europe,DE,Germany,1.218100e+09,2.835,3.258,2.955,1.929
3,4082,WEU,Western Europe,DE,Germany,5.037500e+09,2.861,3.360,2.950,1.857
4,4102,WEU,Western Europe,SE,Sweden,1.415400e+09,2.950,3.550,2.900,1.929


In [8]:
test_df.shape

(49, 10)

In [9]:
test_df.columns

Index(['entity_id', 'region_code', 'region_name', 'country_code',
       'country_name', 'revenue', 'overall_score', 'environmental_score',
       'social_score', 'governance_score'],
      dtype='object')

In [10]:
test_df.dtypes

entity_id                int64
region_code             object
region_name             object
country_code            object
country_name            object
revenue                float64
overall_score          float64
environmental_score    float64
social_score           float64
governance_score       float64
dtype: object

### Environmental Activities Data Set

In [11]:
environmental_activities_df.head()

,entity_id,activity_type,activity_code,env_score_adjustment
0,2709,Transportation,M.70.4.P,0.05
1,107,Operation,MTH002,-0.10
2,10045,Operation,MTH002,-0.10
3,2709,End-use,J.58.16.B,-0.05
4,2677,Operation,J.58.20.P,0.10


In [12]:
environmental_activities_df.shape

(355, 4)

In [13]:
environmental_activities_df.columns

Index(['entity_id', 'activity_type', 'activity_code', 'env_score_adjustment'], dtype='object')

In [14]:
environmental_activities_df.dtypes

entity_id                 int64
activity_type            object
activity_code            object
env_score_adjustment    float64
dtype: object

### Revenue Distribution by Sector Data Set

In [15]:
revenue_distribution_by_sector_df.head()

,entity_id,nace_level_1_code,nace_level_1_name,nace_level_2_code,nace_level_2_name,revenue_pct
0,1735,A,"Agriculture, Forestry And Fishing",1,"Crop and animal production, hunting and relate...",0.031105
1,1195,A,"Agriculture, Forestry And Fishing",1,"Crop and animal production, hunting and relate...",0.362906
2,4092,A,"Agriculture, Forestry And Fishing",1,"Crop and animal production, hunting and relate...",0.222190
3,3592,A,"Agriculture, Forestry And Fishing",1,"Crop and animal production, hunting and relate...",0.063879
4,3169,A,"Agriculture, Forestry And Fishing",1,"Crop and animal production, hunting and relate...",0.422810


In [16]:
revenue_distribution_by_sector_df.shape

(799, 6)

In [17]:
revenue_distribution_by_sector_df.columns

Index(['entity_id', 'nace_level_1_code', 'nace_level_1_name',
       'nace_level_2_code', 'nace_level_2_name', 'revenue_pct'],
      dtype='object')

In [18]:
revenue_distribution_by_sector_df.dtypes

entity_id              int64
nace_level_1_code     object
nace_level_1_name     object
nace_level_2_code      int64
nace_level_2_name     object
revenue_pct          float64
dtype: object

### Sustainable Development Goals Data Set

In [19]:
sustainable_development_goals_df.head()

,entity_id,sdg_id,sdg_name
0,29,9,"Industry, Innovation and Infrastructure"
1,46,12,Responsible Consumption and Production
2,46,7,Affordable and Clean Energy
3,63,3,Good Health and Wellbeing
4,106,9,"Industry, Innovation and Infrastructure"


In [20]:
sustainable_development_goals_df.shape

(165, 3)

In [21]:
sustainable_development_goals_df.columns

Index(['entity_id', 'sdg_id', 'sdg_name'], dtype='object')

In [22]:
sustainable_development_goals_df.dtypes

entity_id     int64
sdg_id        int64
sdg_name     object
dtype: object

## Data Cleaning

### Check Null Values

In [23]:
# Data Cleaning - Check null values

def analyze_missing_values(df, dataset_name):
    """
    Comprehensive missing value analysis for a dataframe
    
    Parameters:
    -----------
    df : pandas DataFrame
        The dataframe to analyze
    dataset_name : str
        Name of the dataset for display purposes
    
    Returns:
    --------
    pandas DataFrame with missing value statistics
    """
    
    print(f"\n{'='*60}")
    print(f"MISSING VALUE ANALYSIS: {dataset_name}")
    print(f"{'='*60}\n")
    
    # Calculate missing values
    missing_count = df.isnull().sum()
    missing_pct = (df.isnull().sum() / len(df)) * 100
    dtypes = df.dtypes
    
    # Create summary dataframe
    missing_df = pd.DataFrame({
        'Column': df.columns,
        'Data_Type': dtypes.values,
        'Missing_Count': missing_count.values,
        'Missing_Percentage': missing_pct.values,
        'Non_Missing_Count': len(df) - missing_count.values
    })
    
    # Sort by missing percentage descending
    missing_df = missing_df.sort_values('Missing_Percentage', ascending=False)
    
    # Filter to show only columns with missing values
    missing_cols = missing_df[missing_df['Missing_Count'] > 0]
    
    if len(missing_cols) == 0:
        print("✓ No missing values found in this dataset!\n")
    else:
        print(f"⚠ Found {len(missing_cols)} columns with missing values:\n")
        print(missing_cols.to_string(index=False))
        print(f"\nTotal missing values: {missing_count.sum():,}")
        print(f"Percentage of dataset with any missing value: {(df.isnull().any(axis=1).sum() / len(df) * 100):.2f}%")
    
    return missing_df

In [24]:
train_missing = analyze_missing_values(train_df, "TRAIN_DF")
test_missing = analyze_missing_values(test_df, "TEST_DF")
environmental_activities_missing = analyze_missing_values(environmental_activities_df, "ENVIRONMENTAL_DF")
revenue_distribution_by_sector_missing = analyze_missing_values(revenue_distribution_by_sector_df, "TRAIN_DF")
sustainable_development_goals_missing = analyze_missing_values(sustainable_development_goals_df, "TRAIN_DF")


MISSING VALUE ANALYSIS: TRAIN_DF

✓ No missing values found in this dataset!


MISSING VALUE ANALYSIS: TEST_DF

✓ No missing values found in this dataset!


MISSING VALUE ANALYSIS: ENVIRONMENTAL_DF

✓ No missing values found in this dataset!


MISSING VALUE ANALYSIS: TRAIN_DF

✓ No missing values found in this dataset!


MISSING VALUE ANALYSIS: TRAIN_DF

✓ No missing values found in this dataset!



### Check Dulplicate Values

In [ ]:
# Data Cleaning - Check dulplicate values

def analyze_duplicates(df, dataset_name):
    """
    Comprehensive duplicate row analysis for a dataframe
    
    Parameters:
    -----------
    df : pandas DataFrame
        The dataframe to analyze
    dataset_name : str
        Name of the dataset for display purposes
    
    Returns:
    --------
    pandas DataFrame with duplicate statistics
    """
    
    print(f"\n{'='*60}")
    print(f"DUPLICATE ROW ANALYSIS: {dataset_name}")
    print(f"{'='*60}\n")
    
    # Count full-row duplicates
    duplicate_mask = df.duplicated(keep=False)
    total_duplicates = duplicate_mask.sum()
    unique_duplicate_rows = df[duplicate_mask].drop_duplicates()
    
    if total_duplicates == 0:
        print("✓ No duplicate rows found in this dataset!\n")
        return pd.DataFrame()  # empty result
    
    # Count how many times each duplicated row appears
    duplicate_summary = (
        df[df.duplicated(keep=False)]
        .value_counts()
        .reset_index(name='Count')
        .sort_values('Count', ascending=False)
    )
    
    print(f"⚠ Found {len(unique_duplicate_rows)} unique duplicated rows.")
    print(f"Total duplicated entries (including repeats): {total_duplicates}\n")
    
    print("Most common duplicate patterns:")
    display(duplicate_summary.head(10))
    
    return duplicate_summary


In [ ]:
# Run the duplicate-value method on each dataset
train_duplicates = analyze_duplicates(train_df, "TRAIN_DF")
test_duplicates = analyze_duplicates(test_df, "TEST_DF")
environmental_activities_duplicates = analyze_duplicates(environmental_activities_df, "ENVIRONMENTAL_DF")
revenue_distribution_by_sector_duplicates = analyze_duplicates(revenue_distribution_by_sector_df, "REVENUE_DF")
sustainable_development_goals_duplicates = analyze_duplicates(sustainable_development_goals_df, "SDG_DF")

### Check Revenue Distribution Coverage

In [ ]:
# CHECKING REVENUE DISTRIBUTION COVERAGE

# Get unique entity_ids from each dataset
train_entities = set(train_df['entity_id'].unique())
test_entities = set(test_df['entity_id'].unique())
revenue_entities = set(revenue_distribution_by_sector_df['entity_id'].unique())

print(f"Unique entities in train: {len(train_entities):,}")
print(f"Unique entities in test: {len(test_entities):,}")
print(f"Unique entities in revenue_distribution: {len(revenue_entities):,}")

# Check coverage
train_covered = train_entities & revenue_entities
test_covered = test_entities & revenue_entities

train_missing = train_entities - revenue_entities
test_missing = test_entities - revenue_entities

print("\n" + "="*60)
print("COVERAGE ANALYSIS")
print("="*60)

print(f"\nTRAIN dataset:")
print(f"  Entities WITH revenue data: {len(train_covered):,} ({len(train_covered)/len(train_entities)*100:.2f}%)")
print(f"  Entities WITHOUT revenue data: {len(train_missing):,} ({len(train_missing)/len(train_entities)*100:.2f}%)")

print(f"\nTEST dataset:")
print(f"  Entities WITH revenue data: {len(test_covered):,} ({len(test_covered)/len(test_entities)*100:.2f}%)")
print(f"  Entities WITHOUT revenue data: {len(test_missing):,} ({len(test_missing)/len(test_entities)*100:.2f}%)")

# Check if there are entities in revenue table that aren't in train/test
orphan_entities = revenue_entities - train_entities - test_entities
print(f"\nEntities in revenue table but NOT in train/test: {len(orphan_entities):,}")

print("\n" + "="*60)
print("IMPLICATIONS")
print("="*60)

if len(train_missing) > 0 or len(test_missing) > 0:
    print("\n⚠ IMPORTANT: Some entities don't have revenue distribution data!")
else:
    print("\n✓ Perfect coverage! All entities have revenue distribution data.")


### Checking Sustainable Distribution Coverage

In [ ]:
# CHECKING SUSTAINABLE DISTRIBUTION COVERAGE

# Get unique entity_ids from each dataset
sustain_entities = set(sustainable_development_goals_df['entity_id'].unique())

print(f"Unique entities in train: {len(train_entities):,}")
print(f"Unique entities in test: {len(test_entities):,}")
print(f"Unique entities in sustain_distribution: {len(sustain_entities):,}")

# Check coverage
train_covered = train_entities & sustain_entities
test_covered = test_entities & sustain_entities

train_missing = train_entities - sustain_entities
test_missing = test_entities - sustain_entities

print("\n" + "="*60)
print("COVERAGE ANALYSIS")
print("="*60)

print(f"\nTRAIN dataset:")
print(f"  Entities WITH sustainable_development_goals data: {len(train_covered):,} ({len(train_covered)/len(train_entities)*100:.2f}%)")
print(f"  Entities WITHOUT sustainable_development_goals data: {len(train_missing):,} ({len(train_missing)/len(train_entities)*100:.2f}%)")

print(f"\nTEST dataset:")
print(f"  Entities WITH sustainable_development_goals data: {len(test_covered):,} ({len(test_covered)/len(test_entities)*100:.2f}%)")
print(f"  Entities WITHOUT sustainable_development_goals data: {len(test_missing):,} ({len(test_missing)/len(test_entities)*100:.2f}%)")

# Check if there are entities in sustain table that aren't in train/test
orphan_entities = sustain_entities - train_entities - test_entities
print(f"\nEntities in sustain table but NOT in train/test: {len(orphan_entities):,}")

print("\n" + "="*60)
print("IMPLICATIONS")
print("="*60)

if len(train_missing) > 0 or len(test_missing) > 0:
    print("\n⚠ IMPORTANT: Some entities don't have sustain distribution data!")
else:
    print("\n✓ Perfect coverage! All entities have sustain distribution data.")


In [ ]:
print("\n" + "="*60)
print("ISOLATED DATASET OF ENTITIES IN TRAIN DATASET with NO sustainable_development_goals data ")
print("="*60)
Train_To_sustain_distribution_unmatched = (
    train_df
    .merge(sustainable_development_goals_df, on="entity_id", how="left", indicator=True)
    .query("_merge == 'left_only'")
    .drop(columns=["_merge"])
)

print(Train_To_sustain_distribution_unmatched.shape)
Train_To_sustain_distribution_unmatched.head()

### Checking Environment Distribution Coverage

In [ ]:
# CHECKING ENVIRONMENT DISTRIBUTION COVERAGE

# Get unique entity_ids from each dataset
environment_entities = set(environmental_activities_df['entity_id'].unique())

print(f"Unique entities in train: {len(train_entities):,}")
print(f"Unique entities in test: {len(test_entities):,}")
print(f"Unique entities in environment_distribution: {len(environment_entities):,}")

# Check coverage
train_covered = train_entities & environment_entities
test_covered = test_entities & environment_entities

train_missing = train_entities - environment_entities
test_missing = test_entities - environment_entities

print("\n" + "="*60)
print("COVERAGE ANALYSIS")
print("="*60)
print(f"\nTRAIN dataset:")
print(f"  Entities WITH environment data: {len(train_covered):,} ({len(train_covered)/len(train_entities)*100:.2f}%)")
print(f"  Entities WITHOUT environment data: {len(train_missing):,} ({len(train_missing)/len(train_entities)*100:.2f}%)")

print(f"\nTEST dataset:")
print(f"  Entities WITH environment data: {len(test_covered):,} ({len(test_covered)/len(test_entities)*100:.2f}%)")
print(f"  Entities WITHOUT environment data: {len(test_missing):,} ({len(test_missing)/len(test_entities)*100:.2f}%)")

# Check if there are entities in environment table that aren't in train/test
orphan_entities = environment_entities - train_entities - test_entities
print(f"\nEntities in environment table but NOT in train/test: {len(orphan_entities):,}")

print("\n" + "="*60)
print("IMPLICATIONS")
print("="*60)

if len(train_missing) > 0 or len(test_missing) > 0:
    print("\n⚠ IMPORTANT: Some entities don't have environment distribution data!")
else:
    print("\n✓ Perfect coverage! All entities have environment distribution data.")

## Revenue Distribution EDA & Feature Engineering

### Deep EDA on Revenue Distribution

#### Basic Statistics

In [ ]:
# How many sectors does each company operate in?
sectors_per_company = revenue_distribution_by_sector_df.groupby('entity_id').agg({
    'nace_level_1_code': 'nunique',
    'nace_level_2_code': 'nunique',
    'revenue_pct': 'sum'  # Should be ~100 for each company
})
sectors_per_company.columns = ['num_L1_sectors', 'num_L2_sectors', 'total_revenue_pct']

print("Sectors per company:")
print(sectors_per_company.describe())
print(f"\nCompanies with revenue_pct not summing to ~100: {(sectors_per_company['total_revenue_pct'] < 99).sum()}")


#### Sector Distribution

In [ ]:
# Most common NACE L1 sectors
print("\nTop 10 NACE L1 sectors by frequency:")
l1_freq = revenue_distribution_by_sector_df.groupby('nace_level_1_code')['entity_id'].nunique().sort_values(ascending=False)
print(l1_freq.head(10))

# Visualize
plt.figure(figsize=(12, 6))
l1_freq.head(15).plot(kind='barh', color='steelblue')
plt.xlabel('Number of Companies')
plt.title('Top 15 NACE Level 1 Sectors by Company Count', fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

#### Revenue Concentration Analysis

In [ ]:
# Calculate dominant sector percentage for each company
dominant_pct = revenue_distribution_by_sector_df.groupby('entity_id')['revenue_pct'].max()
print(f"\nDominant sector revenue percentage:")
print(dominant_pct.describe())

# Visualize concentration
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(dominant_pct, bins=30, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('% Revenue from Dominant Sector')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Revenue Concentration: Dominant Sector %', fontweight='bold')
axes[0].axvline(50, color='red', linestyle='--', label='50%')
axes[0].axvline(dominant_pct.mean(), color='green', linestyle='--', label=f'Mean: {dominant_pct.mean():.1f}%')
axes[0].legend()

axes[1].hist(sectors_per_company['num_L2_sectors'], bins=range(1, 20), edgecolor='black', alpha=0.7, color='orange')
axes[1].set_xlabel('Number of NACE L2 Sectors')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Company Diversification: Number of Sectors', fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\nCompanies with >80% revenue in one sector: {(dominant_pct > 80).sum()} ({(dominant_pct > 80).sum()/len(dominant_pct)*100:.1f}%)")
print(f"Companies with <50% revenue in dominant sector: {(dominant_pct < 50).sum()} ({(dominant_pct < 50).sum()/len(dominant_pct)*100:.1f}%)")

### Hypothesis-Driven Feature Validation

In [ ]:
# First, merge revenue features with train data for full analysis
# Create basic sector features
sector_features = []
for entity_id, group in revenue_distribution_by_sector_df.groupby('entity_id'):
    dominant = group.loc[group['revenue_pct'].idxmax()]
    HIGH_EMISSION = ['A', 'B', 'C', 'D', 'E']
    high_mask = group['nace_level_1_code'].isin(HIGH_EMISSION)
    
    sector_features.append({
        'entity_id': entity_id,
        'dominant_nace_l1': dominant['nace_level_1_code'],
        'dominant_nace_l1_name': dominant['nace_level_1_name'],
        'dominant_pct': dominant['revenue_pct'],
        'high_emission_pct': group[high_mask]['revenue_pct'].sum(),
        'num_l1_sectors': group['nace_level_1_code'].nunique(),
        'num_l2_sectors': group['nace_level_2_code'].nunique(),
        'is_manufacturing': int(dominant['nace_level_1_code'] == 'C'),
        'is_mining': int(dominant['nace_level_1_code'] == 'B'),
        'is_energy': int(dominant['nace_level_1_code'] == 'D'),
        'is_utilities': int(dominant['nace_level_1_code'] == 'E'),
        'is_transport': int(dominant['nace_level_1_code'] == 'H'),
        'is_services': int(dominant['nace_level_1_code'] in ['G', 'I', 'J', 'K', 'M', 'N']),
        'manufacturing_pct': group[group['nace_level_1_code'] == 'C']['revenue_pct'].sum(),
        'mining_pct': group[group['nace_level_1_code'] == 'B']['revenue_pct'].sum(),
    })

sector_df = pd.DataFrame(sector_features)
analysis_df = train_df.merge(sector_df, on='entity_id', how='left')

print(f"✓ Merged {len(analysis_df)} entities for analysis\n")

#### Hypothesis 1: Sector Mix Drives Emissions

In [ ]:
print("\n" + "-"*80)
print("Prediction 1A: Manufacturing, mining, energy → HIGH Scope 1")
print("-"*80)

# Compare high-emission vs low-emission sectors
high_emission_industries = analysis_df[analysis_df['high_emission_pct'] > 0.5]
low_emission_industries = analysis_df[analysis_df['high_emission_pct'] <= 0.5]

print(f"\n📊 Companies with >50% high-emission sectors (n={len(high_emission_industries)}):")
print(f"   Avg Scope 1: {high_emission_industries['target_scope_1'].mean():,.0f}")
print(f"   Median Scope 1: {high_emission_industries['target_scope_1'].median():,.0f}")

print(f"\n📊 Companies with ≤50% high-emission sectors (n={len(low_emission_industries)}):")
print(f"   Avg Scope 1: {low_emission_industries['target_scope_1'].mean():,.0f}")
print(f"   Median Scope 1: {low_emission_industries['target_scope_1'].median():,.0f}")

# Calculate effect size
effect_size = (high_emission_industries['target_scope_1'].mean() - 
               low_emission_industries['target_scope_1'].mean())
print(f"\n💡 Effect size: {effect_size:,.0f} ({effect_size/low_emission_industries['target_scope_1'].mean()*100:.1f}% increase)")

# Statistical test
from scipy import stats
t_stat, p_val = stats.ttest_ind(high_emission_industries['target_scope_1'], 
                                  low_emission_industries['target_scope_1'])
print(f"   Statistical significance: p={p_val:.6f}")
if p_val < 0.001:
    print("   ✅ HYPOTHESIS STRONGLY SUPPORTED (p < 0.001)")
elif p_val < 0.05:
    print("   ✅ HYPOTHESIS SUPPORTED (p < 0.05)")
else:
    print("   ❌ HYPOTHESIS NOT SUPPORTED (p >= 0.05)")

print("\n" + "-"*80)
print("Prediction 1B: Utilities, heavy electricity users → HIGH Scope 2")
print("-"*80)

print(f"\n📊 Companies with >50% high-emission sectors:")
print(f"   Avg Scope 2: {high_emission_industries['target_scope_2'].mean():,.0f}")
print(f"   Median Scope 2: {high_emission_industries['target_scope_2'].median():,.0f}")

print(f"\n📊 Companies with ≤50% high-emission sectors:")
print(f"   Avg Scope 2: {low_emission_industries['target_scope_2'].mean():,.0f}")
print(f"   Median Scope 2: {low_emission_industries['target_scope_2'].median():,.0f}")

# Specific sector analysis
print("\n" + "-"*80)
print("PREDICTION 1C: Services, software, consulting → LOW both")
print("-"*80)

sector_emissions = analysis_df.groupby('dominant_nace_l1_name').agg({
    'target_scope_1': ['mean', 'median', 'count'],
    'target_scope_2': ['mean', 'median']
}).round(0)

print("\n📊 Emissions by Sector (sorted by Scope 1):")
print(sector_emissions.sort_values(('target_scope_1', 'mean'), ascending=False))

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

# Scope 1 by sector
sector_scope1 = analysis_df.groupby('dominant_nace_l1_name')['target_scope_1'].mean().sort_values(ascending=True)
top_10_scope1 = sector_scope1.tail(10)
top_10_scope1.plot(kind='barh', ax=axes[0], color='steelblue')
axes[0].set_xlabel('Average Scope 1 Emissions', fontsize=12)
axes[0].set_title('Top 10 Sectors by Scope 1 Emissions', fontsize=14, fontweight='bold')
axes[0].axvline(analysis_df['target_scope_1'].mean(), color='red', linestyle='--', 
                label=f'Overall Avg: {analysis_df["target_scope_1"].mean():,.0f}')
axes[0].legend()

# Scope 2 by sector
sector_scope2 = analysis_df.groupby('dominant_nace_l1_name')['target_scope_2'].mean().sort_values(ascending=True)
top_10_scope2 = sector_scope2.tail(10)
top_10_scope2.plot(kind='barh', ax=axes[1], color='orange')
axes[1].set_xlabel('Average Scope 2 Emissions', fontsize=12)
axes[1].set_title('Top 10 Sectors by Scope 2 Emissions', fontsize=14, fontweight='bold')
axes[1].axvline(analysis_df['target_scope_2'].mean(), color='red', linestyle='--',
                label=f'Overall Avg: {analysis_df["target_scope_2"].mean():,.0f}')
axes[1].legend()

plt.tight_layout()
plt.show()

print("\n" + "-"*80)
print("FEATURE VALIDATION")
print("-"*80)

# Correlation of sector features with targets
sector_feature_cols = ['high_emission_pct', 'dominant_pct', 'num_l1_sectors', 'num_l2_sectors',
                       'is_manufacturing', 'is_mining', 'manufacturing_pct', 'mining_pct']

print("\n📊 Feature Correlation with Scope 1:")
scope1_corr = analysis_df[sector_feature_cols + ['target_scope_1']].corr()['target_scope_1'].drop('target_scope_1').sort_values(ascending=False)
for feature, corr in scope1_corr.items():
    print(f"   {feature:25s}: {corr:6.3f}")

print("\n📊 Feature Correlation with Scope 2:")
scope2_corr = analysis_df[sector_feature_cols + ['target_scope_2']].corr()['target_scope_2'].drop('target_scope_2').sort_values(ascending=False)
for feature, corr in scope2_corr.items():
    print(f"   {feature:25s}: {corr:6.3f}")

# Visualize correlations
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

scope1_corr.sort_values().plot(kind='barh', ax=axes[0], color='steelblue')
axes[0].set_xlabel('Correlation Coefficient')
axes[0].set_title('Sector Features vs Scope 1', fontweight='bold')
axes[0].axvline(0, color='black', linewidth=0.5)

scope2_corr.sort_values().plot(kind='barh', ax=axes[1], color='orange')
axes[1].set_xlabel('Correlation Coefficient')
axes[1].set_title('Sector Features vs Scope 2', fontweight='bold')
axes[1].axvline(0, color='black', linewidth=0.5)

plt.tight_layout()
plt.show()

print("\n✅ HYPOTHESIS 1 CONCLUSION:")
if scope1_corr['high_emission_pct'] > 0.2 or p_val < 0.05:
    print("   STRONGLY VALIDATED - Sector mix is a key predictor of emissions")
    print("   → HIGH PRIORITY: Include all sector features in final model")
else:
    print("   PARTIALLY VALIDATED - Some sector effects present but weak")
    print("   → MEDIUM PRIORITY: Include dominant sector only")

In [ ]:
print("="*80)
print("HYPOTHESIS 1 FINAL SUMMARY FOR TEAM")
print("="*80)

summary = """
HYPOTHESIS 1: SECTOR MIX DRIVES EMISSIONS ✅ STRONGLY VALIDATED

KEY FINDINGS:
1. Companies with >50% high-emission sectors have:
   - 186% higher Scope 1 emissions (p < 0.001)
   - 156% higher Scope 2 emissions
   
2. Sector features have strong correlations:
   - high_emission_pct: 0.275 (Scope 1), 0.154 (Scope 2)
   - manufacturing_pct: 0.221 (Scope 1), 0.164 (Scope 2)
   
3. Clear sector hierarchy:
   Top Scope 1: Mining (209K) > Energy (179K) > Agriculture (136K)
   Bottom Scope 1: Financial (1K) < Admin Services (3K)
   
RECOMMENDED FEATURES FOR FINAL MODEL:
  Priority 1 (MUST include):
    - high_emission_pct
    - manufacturing_pct, is_manufacturing
    - dominant_nace_l1 (categorical - target encode)
    - is_mining, mining_pct
    
  Priority 2 (Optional):
    - num_l2_sectors
    
HIGH-EMISSION SECTORS DEFINED AS:
  ['A', 'B', 'C', 'D', 'E'] = Agriculture, Mining, Manufacturing, Energy, Utilities

NEXT: Validate Hypothesis 2 (Revenue scaling) and create final feature set.
"""

print(summary)

#### Hypothesis 2: Revenue as Scaling Factor

In [ ]:
print("\n" + "-"*80)
print("PREDICTION 2A: 2x revenue in manufacturing ≈ 2x emissions")
print("-"*80)

# Focus on manufacturing companies
manufacturing = analysis_df[analysis_df['is_manufacturing'] == 1].copy()
services = analysis_df[analysis_df['is_services'] == 1].copy()

print(f"\n📊 MANUFACTURING companies (n={len(manufacturing)}):")
print(f"   Revenue vs Scope 1 correlation: {manufacturing['revenue'].corr(manufacturing['target_scope_1']):.3f}")
print(f"   Revenue vs Scope 2 correlation: {manufacturing['revenue'].corr(manufacturing['target_scope_2']):.3f}")

print(f"\n📊 SERVICE companies (n={len(services)}):")
print(f"   Revenue vs Scope 1 correlation: {services['revenue'].corr(services['target_scope_1']):.3f}")
print(f"   Revenue vs Scope 2 correlation: {services['revenue'].corr(services['target_scope_2']):.3f}")

print("\n" + "-"*80)
print("PREDICTION 2B: Test linear vs log transformation")
print("-"*80)

# Calculate emission intensity (emissions per revenue dollar)
analysis_df['scope1_intensity'] = analysis_df['target_scope_1'] / (analysis_df['revenue'] + 1)
analysis_df['scope2_intensity'] = analysis_df['target_scope_2'] / (analysis_df['revenue'] + 1)

print(f"\n📊 Emission Intensity by Sector:")
intensity_by_sector = analysis_df.groupby('dominant_nace_l1_name').agg({
    'scope1_intensity': 'mean',
    'scope2_intensity': 'mean'
}).sort_values('scope1_intensity', ascending=False).head(10)
print(intensity_by_sector)

# Visualize revenue vs emissions
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# Row 1: Linear scale
axes[0, 0].scatter(analysis_df['revenue'], analysis_df['target_scope_1'], alpha=0.5, s=20)
axes[0, 0].set_xlabel('Revenue')
axes[0, 0].set_ylabel('Scope 1 Emissions')
axes[0, 0].set_title('Revenue vs Scope 1 (Linear)', fontweight='bold')

axes[0, 1].scatter(analysis_df['revenue'], analysis_df['target_scope_2'], alpha=0.5, s=20, color='orange')
axes[0, 1].set_xlabel('Revenue')
axes[0, 1].set_ylabel('Scope 2 Emissions')
axes[0, 1].set_title('Revenue vs Scope 2 (Linear)', fontweight='bold')

# Emission intensity
axes[0, 2].scatter(analysis_df['revenue'], analysis_df['scope1_intensity'], alpha=0.5, s=20, color='green')
axes[0, 2].set_xlabel('Revenue')
axes[0, 2].set_ylabel('Scope 1 Intensity')
axes[0, 2].set_title('Revenue vs Emission Intensity', fontweight='bold')

# Row 2: Log scale
axes[1, 0].scatter(np.log1p(analysis_df['revenue']), np.log1p(analysis_df['target_scope_1']), alpha=0.5, s=20)
axes[1, 0].set_xlabel('Log(Revenue + 1)')
axes[1, 0].set_ylabel('Log(Scope 1 + 1)')
axes[1, 0].set_title('Log-Transformed Revenue vs Scope 1', fontweight='bold')

axes[1, 1].scatter(np.log1p(analysis_df['revenue']), np.log1p(analysis_df['target_scope_2']), alpha=0.5, s=20, color='orange')
axes[1, 1].set_xlabel('Log(Revenue + 1)')
axes[1, 1].set_ylabel('Log(Scope 2 + 1)')
axes[1, 1].set_title('Log-Transformed Revenue vs Scope 2', fontweight='bold')

# By sector color-coded
for sector in analysis_df['dominant_nace_l1'].unique()[:5]:  # Top 5 sectors
    sector_data = analysis_df[analysis_df['dominant_nace_l1'] == sector]
    axes[1, 2].scatter(np.log1p(sector_data['revenue']), 
                      np.log1p(sector_data['target_scope_1']), 
                      alpha=0.6, s=20, label=sector)
axes[1, 2].set_xlabel('Log(Revenue + 1)')
axes[1, 2].set_ylabel('Log(Scope 1 + 1)')
axes[1, 2].set_title('Revenue vs Scope 1 by Sector', fontweight='bold')
axes[1, 2].legend(fontsize=8)

plt.tight_layout()
plt.show()

# Compare correlations
print("\n" + "-"*80)
print("LINEAR vs LOG TRANSFORMATION")
print("-"*80)

corr_linear_1 = analysis_df['revenue'].corr(analysis_df['target_scope_1'])
corr_log_1 = np.log1p(analysis_df['revenue']).corr(np.log1p(analysis_df['target_scope_1']))
corr_linear_2 = analysis_df['revenue'].corr(analysis_df['target_scope_2'])
corr_log_2 = np.log1p(analysis_df['revenue']).corr(np.log1p(analysis_df['target_scope_2']))

print(f"\n📊 Scope 1:")
print(f"   Linear correlation: {corr_linear_1:.3f}")
print(f"   Log correlation:    {corr_log_1:.3f}")
print(f"   → {'Log transformation BETTER' if corr_log_1 > corr_linear_1 else 'Linear BETTER'}")

print(f"\n📊 Scope 2:")
print(f"   Linear correlation: {corr_linear_2:.3f}")
print(f"   Log correlation:    {corr_log_2:.3f}")
print(f"   → {'Log transformation BETTER' if corr_log_2 > corr_linear_2 else 'Linear BETTER'}")

print("\n" + "-"*80)
print("INTERACTION: Revenue × Sector")
print("-"*80)

# Create interaction feature
analysis_df['revenue_x_high_emission'] = analysis_df['revenue'] * analysis_df['high_emission_pct'] / 100

corr_interaction_1 = analysis_df['revenue_x_high_emission'].corr(analysis_df['target_scope_1'])
corr_interaction_2 = analysis_df['revenue_x_high_emission'].corr(analysis_df['target_scope_2'])

print(f"\n📊 Revenue × High-Emission % correlation:")
print(f"   Scope 1: {corr_interaction_1:.3f} (vs raw revenue: {corr_linear_1:.3f})")
print(f"   Scope 2: {corr_interaction_2:.3f} (vs raw revenue: {corr_linear_2:.3f})")

if corr_interaction_1 > corr_linear_1:
    print("   ✅ Interaction feature improves Scope 1 prediction")
else:
    print("   ❌ Interaction doesn't help Scope 1")

if corr_interaction_2 > corr_linear_2:
    print("   ✅ Interaction feature improves Scope 2 prediction")
else:
    print("   ❌ Interaction doesn't help Scope 2")

print("\n✅ HYPOTHESIS 2 CONCLUSION:")
if corr_log_1 > 0.3 or corr_interaction_1 > corr_linear_1:
    print("   VALIDATED - Revenue matters, especially with transformations")
    print(f"   → RECOMMENDED: Use log(revenue) and revenue × sector interactions")
else:
    print("   WEAK - Revenue alone is poor predictor")
    print("   → Revenue less important than sector composition")

In [ ]:
# Calculate key metrics from your results
linear_scope1 = 0.189
log_scope1 = 0.403
linear_scope2 = 0.187
log_scope2 = 0.310
interaction_scope1 = 0.311
interaction_scope2 = 0.229

improvement_scope1 = (log_scope1 - linear_scope1) / linear_scope1 * 100
improvement_scope2 = (log_scope2 - linear_scope2) / log_scope2 * 100

summary = f"""

HYPOTHESIS 2: REVENUE AS SCALING FACTOR ✅ VALIDATED

KEY FINDINGS:
1. Log transformation dramatically improves correlations:
   - Scope 1: Linear r={linear_scope1:.3f} → Log r={log_scope1:.3f} ({improvement_scope1:.0f}% improvement)
   - Scope 2: Linear r={linear_scope2:.3f} → Log r={log_scope2:.3f} ({improvement_scope2:.0f}% improvement)
   → Revenue-emission relationship is LOGARITHMIC, not linear
   
2. Interaction features outperform raw revenue:
   - Revenue × High-Emission %: r={interaction_scope1:.3f} (Scope 1), r={interaction_scope2:.3f} (Scope 2)
   - 64% better than raw revenue for Scope 1
   - 22% better than raw revenue for Scope 2
   → Sector context is CRITICAL for revenue effects
   
3. Emission intensity varies 14x across sectors:
   - Mining: 0.000188 (Scope 1 per revenue dollar) - HIGHEST
   - Manufacturing: 0.000045 
   - Professional Services: 0.000013 - LOWEST
   → Same revenue produces vastly different emissions by sector
   
4. Within-sector scaling is consistent:
   - Manufacturing: Revenue correlation r=0.262
   - Services: Revenue correlation r=0.269
   → Revenue matters WITHIN sectors, not across them
   
INTERPRETATION:
- Doubling revenue does NOT double emissions (logarithmic relationship)
- Economies of scale reduce emission intensity at larger sizes
- Sector sets the baseline, revenue scales within that baseline
- A $1B manufacturing company ≠ $1B software company in emissions

RECOMMENDED FEATURES FOR FINAL MODEL:
  Priority 1 (MUST include):
    - log_revenue (r=0.403 with Scope 1, strongest revenue feature)
    - revenue_x_high_emission_pct (r=0.311, best interaction)
    - log_revenue_x_high_emission_pct (test this too)
    
  Priority 2 (SHOULD include):
    - revenue_x_manufacturing_pct
    - log_revenue_x_manufacturing_pct
    - sector_avg_scope1 (sector baseline intensity)
    - sector_avg_scope2 (sector baseline intensity)
    
  Priority 3 (OPTIONAL):
    - revenue_squared (for polynomial effects)
    - log_revenue_x_num_sectors (diversification effect)

DO NOT INCLUDE:
  - Raw revenue alone (r=0.189, too weak)
  - Linear revenue features without transformation

TRANSFORMATION REQUIREMENTS:
  - ALWAYS use log(revenue + 1) instead of raw revenue
  - Consider log-transforming TARGETS too: log(Scope 1 + 1), log(Scope 2 + 1)
  - This will help with skewed emission distributions

MODEL IMPLICATIONS:
  1. Tree-based models (XGBoost, Random Forest) will work best
     - Naturally handle logarithmic relationships
     - Automatically discover interactions
     
  2. If using linear models:
     - MUST use log-transformed features
     - MUST include interaction terms manually
     - Consider polynomial features
     
  3. Expected performance contribution:
     - Log revenue alone: ~16% of variance explained (R² ≈ 0.16)
     - With interactions: ~20-25% of variance explained
     - Combined with sector features: ~40-50% total variance explained

VALIDATION OF SPECIFIC PREDICTIONS:
  ✅ "2x revenue in manufacturing ≈ 2x emissions"
     → PARTIAL: Within manufacturing, correlation exists (r=0.262)
     → But NOT linear - more like 2x revenue = 1.5x emissions (log relationship)
     
  ✅ "2x revenue in consulting ≠ 2x emissions" 
     → CONFIRMED: Low baseline intensity (0.000013 vs 0.000045 for manufacturing)
     → But within consulting, revenue still correlates (r=0.269)
     
  ✅ "Emission intensity varies by sector"
     → STRONGLY CONFIRMED: 14x variation from highest to lowest
     → Manufacturing produces 3.5x more emissions per dollar than services

VISUAL EVIDENCE:
  - Linear plots: Scattered, weak correlation
  - Log-log plots: Clear positive trend, strong correlation
  - By-sector plot: Distinct clusters, validating sector-specific baselines
  - Intensity plot: Hyperbolic decay pattern (efficiency gains at scale)

NEXT STEPS:
  1. Create all recommended log and interaction features
  2. Test on validation set to confirm correlations hold
  3. Combine with Hypothesis 1 sector features for maximum predictive power
  4. Consider separate feature sets for Scope 1 vs Scope 2 if needed

================================================================================
"""

print(summary)

# Add visual summary of improvements
print("\n📊 TRANSFORMATION IMPACT SUMMARY:\n")
print("Feature Type                  │ Scope 1 Corr │ Scope 2 Corr │ Status")
print("─" * 75)
print(f"Raw Revenue                   │    {linear_scope1:.3f}    │    {linear_scope2:.3f}    │ ❌ Too weak")
print(f"Log(Revenue + 1)              │    {log_scope1:.3f}    │    {log_scope2:.3f}    │ ✅ MUCH better")
print(f"Revenue × High-Emission %     │    {interaction_scope1:.3f}    │    {interaction_scope2:.3f}    │ ✅ Best overall")
print("─" * 75)
print(f"Improvement                   │   +{improvement_scope1:.0f}%     │   +{improvement_scope2:.0f}%     │")
print()

print("💡 KEY TAKEAWAY:")
print("   Revenue is a MULTIPLIER on sector-specific emission intensity,")
print("   not an independent driver. Always combine with sector features!")

#### Hypothesis 6: Revenue Concentration Affects Predictability

In [ ]:
print("="*80)
print("HYPOTHESIS 6: REVENUE CONCENTRATION AFFECTS EMISSION PREDICTABILITY")
print("="*80)

print("\n" + "-"*80)
print("PREDICTION: Specialized companies (high HHI) are more predictable")
print("            than diversified companies (low HHI)")
print("-"*80)

# Check HHI scale and fix if needed
if 'herfindahl_index' not in analysis_df.columns:
    print("⚠️ Creating Herfindahl Index...")
    hhi_data = []
    for entity_id, group in revenue_distribution_by_sector_df.groupby('entity_id'):
        # FIX: Remove the division by 100 since revenue_pct is already in 0-1 scale
        revenue_shares = group['revenue_pct']  # No division needed!
        hhi = (revenue_shares ** 2).sum()  # This will now give proper 0-1 scale HHI
        hhi_data.append({'entity_id': entity_id, 'herfindahl_index': hhi})
    hhi_df = pd.DataFrame(hhi_data)
    analysis_df = analysis_df.merge(hhi_df, on='entity_id', how='left')

# Also calculate CR1 - FIX the same issue
if 'cr1' not in analysis_df.columns:
    cr1_data = []
    for entity_id, group in revenue_distribution_by_sector_df.groupby('entity_id'):
        cr1 = group['revenue_pct'].max()  # No division by 100 needed!
        cr1_data.append({'entity_id': entity_id, 'cr1': cr1})
    cr1_df = pd.DataFrame(cr1_data)
    analysis_df = analysis_df.merge(cr1_df, on='entity_id', how='left')

print("\n📊 CONCENTRATION DISTRIBUTION:")
print(analysis_df['herfindahl_index'].describe())

# FIXED: Use 0-1 scale thresholds instead of 0-10000
print(f"\nCompany concentration breakdown:")
print(f"  Highly concentrated (HHI > 0.8): {(analysis_df['herfindahl_index'] > 0.8).sum()} ({(analysis_df['herfindahl_index'] > 0.8).sum()/len(analysis_df)*100:.1f}%)")
print(f"  Moderately concentrated (0.5-0.8): {((analysis_df['herfindahl_index'] >= 0.5) & (analysis_df['herfindahl_index'] <= 0.8)).sum()} ({((analysis_df['herfindahl_index'] >= 0.5) & (analysis_df['herfindahl_index'] <= 0.8)).sum()/len(analysis_df)*100:.1f}%)")
print(f"  Diversified (HHI < 0.5): {(analysis_df['herfindahl_index'] < 0.5).sum()} ({(analysis_df['herfindahl_index'] < 0.5).sum()/len(analysis_df)*100:.1f}%)")

# ============================================================
# TEST 1: EMISSION VARIANCE BY CONCENTRATION LEVEL
# ============================================================
print("\n" + "-"*80)
print("TEST 1: EMISSION VARIANCE BY CONCENTRATION LEVEL")
print("-"*80)

# FIXED: Use decimal thresholds (0.75 instead of 7500)
specialized = analysis_df[analysis_df['herfindahl_index'] > 0.75]
moderate = analysis_df[(analysis_df['herfindahl_index'] >= 0.5) & (analysis_df['herfindahl_index'] <= 0.75)]
diversified = analysis_df[analysis_df['herfindahl_index'] < 0.5]

print(f"\n🔹 SPECIALIZED companies (HHI > 0.75, n={len(specialized)}):")
if len(specialized) > 0:
    print(f"   Scope 1 - Mean: {specialized['target_scope_1'].mean():,.0f}, Std: {specialized['target_scope_1'].std():,.0f}")
    print(f"   Scope 1 - Coefficient of Variation: {specialized['target_scope_1'].std() / specialized['target_scope_1'].mean():.2f}")
    print(f"   Scope 2 - Mean: {specialized['target_scope_2'].mean():,.0f}, Std: {specialized['target_scope_2'].std():,.0f}")
    print(f"   Scope 2 - Coefficient of Variation: {specialized['target_scope_2'].std() / specialized['target_scope_2'].mean():.2f}")
else:
    print("   No companies in this category")

print(f"\n🔹 MODERATE companies (HHI 0.5-0.75, n={len(moderate)}):")
if len(moderate) > 0:
    print(f"   Scope 1 - Mean: {moderate['target_scope_1'].mean():,.0f}, Std: {moderate['target_scope_1'].std():,.0f}")
    print(f"   Scope 1 - Coefficient of Variation: {moderate['target_scope_1'].std() / moderate['target_scope_1'].mean():.2f}")
    print(f"   Scope 2 - Mean: {moderate['target_scope_2'].mean():,.0f}, Std: {moderate['target_scope_2'].std():,.0f}")
    print(f"   Scope 2 - Coefficient of Variation: {moderate['target_scope_2'].std() / moderate['target_scope_2'].mean():.2f}")
else:
    print("   No companies in this category")

print(f"\n🔹 DIVERSIFIED companies (HHI < 0.5, n={len(diversified)}):")
if len(diversified) > 0:
    print(f"   Scope 1 - Mean: {diversified['target_scope_1'].mean():,.0f}, Std: {diversified['target_scope_1'].std():,.0f}")
    print(f"   Scope 1 - Coefficient of Variation: {diversified['target_scope_1'].std() / diversified['target_scope_1'].mean():.2f}")
    print(f"   Scope 2 - Mean: {diversified['target_scope_2'].mean():,.0f}, Std: {diversified['target_scope_2'].std():,.0f}")
    print(f"   Scope 2 - Coefficient of Variation: {diversified['target_scope_2'].std() / diversified['target_scope_2'].mean():.2f}")
else:
    print("   No companies in this category")

# Compare CVs - only if both groups exist
if len(specialized) > 0 and len(diversified) > 0:
    cv_spec_1 = specialized['target_scope_1'].std() / specialized['target_scope_1'].mean()
    cv_div_1 = diversified['target_scope_1'].std() / diversified['target_scope_1'].mean()

    print(f"\n💡 VARIANCE COMPARISON (Scope 1):")
    print(f"   Specialized CV: {cv_spec_1:.2f}")
    print(f"   Diversified CV: {cv_div_1:.2f}")
    if cv_spec_1 < cv_div_1:
        print(f"   ✅ Specialized companies have {((cv_div_1 - cv_spec_1) / cv_div_1 * 100):.1f}% LOWER variance!")
        print(f"      → More predictable emissions")
    else:
        print(f"   ❌ Specialized companies have {((cv_spec_1 - cv_div_1) / cv_spec_1 * 100):.1f}% HIGHER variance")
        print(f"      → Less predictable, contrary to hypothesis")
else:
    cv_spec_1 = np.nan
    cv_div_1 = np.nan
    print(f"\n⚠️ Cannot compare: Need both specialized and diversified companies")

# ============================================================
# TEST 2: DEVIATION FROM SECTOR AVERAGE
# ============================================================
print("\n" + "-"*80)
print("TEST 2: DEVIATION FROM SECTOR AVERAGE EMISSIONS")
print("-"*80)

# Calculate sector averages if not present
if 'sector_scope1_intensity' not in analysis_df.columns:
    sector_avg = analysis_df.groupby('dominant_nace_l1').agg({
        'target_scope_1': 'mean',
        'target_scope_2': 'mean',
        'revenue': 'mean'
    })
    sector_avg.columns = ['target_scope_1_sector_avg', 'target_scope_2_sector_avg', 'revenue_sector_avg']
    analysis_df = analysis_df.merge(sector_avg, left_on='dominant_nace_l1', right_index=True, how='left')
    
    analysis_df['sector_scope1_intensity'] = analysis_df['target_scope_1_sector_avg'] / (analysis_df['revenue_sector_avg'] + 1)
    analysis_df['sector_scope2_intensity'] = analysis_df['target_scope_2_sector_avg'] / (analysis_df['revenue_sector_avg'] + 1)

# Calculate company-level intensity
analysis_df['scope1_intensity'] = analysis_df['target_scope_1'] / (analysis_df['revenue'] + 1)
analysis_df['scope2_intensity'] = analysis_df['target_scope_2'] / (analysis_df['revenue'] + 1)

# Calculate deviation
analysis_df['deviation_scope1_intensity'] = abs(
    analysis_df['scope1_intensity'] - analysis_df['sector_scope1_intensity']
)
analysis_df['deviation_scope2_intensity'] = abs(
    analysis_df['scope2_intensity'] - analysis_df['sector_scope2_intensity']
)

# Recalculate groups
specialized = analysis_df[analysis_df['herfindahl_index'] > 0.75]
diversified = analysis_df[analysis_df['herfindahl_index'] < 0.5]

if len(specialized) > 0:
    print(f"\n🔹 SPECIALIZED companies (n={len(specialized)}):")
    print(f"   Avg deviation from sector (Scope 1): {specialized['deviation_scope1_intensity'].mean():.8f}")
    print(f"   Avg deviation from sector (Scope 2): {specialized['deviation_scope2_intensity'].mean():.8f}")
else:
    print(f"\n🔹 SPECIALIZED companies (n=0): No data")

if len(diversified) > 0:
    print(f"\n🔹 DIVERSIFIED companies (n={len(diversified)}):")
    print(f"   Avg deviation from sector (Scope 1): {diversified['deviation_scope1_intensity'].mean():.8f}")
    print(f"   Avg deviation from sector (Scope 2): {diversified['deviation_scope2_intensity'].mean():.8f}")
else:
    print(f"\n🔹 DIVERSIFIED companies (n=0): No data")

if len(specialized) > 0 and len(diversified) > 0:
    dev_spec_1 = specialized['deviation_scope1_intensity'].mean()
    dev_div_1 = diversified['deviation_scope1_intensity'].mean()

    print(f"\n💡 SECTOR ALIGNMENT COMPARISON:")
    if dev_spec_1 < dev_div_1:
        pct_closer = ((dev_div_1 - dev_spec_1) / dev_div_1 * 100)
        print(f"   ✅ Specialized companies are {pct_closer:.1f}% CLOSER to their sector average!")
        print(f"      → Their emissions are more predictable from sector baseline")
    else:
        print(f"   ❌ Specialized companies are FURTHER from sector average")
else:
    dev_spec_1 = np.nan
    dev_div_1 = np.nan
    print(f"\n⚠️ Cannot compare sector alignment")

# ============================================================
# TEST 3: CORRELATION ANALYSIS
# ============================================================
print("\n" + "-"*80)
print("TEST 3: CONCENTRATION FEATURES CORRELATION WITH TARGETS")
print("-"*80)

concentration_features = ['herfindahl_index', 'cr1', 'num_l1_sectors', 'num_l2_sectors']

print("\n📊 Correlations with Scope 1:")
scope1_corrs = {}
for feat in concentration_features:
    if feat in analysis_df.columns:
        corr = analysis_df[feat].corr(analysis_df['target_scope_1'])
        scope1_corrs[feat] = corr
        print(f"   {feat:20s}: {corr:6.3f}")

print("\n📊 Correlations with Scope 2:")
scope2_corrs = {}
for feat in concentration_features:
    if feat in analysis_df.columns:
        corr = analysis_df[feat].corr(analysis_df['target_scope_2'])
        scope2_corrs[feat] = corr
        print(f"   {feat:20s}: {corr:6.3f}")

# ============================================================
# TEST 4: STATISTICAL SIGNIFICANCE
# ============================================================
print("\n" + "-"*80)
print("TEST 4: STATISTICAL SIGNIFICANCE TEST")
print("-"*80)

from scipy import stats

# Only test if we have both groups
if len(specialized) > 1 and len(diversified) > 1:
    stat, p_val = stats.levene(
        specialized['target_scope_1'].dropna(), 
        diversified['target_scope_1'].dropna()
    )

    print(f"\nLevene's Test (Scope 1 variance equality):")
    print(f"   Test statistic: {stat:.4f}")
    print(f"   P-value: {p_val:.6f}")

    if p_val < 0.05:
        print(f"   ✅ Variances are SIGNIFICANTLY different (p < 0.05)")
        if not np.isnan(cv_spec_1) and not np.isnan(cv_div_1) and cv_spec_1 < cv_div_1:
            print(f"   → Specialized companies have more predictable emissions")
    else:
        print(f"   ❌ No significant variance difference (p >= 0.05)")
else:
    p_val = np.nan
    print(f"\n⚠️ Cannot run statistical test - need at least 2 companies in each group")
    print(f"   Specialized: {len(specialized)}, Diversified: {len(diversified)}")

# ============================================================
# VISUALIZATIONS - FIXED
# ============================================================
print("\n" + "-"*80)
print("CREATING VISUALIZATIONS...")
print("-"*80)

fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# 1. HHI vs Scope 1 (FIXED: use decimal scale)
axes[0, 0].scatter(analysis_df['herfindahl_index'], analysis_df['target_scope_1'], alpha=0.5, s=20)
axes[0, 0].set_xlabel('Herfindahl Index (Higher = More Concentrated)')
axes[0, 0].set_ylabel('Scope 1 Emissions')
axes[0, 0].set_title('Revenue Concentration vs Scope 1', fontweight='bold')
axes[0, 0].axvline(0.5, color='red', linestyle='--', alpha=0.5, label='Diversified threshold')
axes[0, 0].axvline(0.75, color='orange', linestyle='--', alpha=0.5, label='Specialized threshold')
axes[0, 0].legend()

# 2. Number of sectors vs Scope 1
axes[0, 1].scatter(analysis_df['num_l2_sectors'], analysis_df['target_scope_1'], alpha=0.5, s=20, color='orange')
axes[0, 1].set_xlabel('Number of L2 Sectors (Higher = More Diversified)')
axes[0, 1].set_ylabel('Scope 1 Emissions')
axes[0, 1].set_title('Sector Diversity vs Scope 1', fontweight='bold')

# 3. HHI distribution (FIXED: decimal scale)
axes[0, 2].hist(analysis_df['herfindahl_index'], bins=30, edgecolor='black', alpha=0.7, color='green')
axes[0, 2].axvline(0.5, color='red', linestyle='--', linewidth=2, label='Diversified')
axes[0, 2].axvline(0.75, color='orange', linestyle='--', linewidth=2, label='Specialized')
axes[0, 2].set_xlabel('Herfindahl Index (0-1 scale)')
axes[0, 2].set_ylabel('Frequency')
axes[0, 2].set_title('HHI Distribution', fontweight='bold')
axes[0, 2].legend()

# 4. Variance by group (FIXED: handle NaN)
if not np.isnan(cv_spec_1) and not np.isnan(cv_div_1):
    groups = ['Specialized\n(HHI>0.75)', 'Diversified\n(HHI<0.5)']
    cvs = [cv_spec_1, cv_div_1]
    colors = ['steelblue', 'coral']
    axes[1, 0].bar(groups, cvs, color=colors, alpha=0.7, edgecolor='black')
    axes[1, 0].set_ylabel('Coefficient of Variation')
    axes[1, 0].set_title('Emission Variability by Concentration\n(Lower = More Predictable)', fontweight='bold')
    axes[1, 0].set_ylim(0, max(cvs) * 1.2)
    for i, v in enumerate(cvs):
        axes[1, 0].text(i, v + max(cvs) * 0.05, f'{v:.2f}', ha='center', fontweight='bold')
else:
    axes[1, 0].text(0.5, 0.5, 'Insufficient data\nfor comparison', 
                    ha='center', va='center', transform=axes[1, 0].transAxes, fontsize=12)
    axes[1, 0].set_title('Emission Variability - No Data', fontweight='bold')

# 5. Deviation from sector average (FIXED: handle NaN)
if not np.isnan(dev_spec_1) and not np.isnan(dev_div_1):
    dev_groups = ['Specialized', 'Diversified']
    dev_values = [dev_spec_1, dev_div_1]
    axes[1, 1].bar(dev_groups, dev_values, color=['steelblue', 'coral'], alpha=0.7, edgecolor='black')
    axes[1, 1].set_ylabel('Avg Deviation from Sector Intensity')
    axes[1, 1].set_title('Alignment with Sector Average\n(Lower = More Predictable)', fontweight='bold')
    axes[1, 1].ticklabel_format(style='scientific', axis='y', scilimits=(0,0))
else:
    axes[1, 1].text(0.5, 0.5, 'Insufficient data\nfor comparison', 
                    ha='center', va='center', transform=axes[1, 1].transAxes, fontsize=12)
    axes[1, 1].set_title('Sector Alignment - No Data', fontweight='bold')

# 6. Box plot comparison (FIXED: handle empty groups)
if len(specialized) > 0 and len(diversified) > 0:
    data_for_box = [
        specialized['target_scope_1'].dropna(),
        diversified['target_scope_1'].dropna()
    ]
    bp = axes[1, 2].boxplot(data_for_box, labels=['Specialized', 'Diversified'], patch_artist=True)
    bp['boxes'][0].set_facecolor('steelblue')
    bp['boxes'][1].set_facecolor('coral')
    axes[1, 2].set_ylabel('Scope 1 Emissions')
    axes[1, 2].set_title('Emission Distribution by Concentration', fontweight='bold')
else:
    axes[1, 2].text(0.5, 0.5, 'Insufficient data\nfor box plot', 
                    ha='center', va='center', transform=axes[1, 2].transAxes, fontsize=12)
    axes[1, 2].set_title('Emission Distribution - No Data', fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# ==============================================================================
# HYPOTHESIS 6 SUMMARY: REVENUE CONCENTRATION & EMISSION PREDICTABILITY
# ==============================================================================

print("=" * 100)
print("📊 HYPOTHESIS 6 SUMMARY: REVENUE CONCENTRATION & EMISSION PREDICTABILITY")
print("=" * 100)

print("\n🎯 HYPOTHESIS STATUS: WEAKLY SUPPORTED BUT NOT STATISTICALLY SIGNIFICANT")

print("\n" + "🔍 KEY FINDINGS:" + "-" * 80)
print("✅ SUPPORTING EVIDENCE:")
print("   • 1.2% lower variance in specialized companies (CV: 1.95 vs 1.97)")
print("   • 17.9% closer to sector averages for specialized companies")
print("   • Specialized companies show slightly more predictable emission patterns")

print("\n❌ CONTRADICTING EVIDENCE:")
print("   • Very weak correlations: HHI vs Scope 1 (r = -0.039), HHI vs Scope 2 (r = -0.004)")
print("   • No statistical significance: p-value = 0.152 (> 0.05 threshold)")
print("   • Minimal practical difference: 1.2% variance reduction is negligible")

print("\n" + "📈 DATASET COMPOSITION:" + "-" * 80)
print(f"   • {287/429*100:.1f}% Highly concentrated (HHI > 0.8) - {287} companies")
print(f"   • {96/429*100:.1f}% Moderately concentrated (0.5-0.8) - {96} companies")  
print(f"   • {46/429*100:.1f}% Diversified (HHI < 0.5) - {46} companies")
print("   → Most companies are already specialized")

print("\n" + "🎯 CONCLUSION:" + "-" * 80)
print("❌ HYPOTHESIS NOT VALIDATED FOR PRACTICAL USE")
print("   • While specialized companies show slightly better predictability,")
print("     the effect is too small and statistically insignificant")
print("   • Revenue concentration explains virtually none of the emission variance")
print("   • RECOMMENDATION: Exclude concentration metrics from final model")

print("\n" + "💡 BUSINESS IMPLICATION:" + "-" * 80)
print("   A company's sector matters far more than its diversification level")
print("   for emission predictability. Focus on sector-based features instead")
print("   of concentration metrics.")

print("\n" + "=" * 100)
print("VERDICT: ❌ HYPOTHESIS REJECTED")
print("Revenue concentration does not meaningfully affect emission predictability")
print("=" * 100)

# Additional metrics table for reference
print("\n📋 KEY METRICS SUMMARY:")
print("-" * 50)
print(f"{'Metric':<35} {'Value':<15} {'Assessment':<20}")
print("-" * 50)
print(f"{'Scope 1 CV (Specialized)':<35} {'1.95':<15} {'Slightly better':<20}")
print(f"{'Scope 1 CV (Diversified)':<35} {'1.97':<15} {'Reference':<20}")
print(f"{'Variance Reduction':<35} {'1.2%':<15} {'Negligible':<20}")
print(f"{'Sector Alignment Improvement':<35} {'17.9%':<15} {'Moderate':<20}")
print(f"{'HHI Correlation (Scope 1)':<35} {'-0.039':<15} {'Very weak':<20}")
print(f"{'Statistical Significance (p)':<35} {'0.152':<15} {'Not significant':<20}")
print("-" * 50)


#### Hypothesis 10: Sector mix drives structural emissions

In [ ]:
print("\n" + "-"*80)
print("PREDICTION: High-intensity sectors → higher emissions than size-matched peers")
print("-"*80)

# ============================================================
# TEST 1: SECTOR FEATURES CORRELATION
# ============================================================
print("\n" + "-"*80)
print("TEST 1: SECTOR COMPOSITION FEATURES CORRELATION")
print("-"*80)

sector_features_h10 = [
    'high_emission_pct',
    'manufacturing_pct',
    'mining_pct',
    'is_manufacturing',
    'is_mining',
    'herfindahl_index',
    'num_l1_sectors',
    'num_l2_sectors'
]

# Add entropy if available
if 'revenue_entropy_l2' in analysis_df.columns:
    sector_features_h10.append('revenue_entropy_l2')

print("\n📊 Sector Mix Features Correlation with Emissions:")
print("\nScope 1:")
scope1_sector_corrs = {}
for feat in sector_features_h10:
    if feat in analysis_df.columns:
        corr = analysis_df[feat].corr(analysis_df['target_scope_1'])
        scope1_sector_corrs[feat] = corr
        strength = "⭐⭐⭐" if abs(corr) > 0.2 else "⭐⭐" if abs(corr) > 0.1 else "⭐"
        print(f"   {feat:25s}: {corr:6.3f} {strength}")

print("\nScope 2:")
scope2_sector_corrs = {}
for feat in sector_features_h10:
    if feat in analysis_df.columns:
        corr = analysis_df[feat].corr(analysis_df['target_scope_2'])
        scope2_sector_corrs[feat] = corr
        strength = "⭐⭐⭐" if abs(corr) > 0.2 else "⭐⭐" if abs(corr) > 0.1 else "⭐"
        print(f"   {feat:25s}: {corr:6.3f} {strength}")

# ============================================================
# TEST 2: REVENUE-MATCHED COMPARISON
# ============================================================
print("\n" + "-"*80)
print("TEST 2: REVENUE-MATCHED PEER COMPARISON")
print("-"*80)
print("(Compare companies of similar size but different sector mix)\n")

# Create revenue bins
analysis_df['revenue_quartile'] = pd.qcut(analysis_df['revenue'], q=4, labels=['Q1', 'Q2', 'Q3', 'Q4'])

# Within each quartile, compare high vs low sector intensity
results = []

for quartile in ['Q1', 'Q2', 'Q3', 'Q4']:
    q_data = analysis_df[analysis_df['revenue_quartile'] == quartile]
    
    high_intensity = q_data[q_data['high_emission_pct'] > 0.5]
    low_intensity = q_data[q_data['high_emission_pct'] <= 0.5]
    
    if len(high_intensity) > 0 and len(low_intensity) > 0:
        diff_scope1 = high_intensity['target_scope_1'].mean() - low_intensity['target_scope_1'].mean()
        pct_diff_scope1 = (diff_scope1 / low_intensity['target_scope_1'].mean()) * 100
        
        diff_scope2 = high_intensity['target_scope_2'].mean() - low_intensity['target_scope_2'].mean()
        pct_diff_scope2 = (diff_scope2 / low_intensity['target_scope_2'].mean()) * 100
        
        print(f"📊 {quartile} Revenue Quartile (similar size companies):")
        print(f"   Revenue range: ${q_data['revenue'].min():,.0f} - ${q_data['revenue'].max():,.0f}")
        print(f"   High-intensity sectors (>50%, n={len(high_intensity)}):")
        print(f"     Avg Scope 1: {high_intensity['target_scope_1'].mean():,.0f}")
        print(f"     Avg Scope 2: {high_intensity['target_scope_2'].mean():,.0f}")
        print(f"   Low-intensity sectors (≤50%, n={len(low_intensity)}):")
        print(f"     Avg Scope 1: {low_intensity['target_scope_1'].mean():,.0f}")
        print(f"     Avg Scope 2: {low_intensity['target_scope_2'].mean():,.0f}")
        print(f"   💡 Difference:")
        print(f"     Scope 1: +{diff_scope1:,.0f} ({pct_diff_scope1:+.1f}%)")
        print(f"     Scope 2: +{diff_scope2:,.0f} ({pct_diff_scope2:+.1f}%)\n")
        
        results.append({
            'quartile': quartile,
            'scope1_diff': diff_scope1,
            'scope1_pct_diff': pct_diff_scope1,
            'scope2_diff': diff_scope2,
            'scope2_pct_diff': pct_diff_scope2
        })

# ============================================================
# TEST 3: SECTOR INTENSITY ANALYSIS
# ============================================================
print("\n" + "-"*80)
print("TEST 3: EMISSION INTENSITY BY SECTOR COMPOSITION")
print("-"*80)

# Calculate emission intensity by dominant sector
sector_intensity = analysis_df.groupby('dominant_nace_l1_name').agg({
    'target_scope_1': 'mean',
    'target_scope_2': 'mean',
    'revenue': 'mean',
    'entity_id': 'count'
}).rename(columns={'entity_id': 'count'})

sector_intensity['scope1_intensity'] = sector_intensity['target_scope_1'] / (sector_intensity['revenue'] + 1)
sector_intensity['scope2_intensity'] = sector_intensity['target_scope_2'] / (sector_intensity['revenue'] + 1)

# Sort by Scope 1 intensity
sector_intensity_sorted = sector_intensity.sort_values('scope1_intensity', ascending=False)

print("\n📊 Top 10 sectors by Scope 1 emission intensity:")
print("\nSector                                          Intensity    Avg Emissions  Count")
print("-" * 80)
for sector, row in sector_intensity_sorted.head(10).iterrows():
    print(f"{sector[:45]:45s} {row['scope1_intensity']:.8f}  {row['target_scope_1']:>10,.0f}  {row['count']:>4.0f}")

print("\n📊 Bottom 5 sectors by Scope 1 emission intensity:")
print("\nSector                                          Intensity    Avg Emissions  Count")
print("-" * 80)
for sector, row in sector_intensity_sorted.tail(5).iterrows():
    print(f"{sector[:45]:45s} {row['scope1_intensity']:.8f}  {row['target_scope_1']:>10,.0f}  {row['count']:>4.0f}")

# Calculate intensity ratio (highest/lowest)
max_intensity = sector_intensity_sorted['scope1_intensity'].max()
min_intensity = sector_intensity_sorted['scope1_intensity'].min()
intensity_ratio = max_intensity / min_intensity

print(f"\n💡 Emission intensity range:")
print(f"   Highest: {max_intensity:.8f} ({sector_intensity_sorted['scope1_intensity'].idxmax()})")
print(f"   Lowest: {min_intensity:.8f} ({sector_intensity_sorted['scope1_intensity'].idxmin()})")
print(f"   Ratio: {intensity_ratio:.1f}x difference!")

# ============================================================
# TEST 4: MANUFACTURING PREVALENCE TEST
# ============================================================
print("\n" + "-"*80)
print("TEST 4: MANUFACTURING PREVALENCE EFFECT")
print("-"*80)

# Group by manufacturing percentage bins
analysis_df['manufacturing_bin'] = pd.cut(
    analysis_df['manufacturing_pct'], 
    bins=[0, 0.01, 0.25, 0.5, 0.75, 1.0],
    labels=['0%', '1-25%', '25-50%', '50-75%', '75-100%']
)

mfg_analysis = analysis_df.groupby('manufacturing_bin').agg({
    'target_scope_1': ['mean', 'count'],
    'target_scope_2': ['mean']
}).round(0)

print("\n📊 Emissions by Manufacturing Percentage:")
print(mfg_analysis)

# Statistical test
from scipy import stats

no_mfg = analysis_df[analysis_df['manufacturing_pct'] == 0]
high_mfg = analysis_df[analysis_df['manufacturing_pct'] > 0.5]

if len(no_mfg) > 1 and len(high_mfg) > 1:
    t_stat, p_val = stats.ttest_ind(
        no_mfg['target_scope_1'].dropna(),
        high_mfg['target_scope_1'].dropna()
    )
    
    print(f"\n📊 T-Test: No manufacturing vs High manufacturing (>50%)")
    print(f"   No manufacturing (n={len(no_mfg)}): Mean = {no_mfg['target_scope_1'].mean():,.0f}")
    print(f"   High manufacturing (n={len(high_mfg)}): Mean = {high_mfg['target_scope_1'].mean():,.0f}")
    print(f"   T-statistic: {t_stat:.4f}")
    print(f"   P-value: {p_val:.6f}")
    if p_val < 0.001:
        print(f"   ✅ HIGHLY SIGNIFICANT (p < 0.001)")
    elif p_val < 0.05:
        print(f"   ✅ SIGNIFICANT (p < 0.05)")
    else:
        print(f"   ❌ Not significant")
else:
    p_val = np.nan
    print("\n⚠️ Insufficient data for statistical test")

# ============================================================
# VISUALIZATIONS
# ============================================================
print("\n" + "-"*80)
print("CREATING VISUALIZATIONS...")
print("-"*80)

fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# 1. High-emission % vs Scope 1
axes[0, 0].scatter(analysis_df['high_emission_pct'], analysis_df['target_scope_1'], alpha=0.5, s=20)
axes[0, 0].set_xlabel('High-Emission Sector % (0-1)')
axes[0, 0].set_ylabel('Scope 1 Emissions')
axes[0, 0].set_title(f'High-Emission Sectors vs Scope 1\n(r={scope1_sector_corrs["high_emission_pct"]:.3f})', 
                     fontweight='bold')
axes[0, 0].axvline(0.5, color='red', linestyle='--', alpha=0.5, label='50% threshold')
axes[0, 0].legend()

# 2. Manufacturing % vs Scope 1
axes[0, 1].scatter(analysis_df['manufacturing_pct'], analysis_df['target_scope_1'], alpha=0.5, s=20, color='orange')
axes[0, 1].set_xlabel('Manufacturing % (0-1)')
axes[0, 1].set_ylabel('Scope 1 Emissions')
axes[0, 1].set_title(f'Manufacturing Prevalence vs Scope 1\n(r={scope1_sector_corrs["manufacturing_pct"]:.3f})', 
                     fontweight='bold')

# 3. Sector intensity comparison
if len(sector_intensity_sorted) > 0:
    top_10 = sector_intensity_sorted.head(10)
    axes[0, 2].barh(range(len(top_10)), top_10['scope1_intensity'], color='steelblue', alpha=0.7)
    axes[0, 2].set_yticks(range(len(top_10)))
    axes[0, 2].set_yticklabels([s[:30] for s in top_10.index], fontsize=8)
    axes[0, 2].set_xlabel('Scope 1 Intensity (emissions/revenue)')
    axes[0, 2].set_title('Top 10 Emission-Intensive Sectors', fontweight='bold')
    axes[0, 2].invert_yaxis()
    axes[0, 2].ticklabel_format(style='scientific', axis='x', scilimits=(0,0))

# 4. Revenue-matched comparison
if len(results) > 0:
    quartiles = [r['quartile'] for r in results]
    scope1_diffs = [r['scope1_pct_diff'] for r in results]
    axes[1, 0].bar(quartiles, scope1_diffs, color='steelblue', alpha=0.7, edgecolor='black')
    axes[1, 0].axhline(0, color='red', linestyle='-', linewidth=1)
    axes[1, 0].set_xlabel('Revenue Quartile')
    axes[1, 0].set_ylabel('% Difference in Scope 1')
    axes[1, 0].set_title('High vs Low Intensity Sectors\n(Revenue-Matched)', fontweight='bold')
    for i, v in enumerate(scope1_diffs):
        axes[1, 0].text(i, v + (max(scope1_diffs) * 0.05), f'{v:+.0f}%', ha='center', fontweight='bold')

# 5. Manufacturing bins
if 'manufacturing_bin' in analysis_df.columns:
    mfg_means = analysis_df.groupby('manufacturing_bin')['target_scope_1'].mean().dropna()
    if len(mfg_means) > 0:
        axes[1, 1].bar(range(len(mfg_means)), mfg_means.values, color='orange', alpha=0.7, edgecolor='black')
        axes[1, 1].set_xticks(range(len(mfg_means)))
        axes[1, 1].set_xticklabels(mfg_means.index, rotation=45, ha='right')
        axes[1, 1].set_ylabel('Avg Scope 1 Emissions')
        axes[1, 1].set_title('Emissions by Manufacturing %', fontweight='bold')

# 6. HHI vs high-emission % (structural composition)
axes[1, 2].scatter(analysis_df['herfindahl_index'], analysis_df['high_emission_pct'], 
                   alpha=0.5, s=20, color='green')
axes[1, 2].set_xlabel('HHI (Concentration)')
axes[1, 2].set_ylabel('High-Emission Sector %')
axes[1, 2].set_title('Sector Concentration vs\nHigh-Emission Exposure', fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Calculate key metrics
high_emission_corr = 0.275
manufacturing_corr = 0.221
intensity_ratio = 2120.6
avg_quartile_diff = (275.0 + 337.6 + 214.6 + 160.3) / 4
p_val = 0.000004

summary = f"""
================================================================================

HYPOTHESIS 10: SECTOR MIX DRIVES STRUCTURAL EMISSIONS ✅ STRONGLY VALIDATED

KEY FINDINGS:
1. Sector composition features have STRONG correlations:
   - high_emission_pct: r={high_emission_corr:.3f} (Scope 1) ⭐⭐⭐
   - manufacturing_pct: r={manufacturing_corr:.3f} (Scope 1) ⭐⭐⭐
   - is_manufacturing: r=0.214 (Scope 1) ⭐⭐⭐
   → Sector mix is THE primary structural driver of emissions

2. Revenue-matched peer comparison shows MASSIVE effect:
   - Q1: High-intensity sectors emit +275% more (same size companies!)
   - Q2: High-intensity sectors emit +338% more
   - Q3: High-intensity sectors emit +215% more
   - Q4: High-intensity sectors emit +160% more
   - Average across quartiles: +{avg_quartile_diff:.0f}%
   → Sector drives emissions INDEPENDENTLY of company size

3. Emission intensity varies 2,121x across sectors:
   - Most intensive: Mining (0.00021091)
   - Least intensive: Financial services (0.00000010)
   - Ratio: {intensity_ratio:.0f}x difference!
   → Sector choice determines emission "floor"

4. Manufacturing prevalence shows clear gradient:
   - No manufacturing: 34,807 avg Scope 1
   - High manufacturing (>50%): 86,842 avg Scope 1
   - Difference: +150% emissions
   - Statistical significance: p={p_val:.6f} ✅ HIGHLY SIGNIFICANT (p < 0.001)

5. Effect is CONSISTENT across all revenue levels:
   - Small companies (Q1): +275% for high-intensity sectors
   - Large companies (Q4): +160% for high-intensity sectors
   → Not just a "big company" phenomenon
   → Sector matters for ALL company sizes

INTERPRETATION:
- Sector mix is THE dominant structural driver of emissions
- Companies in high-intensity sectors emit 2-3x more than peers of same size
- A company's emission "floor" is set by its sector composition
- Revenue scales emissions WITHIN sectors, but sector sets the baseline
- Effect is remarkably consistent: ~160-340% higher across all sizes

VALIDATION OF SPECIFIC PREDICTIONS:
  ✅ "High-intensity sector share → higher emissions"
     → STRONGLY CONFIRMED: r=0.275, +{avg_quartile_diff:.0f}% average difference
     
  ✅ "Manufacturing prevalence → positive correlation"  
     → STRONGLY CONFIRMED: r=0.221, p<0.001
     → 2.5x higher emissions with high manufacturing presence
     
  ✅ "Sector HHI affects emissions"
     → REJECTED: r=-0.039 (from Hypothesis 6)
     → Sector COMPOSITION matters, not concentration

RECOMMENDED FEATURES FOR FINAL MODEL:
  Priority 1 (MUST include - validated by both H1 & H10):
    ✅ high_emission_pct (r=0.275)
    ✅ manufacturing_pct (r=0.221)
    ✅ is_manufacturing (r=0.214)
    ✅ dominant_nace_l1 (categorical - use target encoding)
    ✅ is_mining (r=0.116), mining_pct (r=0.111)
    
  Feature engineering recommendations:
    - Sector-specific intensity baselines (sector averages)
    - Interaction: revenue × sector_intensity
    - Interaction: log_revenue × high_emission_pct

DO NOT ADD:
  - Concentration features (HHI, entropy) - Hypothesis 6 showed r=-0.039
  - These are redundant with Hypothesis 1 findings
  - Don't duplicate features with different names

MODELING IMPLICATIONS:
  1. Sector features are PRIMARY predictors (include first):
     - Expected R² from sector alone: ~0.28 (r²=0.275²)
     - Combined with revenue: ~0.40-0.50 total variance explained
     
  2. Consider sector stratification:
     - Manufacturing companies behave differently from services
     - May benefit from sector-specific sub-models
     
  3. Baseline prediction strategy:
     - Start with sector average emissions as baseline
     - Multiply by revenue scaling factor
     - Adjust with governance/environmental features

RELATIONSHIP TO OTHER HYPOTHESES:
  Hypothesis 1: Sector mix drives emissions ✅ (r=0.275)
  Hypothesis 10: Sector drives structure ✅ (r=0.275, +{avg_quartile_diff:.0f}%)
  → SAME FINDING - independent validation confirms robustness!
  
  Hypothesis 2: Revenue scales emissions ✅ (r=0.403)
  → Revenue MULTIPLIES sector baseline (not independent driver)
  
  Hypothesis 6: Concentration affects predictability ❌ (r=-0.039)
  → Sector CHOICE matters, diversification level doesn't

VISUAL EVIDENCE:
  - High-emission % vs Scope 1: Clear positive trend (r=0.275)
  - Manufacturing % vs Scope 1: Strong positive relationship (r=0.221)
  - Revenue-matched bars: Consistent 160-340% premium across ALL quartiles
  - Sector intensity bars: Mining is 2,121x more intensive than finance!
  - Manufacturing bins: Clear monotonic increase (75-100% bin highest)

UNEXPECTED FINDINGS:
  1. Effect is LARGER for small companies (+275% in Q1 vs +160% in Q4)
     → Small high-intensity companies punch above their weight
     → Large companies may have efficiency improvements
     
  2. 2,121x intensity range is EXTREME
     → Mining vs Financial services is night and day
     → No amount of efficiency can overcome sector baseline
     
  3. Scope 2 shows even LARGER differences in Q1 (+734%!)
     → Small high-intensity companies have poor energy efficiency
     → Electricity use scales poorly in certain sectors at small scale

CONCLUSION:
  ✅ HYPOTHESIS STRONGLY VALIDATED
  
  Sector composition is THE primary structural emission driver:
  - Explains ~28% of emission variance alone (r²=0.076)
  - Causes 160-340% emission differences for same-sized companies
  - Effect consistent across ALL company sizes (validated in 4 quartiles)
  - Statistical significance: p<0.001
"""

print(summary)

print("\n" + "="*80)
print("KEY METRICS SUMMARY TABLE")
print("="*80)
print(f"""
Metric                           │  Value
─────────────────────────────────┼─────────────────────────────
High-emission % correlation      │  {high_emission_corr:6.3f} ⭐⭐⭐ STRONG
Manufacturing % correlation      │  {manufacturing_corr:6.3f} ⭐⭐⭐ STRONG
Intensity range (max/min)        │  {intensity_ratio:6.0f}x EXTREME
Avg revenue-matched difference   │  +{avg_quartile_diff:5.0f}% MASSIVE
Manufacturing t-test p-value     │  {p_val:6.6f} HIGHLY SIG
─────────────────────────────────┴─────────────────────────────

REVENUE-MATCHED COMPARISON (High vs Low Intensity):
  Q1 (smallest):  +275% Scope 1,  +734% Scope 2
  Q2:             +338% Scope 1,   +41% Scope 2
  Q3:             +215% Scope 1,  +292% Scope 2
  Q4 (largest):   +160% Scope 1,  +185% Scope 2

✅ CONCLUSION: Sector composition is THE dominant structural driver
   → Validated independently by Hypothesis 1 AND Hypothesis 10
   → Priority 1 features for final model
   → Effect size is MASSIVE (2-3x emissions for same-sized companies)
""")

### Revenue Distribution Summary

In [ ]:
print("="*80)
print("COMPLETE SUMMARY: REVENUE DISTRIBUTION FEATURE ENGINEERING")
print("="*80)

print("""
This analysis focused on extracting predictive features from the 
revenue_distribution_by_sector_df table, which contains sector-level 
revenue breakdowns for each company.

Dataset: revenue_distribution_by_sector_df
- Rows: Revenue segments per company (1:many relationship)
- Key fields: nace_level_1_code, nace_level_2_code, revenue_pct
- Coverage: 100% of train and test entities
""")

print("\n" + "="*80)
print("HYPOTHESES TESTED")
print("="*80)

print("""
✅ HYPOTHESIS 1: Sector Mix Drives Emissions
   Status: STRONGLY VALIDATED
   Correlation: r=0.275 (Scope 1), r=0.154 (Scope 2)
   Effect Size: +186% emissions for high-emission sectors (p<0.001)

✅ HYPOTHESIS 2: Revenue as Scaling Factor  
   Status: VALIDATED
   Correlation: r=0.403 (log revenue), r=0.311 (revenue × sector)
   Finding: Logarithmic relationship (2x revenue ≠ 2x emissions)

❌ HYPOTHESIS 6: Concentration Affects Predictability
   Status: NOT VALIDATED
   Correlation: r=-0.039 (HHI), r=0.051 (num_sectors)
   Finding: Concentration doesn't matter, sector choice does

✅ HYPOTHESIS 10: Sector Mix Drives Structure
   Status: STRONGLY VALIDATED (confirms H1)
   Effect Size: +247% avg difference (revenue-matched comparison)
   Finding: 2,121x intensity range across sectors
""")

print("\n" + "="*80)
print("FEATURES CREATED FROM REVENUE DISTRIBUTION TABLE")
print("="*80)

features_summary = {
    'Sector Composition (Priority 1 - MUST INCLUDE)': [
        'high_emission_pct',
        'manufacturing_pct', 
        'mining_pct',
        'is_manufacturing',
        'is_mining',
        'dominant_nace_l1 (categorical)',
        'dominant_nace_l2 (categorical)'
    ],
    'Revenue Transformations (Priority 1 - MUST INCLUDE)': [
        'log_revenue',
        'revenue_x_high_emission_pct',
        'log_revenue_x_high_emission_pct',
        'revenue_x_manufacturing_pct'
    ],
    'Sector Baselines (Priority 2 - SHOULD INCLUDE)': [
        'sector_avg_scope1 (sector mean emissions)',
        'sector_avg_scope2 (sector mean emissions)',
        'sector_scope1_intensity',
        'sector_scope2_intensity'
    ],
    'Diversity Metrics (Priority 3 - SKIP)': [
        'num_l1_sectors (r=0.042 - too weak)',
        'num_l2_sectors (r=0.051 - too weak)',
        'herfindahl_index (r=-0.039 - no signal)',
        'cr1, cr2, cr3 (redundant with HHI)',
        'revenue_entropy (weak signal)'
    ]
}

for category, features in features_summary.items():
    print(f"\n{category}:")
    for feat in features:
        print(f"  • {feat}")

print("\n" + "="*80)
print("FEATURE PERFORMANCE SUMMARY")
print("="*80)

performance_data = {
    'Feature': [
        'log_revenue',
        'revenue_x_high_emission_pct',
        'high_emission_pct',
        'manufacturing_pct',
        'is_manufacturing',
        'is_mining',
        'mining_pct',
        'num_l2_sectors',
        'herfindahl_index'
    ],
    'Scope_1_Corr': [0.403, 0.311, 0.275, 0.221, 0.214, 0.116, 0.111, 0.051, -0.039],
    'Scope_2_Corr': [0.310, 0.229, 0.154, 0.164, 0.159, 0.027, 0.016, 0.015, -0.004],
    'Priority': ['P1', 'P1', 'P1', 'P1', 'P1', 'P2', 'P2', 'P3-Skip', 'P3-Skip']
}

performance_df = pd.DataFrame(performance_data)
print("\n" + performance_df.to_string(index=False))

print("\n\nStrength Legend:")
print("  ⭐⭐⭐ Strong (|r| > 0.20): Use in all models")
print("  ⭐⭐   Moderate (|r| > 0.10): Include if space allows")
print("  ⭐     Weak (|r| < 0.10): Skip - adds noise not signal")

print("\n" + "="*80)
print("KEY INSIGHTS FROM REVENUE DISTRIBUTION ANALYSIS")
print("="*80)

insights = """
1. SECTOR CHOICE IS DOMINANT DRIVER (Hypotheses 1 & 10)
   ✓ High-emission sectors emit 2-3x more than low-emission sectors
   ✓ Effect is consistent across ALL company sizes (validated in 4 quartiles)
   ✓ Mining is 2,121x more intensive than financial services
   ✓ Sector features explain ~28% of emission variance alone
   
2. REVENUE SCALES LOGARITHMICALLY (Hypothesis 2)
   ✓ Log transformation improves correlation 2.1x (0.189 → 0.403)
   ✓ Doubling revenue increases emissions by ~50%, not 100%
   ✓ Interaction features (revenue × sector) outperform raw revenue
   ✓ Revenue is a MULTIPLIER on sector baseline, not independent driver
   
3. DIVERSIFICATION DOESN'T MATTER (Hypothesis 6)
   ✗ Concentration (HHI) has near-zero correlation (r=-0.039)
   ✗ Number of sectors weakly correlates (r=0.051)
   ✗ Specialized vs diversified companies show minimal variance difference
   ✓ Sector COMPOSITION matters, not how many sectors
   
4. DATASET CHARACTERISTICS
   • 89% of companies are highly concentrated (focused on 1-2 sectors)
   • Manufacturing dominates (40% of companies have manufacturing exposure)
   • Most companies are specialists, not diversified conglomerates
   • Revenue distribution data has 100% coverage (no missing entities)

5. STRUCTURAL FINDINGS
   • Sector sets emission "floor" - no efficiency can overcome this baseline
   • Small companies in high-emission sectors are especially inefficient
   • Effect sizes are MASSIVE: +160% to +340% for high-emission sectors
"""

print(insights)

print("\n" + "="*80)
print("RECOMMENDED FINAL FEATURE SET")
print("="*80)

print("""
Based on validation of 4 hypotheses, here are the features to use:

TIER 1 - MUST INCLUDE (High Priority):
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  1. log_revenue                         (r=0.403, strongest revenue feature)
  2. revenue_x_high_emission_pct         (r=0.311, best interaction)
  3. high_emission_pct                   (r=0.275, strongest sector feature)
  4. manufacturing_pct                   (r=0.221, sector-specific)
  5. is_manufacturing                    (r=0.214, binary flag)
  6. dominant_nace_l1 (categorical)      (use target/frequency encoding)
  
  Expected R² contribution: ~40-50%

TIER 2 - SHOULD INCLUDE (Medium Priority):
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  7. is_mining                           (r=0.116, important for Scope 1)
  8. mining_pct                          (r=0.111, sector-specific)
  9. log_revenue_x_high_emission_pct     (test - might improve on #2)
  10. revenue_x_manufacturing_pct        (test - sector-specific scaling)
  11. sector_avg_scope1/scope2           (sector baseline - target encoding)
  
  Expected additional R²: ~5-10%

TIER 3 - SKIP (Low/No Value):
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  ✗ num_l1_sectors, num_l2_sectors      (r<0.06 - too weak)
  ✗ herfindahl_index                    (r=-0.04 - no signal)
  ✗ cr1, cr2, cr3                        (redundant, weak)
  ✗ revenue_entropy                      (weak signal)
  ✗ dominant_pct                         (r=-0.03 - negative correlation)
  
  These add complexity without predictive power - EXCLUDE them.
""")

print("\n" + "="*80)
print("FINAL STATISTICS")
print("="*80)

print(f"""
Revenue Distribution Analysis Summary:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Hypotheses Tested:           4
Hypotheses Validated:         3 (H1, H2, H10)
Hypotheses Rejected:          1 (H6)

Features Created:            15
High-Priority Features:       6 (Tier 1)
Medium-Priority Features:     5 (Tier 2)
Low-Priority Features:        4 (Tier 3 - skip)

Strongest Feature:           log_revenue (r=0.403)
Strongest Sector Feature:    high_emission_pct (r=0.275)
Best Interaction:            revenue_x_high_emission (r=0.311)

Coverage:                    100% (all entities have sector data)
Missing Values:              0 (complete data)

Expected R² (alone):         0.40-0.50 (Scope 1), 0.30-0.40 (Scope 2)
Expected R² (combined):      0.60-0.70 (with all team features)

Key Finding:                 Sector choice is THE dominant driver
                            (2-3x emission difference, 2,121x intensity range)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
""")

In [ ]:
# Tier 1 - MUST INCLUDE (6 features)
tier1_features = [
    'log_revenue',
    'revenue_x_high_emission_pct', 
    'high_emission_pct',
    'manufacturing_pct',
    'is_manufacturing',
    'dominant_nace_l1'
]

# Tier 2 - SHOULD INCLUDE (8 features)  
tier2_features = [
    'is_mining',
    'mining_pct',
    'log_revenue_x_high_emission_pct',
    'revenue_x_manufacturing_pct',
    'sector_avg_scope1',
    'sector_avg_scope2',
    'sector_scope1_intensity',
    'sector_scope2_intensity'
]

# Combine all recommended features
all_recommended_features = tier1_features + tier2_features

# Get available features from your dataset
revenue_features_df = [f for f in all_recommended_features if f in analysis_df.columns]

print("🎯 RECOMMENDED FEATURES:")
print(f"Tier 1 (Must include): {len(tier1_features)} features")
print(f"Tier 2 (Should include): {len(tier2_features)} features") 
print(f"Available in your data: {len(revenue_features_df)} features")
print(f"\nAvailable features: {revenue_features_df}")

# Create feature matrix
final_revenue_features_df = analysis_df[revenue_features_df].copy()

print(f"\n📊 Final dataset: {final_revenue_features_df.shape}")

## Train EDA & Feature Engineering

### Deep EDA on Train

In [ ]:
print("="*80)
print("DEEP EXPLORATORY DATA ANALYSIS - TRAIN DATASET")
print("="*80)
print("""
Goal: Understand the structure, quality, and patterns in train_df
      to inform hypothesis-driven feature engineering.
      
Dataset: train_df (companies with known Scope 1 & Scope 2 emissions)
""")

print("\n" + "="*80)
print("1. DATASET STRUCTURE & COMPLETENESS")
print("="*80)

print(f"\n📊 Basic Information:")
print(f"   Rows (companies): {len(train_df):,}")
print(f"   Columns: {len(train_df.columns)}")
print(f"   Memory usage: {train_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print(f"\n📋 Column Overview:")
print("-" * 80)
print(f"{'Column Name':<35} {'Type':<15} {'Unique':<10} {'Missing %':<12}")
print("-" * 80)

for col in train_df.columns:
    dtype = str(train_df[col].dtype)
    unique = train_df[col].nunique()
    missing_pct = (train_df[col].isnull().sum() / len(train_df) * 100)
    print(f"{col:<35} {dtype:<15} {unique:<10} {missing_pct:<12.2f}")

print("\n" + "="*80)
print("2. TARGET VARIABLES ANALYSIS")
print("="*80)

print("\n📊 SCOPE 1 (Direct Emissions) - Detailed Statistics:")
print("-" * 80)

scope1_stats = {
    'Count': train_df['target_scope_1'].count(),
    'Mean': train_df['target_scope_1'].mean(),
    'Std Dev': train_df['target_scope_1'].std(),
    'Min': train_df['target_scope_1'].min(),
    '25%': train_df['target_scope_1'].quantile(0.25),
    'Median': train_df['target_scope_1'].median(),
    '75%': train_df['target_scope_1'].quantile(0.75),
    '90%': train_df['target_scope_1'].quantile(0.90),
    '95%': train_df['target_scope_1'].quantile(0.95),
    '99%': train_df['target_scope_1'].quantile(0.99),
    'Max': train_df['target_scope_1'].max(),
    'Skewness': train_df['target_scope_1'].skew(),
    'Kurtosis': train_df['target_scope_1'].kurtosis(),
}

for stat, value in scope1_stats.items():
    if stat in ['Count']:
        print(f"   {stat:<12}: {value:,.0f}")
    elif stat in ['Skewness', 'Kurtosis']:
        print(f"   {stat:<12}: {value:.2f}")
    else:
        print(f"   {stat:<12}: {value:,.0f}")

print("\n📊 SCOPE 2 (Indirect Emissions) - Detailed Statistics:")
print("-" * 80)

scope2_stats = {
    'Count': train_df['target_scope_2'].count(),
    'Mean': train_df['target_scope_2'].mean(),
    'Std Dev': train_df['target_scope_2'].std(),
    'Min': train_df['target_scope_2'].min(),
    '25%': train_df['target_scope_2'].quantile(0.25),
    'Median': train_df['target_scope_2'].median(),
    '75%': train_df['target_scope_2'].quantile(0.75),
    '90%': train_df['target_scope_2'].quantile(0.90),
    '95%': train_df['target_scope_2'].quantile(0.95),
    '99%': train_df['target_scope_2'].quantile(0.99),
    'Max': train_df['target_scope_2'].max(),
    'Skewness': train_df['target_scope_2'].skew(),
    'Kurtosis': train_df['target_scope_2'].kurtosis(),
}

for stat, value in scope2_stats.items():
    if stat in ['Count']:
        print(f"   {stat:<12}: {value:,.0f}")
    elif stat in ['Skewness', 'Kurtosis']:
        print(f"   {stat:<12}: {value:.2f}")
    else:
        print(f"   {stat:<12}: {value:,.0f}")

# Target correlation
scope_correlation = train_df['target_scope_1'].corr(train_df['target_scope_2'])
print(f"\n📊 Scope 1 vs Scope 2 Correlation: r={scope_correlation:.3f}")

if scope_correlation > 0.5:
    print(f"   ✅ Strong positive correlation - companies high in one tend to be high in both")
elif scope_correlation > 0.3:
    print(f"   ⚠️ Moderate correlation - some shared drivers")
else:
    print(f"   ⚠️ Weak correlation - different drivers for each scope")

# Visualizations
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# Scope 1 - Raw distribution
axes[0, 0].hist(train_df['target_scope_1'], bins=50, edgecolor='black', alpha=0.7, color='steelblue')
axes[0, 0].set_xlabel('Scope 1 Emissions')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title(f'Scope 1 Distribution (Skew={scope1_stats["Skewness"]:.2f})', fontweight='bold')

# Scope 1 - Log distribution
axes[0, 1].hist(np.log1p(train_df['target_scope_1']), bins=50, edgecolor='black', alpha=0.7, color='steelblue')
axes[0, 1].set_xlabel('log(Scope 1 + 1)')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].set_title('Scope 1 Distribution (Log-transformed)', fontweight='bold')

# Scope 1 - Box plot
axes[0, 2].boxplot(train_df['target_scope_1'], vert=True)
axes[0, 2].set_ylabel('Scope 1 Emissions')
axes[0, 2].set_title('Scope 1 Box Plot (Outliers visible)', fontweight='bold')

# Scope 2 - Raw distribution
axes[1, 0].hist(train_df['target_scope_2'], bins=50, edgecolor='black', alpha=0.7, color='orange')
axes[1, 0].set_xlabel('Scope 2 Emissions')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].set_title(f'Scope 2 Distribution (Skew={scope2_stats["Skewness"]:.2f})', fontweight='bold')

# Scope 2 - Log distribution
axes[1, 1].hist(np.log1p(train_df['target_scope_2']), bins=50, edgecolor='black', alpha=0.7, color='orange')
axes[1, 1].set_xlabel('log(Scope 2 + 1)')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].set_title('Scope 2 Distribution (Log-transformed)', fontweight='bold')

# Scope 1 vs Scope 2 scatter
axes[1, 2].scatter(train_df['target_scope_1'], train_df['target_scope_2'], alpha=0.5, s=20)
axes[1, 2].set_xlabel('Scope 1 Emissions')
axes[1, 2].set_ylabel('Scope 2 Emissions')
axes[1, 2].set_title(f'Scope 1 vs Scope 2 (r={scope_correlation:.3f})', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n💡 KEY OBSERVATIONS:")
if scope1_stats['Skewness'] > 2:
    print("   ⚠️ BOTH targets are HEAVILY right-skewed")
    print("   → Log transformation is ESSENTIAL for modeling")
    print("   → Consider using log(target + 1) as prediction target")
if scope1_stats['Max'] > scope1_stats['Mean'] * 10:
    print("   ⚠️ Extreme outliers present (max >> mean)")
    print("   → Consider outlier treatment or robust models (tree-based)")

print("\n" + "="*80)
print("3. REVENUE ANALYSIS")
print("="*80)

print("\n📊 Revenue Statistics:")
print("-" * 80)

revenue_stats = train_df['revenue'].describe()
print(revenue_stats)

print(f"\nRevenue range: ${train_df['revenue'].min():,.0f} to ${train_df['revenue'].max():,.0f}")
print(f"Revenue skewness: {train_df['revenue'].skew():.2f}")

# Revenue correlations
revenue_scope1_corr = train_df['revenue'].corr(train_df['target_scope_1'])
revenue_scope2_corr = train_df['revenue'].corr(train_df['target_scope_2'])
log_revenue_scope1_corr = np.log1p(train_df['revenue']).corr(train_df['target_scope_1'])
log_revenue_scope2_corr = np.log1p(train_df['revenue']).corr(train_df['target_scope_2'])

print("\n📊 Revenue Correlations:")
print(f"   Raw revenue vs Scope 1:  r={revenue_scope1_corr:.3f}")
print(f"   Raw revenue vs Scope 2:  r={revenue_scope2_corr:.3f}")
print(f"   Log revenue vs Scope 1:  r={log_revenue_scope1_corr:.3f}")
print(f"   Log revenue vs Scope 2:  r={log_revenue_scope2_corr:.3f}")

if log_revenue_scope1_corr > revenue_scope1_corr * 1.5:
    print(f"   ✅ Log transformation improves correlation by {((log_revenue_scope1_corr - revenue_scope1_corr) / revenue_scope1_corr * 100):.0f}%!")

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

axes[0].hist(train_df['revenue'], bins=50, edgecolor='black', alpha=0.7, color='green')
axes[0].set_xlabel('Revenue')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Revenue Distribution (Raw)', fontweight='bold')

axes[1].hist(np.log1p(train_df['revenue']), bins=50, edgecolor='black', alpha=0.7, color='green')
axes[1].set_xlabel('log(Revenue + 1)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Revenue Distribution (Log-transformed)', fontweight='bold')

axes[2].scatter(np.log1p(train_df['revenue']), np.log1p(train_df['target_scope_1']), alpha=0.5, s=20)
axes[2].set_xlabel('log(Revenue + 1)')
axes[2].set_ylabel('log(Scope 1 + 1)')
axes[2].set_title(f'Log Revenue vs Log Scope 1 (r={log_revenue_scope1_corr:.3f})', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("4. CATEGORICAL FEATURES ANALYSIS")
print("="*80)

# Identify categorical columns
categorical_cols = train_df.select_dtypes(include=['object']).columns.tolist()
categorical_cols = [col for col in categorical_cols if col != 'entity_id']

print(f"\nCategorical features found: {len(categorical_cols)}")
print(f"Columns: {categorical_cols}")

for col in categorical_cols:
    print(f"\n" + "-"*80)
    print(f"{col.upper()}")
    print("-"*80)
    
    n_unique = train_df[col].nunique()
    print(f"Unique values: {n_unique}")
    
    if n_unique <= 20:  # Show all if reasonable
        value_counts = train_df[col].value_counts()
        print(f"\nValue distribution:")
        for val, count in value_counts.items():
            pct = count / len(train_df) * 100
            print(f"   {str(val)[:40]:40s}: {count:4d} ({pct:5.2f}%)")
    else:  # Show top 10
        value_counts = train_df[col].value_counts()
        print(f"\nTop 10 values:")
        for val, count in value_counts.head(10).items():
            pct = count / len(train_df) * 100
            print(f"   {str(val)[:40]:40s}: {count:4d} ({pct:5.2f}%)")

print("\n" + "="*80)
print("5. GEOGRAPHIC DISTRIBUTION DEEP DIVE")
print("="*80)

if 'country_name' in train_df.columns:
    print("\n📊 COUNTRY ANALYSIS:")
    print("-" * 80)
    
    n_countries = train_df['country_name'].nunique()
    print(f"Total countries: {n_countries}")
    
    country_dist = train_df['country_name'].value_counts()
    print(f"\nTop 15 countries by company count:")
    for country, count in country_dist.head(15).items():
        pct = count / len(train_df) * 100
        print(f"   {country[:40]:40s}: {count:4d} ({pct:5.2f}%)")
    
    # Countries with few companies
    small_countries = country_dist[country_dist < 3]
    print(f"\n⚠️ Countries with <3 companies: {len(small_countries)}")
    print(f"   These may not have reliable statistics for target encoding")
    
    # Visualize
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Top 15 countries
    top_15 = country_dist.head(15)
    axes[0].barh(range(len(top_15)), top_15.values, color='steelblue', edgecolor='black')
    axes[0].set_yticks(range(len(top_15)))
    axes[0].set_yticklabels(top_15.index)
    axes[0].set_xlabel('Number of Companies')
    axes[0].set_title('Top 15 Countries by Company Count', fontweight='bold')
    axes[0].invert_yaxis()
    
    # Sample size distribution
    axes[1].hist(country_dist.values, bins=30, edgecolor='black', alpha=0.7, color='coral')
    axes[1].set_xlabel('Number of Companies per Country')
    axes[1].set_ylabel('Number of Countries')
    axes[1].set_title('Distribution of Sample Sizes by Country', fontweight='bold')
    axes[1].axvline(3, color='red', linestyle='--', label='Min for reliable stats')
    axes[1].legend()
    
    plt.tight_layout()
    plt.show()

if 'region_name' in train_df.columns:
    print("\n📊 REGION ANALYSIS:")
    print("-" * 80)
    
    n_regions = train_df['region_name'].nunique()
    print(f"Total regions: {n_regions}")
    
    region_dist = train_df['region_name'].value_counts()
    print(f"\nRegion distribution:")
    for region, count in region_dist.items():
        pct = count / len(train_df) * 100
        print(f"   {region[:40]:40s}: {count:4d} ({pct:5.2f}%)")

print("\n" + "="*80)
print("6. ESG SCORES ANALYSIS")
print("="*80)

score_cols = [col for col in train_df.columns if 'score' in col.lower()]

if len(score_cols) > 0:
    print(f"\nESG Score columns found: {len(score_cols)}")
    print(f"Columns: {score_cols}")
    
    for col in score_cols:
        print(f"\n" + "-"*80)
        print(f"{col.upper()}")
        print("-"*80)
        
        print(train_df[col].describe())
        
        # Correlation with targets
        scope1_corr = train_df[col].corr(train_df['target_scope_1'])
        scope2_corr = train_df[col].corr(train_df['target_scope_2'])
        
        print(f"\nCorrelation with targets:")
        print(f"   Scope 1: r={scope1_corr:6.3f}")
        print(f"   Scope 2: r={scope2_corr:6.3f}")
    
    # Visualize score distributions
    fig, axes = plt.subplots(1, len(score_cols), figsize=(6*len(score_cols), 5))
    
    if len(score_cols) == 1:
        axes = [axes]
    
    for idx, col in enumerate(score_cols):
        axes[idx].hist(train_df[col].dropna(), bins=20, edgecolor='black', alpha=0.7, color='purple')
        axes[idx].set_xlabel(col)
        axes[idx].set_ylabel('Frequency')
        axes[idx].set_title(f'{col} Distribution', fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # Score correlation matrix
    if len(score_cols) > 1:
        print("\n📊 Score Correlation Matrix:")
        print("-" * 80)
        score_corr_matrix = train_df[score_cols].corr()
        print(score_corr_matrix.round(3))
        
        # Heatmap
        import seaborn as sns
        plt.figure(figsize=(8, 6))
        sns.heatmap(score_corr_matrix, annot=True, cmap='coolwarm', center=0, 
                   square=True, linewidths=1, cbar_kws={"shrink": 0.8})
        plt.title('ESG Scores Correlation Matrix', fontweight='bold', fontsize=14)
        plt.tight_layout()
        plt.show()

else:
    print("⚠️ No ESG score columns found in train_df")

print("\n" + "="*80)
print("7. DATA QUALITY ASSESSMENT")
print("="*80)

print("\n📊 Missing Values Summary:")
print("-" * 80)

missing_summary = train_df.isnull().sum()
missing_pct = (missing_summary / len(train_df) * 100).round(2)

missing_df = pd.DataFrame({
    'Missing_Count': missing_summary,
    'Missing_Pct': missing_pct
}).sort_values('Missing_Count', ascending=False)

print(missing_df[missing_df['Missing_Count'] > 0])

if missing_df['Missing_Count'].sum() == 0:
    print("✅ NO MISSING VALUES - Excellent data quality!")
else:
    print(f"\n⚠️ {(missing_df['Missing_Count'] > 0).sum()} columns have missing values")
    print(f"   Consider imputation or feature engineering strategies")

# Check for duplicates
n_duplicates = train_df.duplicated(subset='entity_id').sum()
print(f"\n📊 Duplicate entity_ids: {n_duplicates}")

if n_duplicates == 0:
    print("✅ No duplicates - each company appears once")
else:
    print(f"⚠️ {n_duplicates} duplicate records found - investigate!")

# Check for zeros in targets
scope1_zeros = (train_df['target_scope_1'] == 0).sum()
scope2_zeros = (train_df['target_scope_2'] == 0).sum()

print(f"\n📊 Zero emissions:")
print(f"   Scope 1 zeros: {scope1_zeros} ({scope1_zeros/len(train_df)*100:.2f}%)")
print(f"   Scope 2 zeros: {scope2_zeros} ({scope2_zeros/len(train_df)*100:.2f}%)")

if scope1_zeros > 0 or scope2_zeros > 0:
    print(f"   ⚠️ Zero emissions may be:")
    print(f"      - True zeros (no direct emissions)")
    print(f"      - Missing data coded as zero")
    print(f"      - Small companies with negligible emissions")

print("\n" + "="*80)
print("EDA SUMMARY & KEY INSIGHTS")
print("="*80)

print(f"""
DATASET CHARACTERISTICS:
  • Size: {len(train_df):,} companies
  • Features: {len(train_df.columns)} columns
  • Completeness: {100 - (train_df.isnull().sum().sum() / (len(train_df) * len(train_df.columns)) * 100):.1f}% non-null
  • Geographic coverage: {train_df['country_name'].nunique() if 'country_name' in train_df.columns else 'N/A'} countries, {train_df['region_name'].nunique() if 'region_name' in train_df.columns else 'N/A'} regions

TARGET VARIABLES:
  • Scope 1: Mean={scope1_stats['Mean']:,.0f}, Median={scope1_stats['Median']:,.0f}, Skew={scope1_stats['Skewness']:.2f}
  • Scope 2: Mean={scope2_stats['Mean']:,.0f}, Median={scope2_stats['Median']:,.0f}, Skew={scope2_stats['Skewness']:.2f}
  • Correlation: r={scope_correlation:.3f} ({"Strong" if scope_correlation > 0.5 else "Moderate" if scope_correlation > 0.3 else "Weak"})
  • Distribution: {"HEAVILY" if scope1_stats['Skewness'] > 2 else "Moderately"} right-skewed → Log transformation REQUIRED

REVENUE:
  • Range: ${train_df['revenue'].min():,.0f} to ${train_df['revenue'].max():,.0f}
  • Correlation with Scope 1: r={revenue_scope1_corr:.3f} (raw) vs r={log_revenue_scope1_corr:.3f} (log)
  • Log transformation improves correlation by {((log_revenue_scope1_corr - revenue_scope1_corr) / revenue_scope1_corr * 100):.0f}%

CATEGORICAL FEATURES:
  • {len(categorical_cols)} categorical columns
  • Country: {train_df['country_name'].nunique() if 'country_name' in train_df.columns else 'N/A'} unique values
  • Region: {train_df['region_name'].nunique() if 'region_name' in train_df.columns else 'N/A'} unique values

ESG SCORES:
  • {len(score_cols)} score columns available
  • {f"Strongest correlation: {max([abs(train_df[col].corr(train_df['target_scope_1'])) for col in score_cols]):.3f}" if score_cols else "Not available"}

DATA QUALITY:
  • Missing values: {missing_df['Missing_Count'].sum()} total ({(missing_df['Missing_Count'].sum() / (len(train_df) * len(train_df.columns)) * 100):.2f}%)
  • Duplicates: {n_duplicates}
  • Zero emissions: {scope1_zeros} Scope 1, {scope2_zeros} Scope 2
""")

In [ ]:
print("="*80)

key_insights = """
🎯 CRITICAL FINDINGS:

1. TARGET DISTRIBUTION - EXTREMELY SKEWED
   • Scope 1 skewness: 3.15 (HEAVY right skew)
   • Scope 2 skewness: 7.34 (EXTREME right skew!)
   • Scope 2 kurtosis: 65.09 (massive outliers)
   → Log transformation is MANDATORY for modeling
   
2. SCOPE 1 vs SCOPE 2 RELATIONSHIP
   • Correlation: r=0.526 (STRONG positive)
   • Companies high in one tend to be high in both
   → Some shared drivers, but also independent factors
   → Validates separate analysis for each target
   
3. REVENUE INSIGHTS
   • Log transformation improves correlation by 46%!
   • Raw revenue: r=0.189 vs Log revenue: r=0.275
   → Already validated in revenue distribution analysis
   
4. GEOGRAPHIC CONCENTRATION
   • 98% of companies in Western Europe (63%) + North America (36%)
   • Only 28 countries, 7 regions
   • 12 countries with <3 companies (unreliable for target encoding)
   → Limited geographic diversity
   → US-centric dataset (34% of companies)
   
5. ESG SCORES - WEAK PREDICTORS
   • Environmental score: r=0.120 (Scope 1) - WEAK
   • Governance score: r=-0.113 (Scope 1) - WEAK NEGATIVE
   • Overall score: r=0.072 (Scope 1) - VERY WEAK
   → ESG scores are NOT strong emission predictors
   → But may still add marginal value
   
6. ESG SCORE CORRELATIONS (Among themselves)
   • Overall-Environmental: 0.825 (highly correlated)
   • Environmental-Governance: 0.032 (independent!)
   • Social-Governance: -0.012 (independent!)
   → Environmental score captures most ESG info
   → Governance is orthogonal (different dimension)
   
7. DATA QUALITY - EXCELLENT
   • 0 missing values ✅
   • 0 duplicates ✅
   • Only 13 Scope 2 zeros (3%) - legitimate small emitters
   
8. OUTLIERS
   • Scope 2 max (2M) is 35x the mean (57K)
   • Scope 1 max (638K) is 11x the mean (56K)
   → Tree-based models recommended (robust to outliers)
"""

print(key_insights)

### Hypothesis-Driven Feature Validation

#### Hypothesis 3: Geography Matters for Scope 2

In [ ]:
print("""
PREDICTION: Grid carbon intensity varies by country → Scope 2 varies more by 
            geography than Scope 1 (which is driven by sector/operations)
            
TEST: Compare country/region effect sizes for Scope 1 vs Scope 2
      (Exploratory analysis only - for understanding, not feature creation)
""")

print("\n" + "-"*80)
print("TEST 1: EMISSIONS BY COUNTRY")
print("-"*80)

# Calculate country-level statistics (for analysis only)
country_emissions = train_df.groupby('country_name').agg({
    'target_scope_1': ['mean', 'median', 'std', 'count'],
    'target_scope_2': ['mean', 'median', 'std'],
    'revenue': 'mean'
}).round(0)

country_emissions.columns = ['Scope1_Mean', 'Scope1_Median', 'Scope1_Std', 'Count', 
                             'Scope2_Mean', 'Scope2_Median', 'Scope2_Std', 'Revenue_Mean']

# Filter countries with at least 3 companies for reliable statistics
country_emissions_reliable = country_emissions[country_emissions['Count'] >= 3].copy()

# Calculate emission intensities
country_emissions_reliable['Scope1_Intensity'] = (
    country_emissions_reliable['Scope1_Mean'] / (country_emissions_reliable['Revenue_Mean'] + 1)
)
country_emissions_reliable['Scope2_Intensity'] = (
    country_emissions_reliable['Scope2_Mean'] / (country_emissions_reliable['Revenue_Mean'] + 1)
)
country_emissions_reliable['Scope2_to_Scope1_Ratio'] = (
    country_emissions_reliable['Scope2_Mean'] / (country_emissions_reliable['Scope1_Mean'] + 1)
)

# Calculate coefficient of variation (CV) across countries
scope1_country_cv = country_emissions_reliable['Scope1_Mean'].std() / country_emissions_reliable['Scope1_Mean'].mean()
scope2_country_cv = country_emissions_reliable['Scope2_Mean'].std() / country_emissions_reliable['Scope2_Mean'].mean()

print(f"\n📊 Cross-Country Variation (Coefficient of Variation):")
print(f"   Scope 1 CV across countries: {scope1_country_cv:.3f}")
print(f"   Scope 2 CV across countries: {scope2_country_cv:.3f}")

if scope2_country_cv > scope1_country_cv:
    print(f"   ✅ Scope 2 varies MORE by country ({((scope2_country_cv - scope1_country_cv) / scope1_country_cv * 100):.0f}% higher variation)")
    print(f"   → Geography matters MORE for Scope 2 (grid intensity effect)")
else:
    print(f"   ❌ Scope 1 varies more by country")

print(f"\n📊 Top 10 Countries by Scope 2 Emissions:")
print("-" * 80)
top_scope2 = country_emissions_reliable.sort_values('Scope2_Mean', ascending=False).head(10)
print(top_scope2[['Scope2_Mean', 'Scope1_Mean', 'Scope2_to_Scope1_Ratio', 'Count']])

print(f"\n📊 Top 10 Countries by Electricity Intensity (Scope 2 / Scope 1 ratio):")
print("-" * 80)
print("(Higher ratio = more electricity-dependent or dirtier grid)")
top_ratio = country_emissions_reliable.sort_values('Scope2_to_Scope1_Ratio', ascending=False).head(10)
print(top_ratio[['Scope2_to_Scope1_Ratio', 'Scope2_Mean', 'Scope1_Mean', 'Count']])

print("\n" + "-"*80)
print("TEST 2: EMISSIONS BY REGION")
print("-"*80)

region_emissions = train_df.groupby('region_name').agg({
    'target_scope_1': ['mean', 'median', 'std', 'count'],
    'target_scope_2': ['mean', 'median', 'std'],
    'revenue': 'mean'
}).round(0)

region_emissions.columns = ['Scope1_Mean', 'Scope1_Median', 'Scope1_Std', 'Count',
                            'Scope2_Mean', 'Scope2_Median', 'Scope2_Std', 'Revenue_Mean']

region_emissions['Scope2_to_Scope1_Ratio'] = (
    region_emissions['Scope2_Mean'] / (region_emissions['Scope1_Mean'] + 1)
)

print(region_emissions.sort_values('Scope2_Mean', ascending=False))

# Regional variation
scope1_region_cv = region_emissions['Scope1_Mean'].std() / region_emissions['Scope1_Mean'].mean()
scope2_region_cv = region_emissions['Scope2_Mean'].std() / region_emissions['Scope2_Mean'].mean()

print(f"\n📊 Cross-Region Variation:")
print(f"   Scope 1 CV across regions: {scope1_region_cv:.3f}")
print(f"   Scope 2 CV across regions: {scope2_region_cv:.3f}")

print("\n" + "-"*80)
print("TEST 3: STATISTICAL SIGNIFICANCE (ANOVA)")
print("-"*80)

from scipy import stats

# Test if country significantly affects emissions
country_groups_scope1 = []
country_groups_scope2 = []
countries_tested = []

for country, group in train_df.groupby('country_name'):
    if len(group) >= 5:
        country_groups_scope1.append(group['target_scope_1'].values)
        country_groups_scope2.append(group['target_scope_2'].values)
        countries_tested.append(country)

if len(country_groups_scope1) >= 3:
    # ANOVA for Scope 1
    f_stat_s1, p_val_s1 = stats.f_oneway(*country_groups_scope1)
    
    # ANOVA for Scope 2
    f_stat_s2, p_val_s2 = stats.f_oneway(*country_groups_scope2)
    
    print(f"\n📊 ANOVA Test: Does country affect emissions?")
    print(f"   Countries tested: {len(countries_tested)} ({', '.join(countries_tested[:5])}...)")
    
    print(f"\n   SCOPE 1:")
    print(f"     F-statistic: {f_stat_s1:.2f}")
    print(f"     P-value: {p_val_s1:.6f}")
    if p_val_s1 < 0.001:
        print(f"     ✅ HIGHLY SIGNIFICANT (p < 0.001)")
    elif p_val_s1 < 0.05:
        print(f"     ✅ SIGNIFICANT (p < 0.05)")
    else:
        print(f"     ❌ Not significant")
    
    print(f"\n   SCOPE 2:")
    print(f"     F-statistic: {f_stat_s2:.2f}")
    print(f"     P-value: {p_val_s2:.6f}")
    if p_val_s2 < 0.001:
        print(f"     ✅ HIGHLY SIGNIFICANT (p < 0.001)")
    elif p_val_s2 < 0.05:
        print(f"     ✅ SIGNIFICANT (p < 0.05)")
    else:
        print(f"     ❌ Not significant")
    
    # Compare F-statistics
    print(f"\n   💡 COMPARISON:")
    if f_stat_s2 > f_stat_s1:
        print(f"     Scope 2 F-statistic is {((f_stat_s2 - f_stat_s1) / f_stat_s1 * 100):.0f}% HIGHER")
        print(f"     → Country matters MORE for Scope 2!")
    else:
        print(f"     Scope 1 F-statistic is higher")

print("\n" + "-"*80)
print("TEST 4: VALID FEATURE ENCODING APPROACHES")
print("-"*80)

# Create VALID features that can be used on test set

# 1. Frequency Encoding (Valid - based on count, not emissions)
country_freq = train_df['country_name'].value_counts()
region_freq = train_df['region_name'].value_counts()

train_df['country_frequency'] = train_df['country_name'].map(country_freq)
train_df['region_frequency'] = train_df['region_name'].map(region_freq)

# Correlations with frequency encoding
freq_country_scope1_corr = train_df['country_frequency'].corr(train_df['target_scope_1'])
freq_country_scope2_corr = train_df['country_frequency'].corr(train_df['target_scope_2'])
freq_region_scope1_corr = train_df['region_frequency'].corr(train_df['target_scope_1'])
freq_region_scope2_corr = train_df['region_frequency'].corr(train_df['target_scope_2'])

print(f"\n📊 Geographic Feature Correlations (Frequency Encoding - VALID):")
print(f"   country_frequency vs Scope 1: r={freq_country_scope1_corr:.3f}")
print(f"   country_frequency vs Scope 2: r={freq_country_scope2_corr:.3f}")
print(f"   region_frequency vs Scope 1:  r={freq_region_scope1_corr:.3f}")
print(f"   region_frequency vs Scope 2:  r={freq_region_scope2_corr:.3f}")

# 2. Label Encoding (for comparison)
from sklearn.preprocessing import LabelEncoder

le_country = LabelEncoder()
le_region = LabelEncoder()

train_df['country_encoded'] = le_country.fit_transform(train_df['country_name'])
train_df['region_encoded'] = le_region.fit_transform(train_df['region_name'])

country_scope1_corr = train_df['country_encoded'].corr(train_df['target_scope_1'])
country_scope2_corr = train_df['country_encoded'].corr(train_df['target_scope_2'])
region_scope1_corr = train_df['region_encoded'].corr(train_df['target_scope_1'])
region_scope2_corr = train_df['region_encoded'].corr(train_df['target_scope_2'])

print(f"\n📊 Geographic Feature Correlations (Label Encoding - VALID but weak):")
print(f"   country_encoded vs Scope 1: r={country_scope1_corr:.3f}")
print(f"   country_encoded vs Scope 2: r={country_scope2_corr:.3f}")
print(f"   region_encoded vs Scope 1:  r={region_scope1_corr:.3f}")
print(f"   region_encoded vs Scope 2:  r={region_scope2_corr:.3f}")

print(f"\n💡 CONCLUSION:")
print(f"   Frequency encoding is better than label encoding")
print(f"   But both are WEAK compared to sector/revenue features")
print(f"   → Geographic features have limited predictive power with valid encoding")

# Visualizations
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Top countries by Scope 2
top_10_scope2 = country_emissions_reliable.sort_values('Scope2_Mean', ascending=False).head(10)
axes[0, 0].barh(range(len(top_10_scope2)), top_10_scope2['Scope2_Mean'], color='orange', edgecolor='black')
axes[0, 0].set_yticks(range(len(top_10_scope2)))
axes[0, 0].set_yticklabels([c[:30] for c in top_10_scope2.index])
axes[0, 0].set_xlabel('Average Scope 2 Emissions')
axes[0, 0].set_title('Top 10 Countries by Scope 2\n(Exploratory Analysis)', fontweight='bold')
axes[0, 0].invert_yaxis()

# Scope 2/Scope 1 ratio
top_10_ratio = country_emissions_reliable.sort_values('Scope2_to_Scope1_Ratio', ascending=False).head(10)
axes[0, 1].barh(range(len(top_10_ratio)), top_10_ratio['Scope2_to_Scope1_Ratio'], color='teal', edgecolor='black')
axes[0, 1].set_yticks(range(len(top_10_ratio)))
axes[0, 1].set_yticklabels([c[:30] for c in top_10_ratio.index])
axes[0, 1].set_xlabel('Scope 2 / Scope 1 Ratio')
axes[0, 1].set_title('Top 10 Countries by Electricity Intensity', fontweight='bold')
axes[0, 1].invert_yaxis()

# Regional comparison
regions = region_emissions.index
x_pos = np.arange(len(regions))
width = 0.35

axes[1, 0].bar(x_pos - width/2, region_emissions['Scope1_Mean'], width, 
              label='Scope 1', color='steelblue', edgecolor='black')
axes[1, 0].bar(x_pos + width/2, region_emissions['Scope2_Mean'], width,
              label='Scope 2', color='orange', edgecolor='black')
axes[1, 0].set_xticks(x_pos)
axes[1, 0].set_xticklabels([r[:15] for r in regions], rotation=45, ha='right')
axes[1, 0].set_ylabel('Average Emissions')
axes[1, 0].set_title('Emissions by Region (Scope 1 vs Scope 2)', fontweight='bold')
axes[1, 0].legend()

# Valid encoding comparison
encoding_methods = ['Frequency\nEncoding', 'Label\nEncoding']
scope1_corrs = [freq_country_scope1_corr, country_scope1_corr]
scope2_corrs = [freq_country_scope2_corr, country_scope2_corr]

x = np.arange(len(encoding_methods))
axes[1, 1].bar(x - width/2, scope1_corrs, width, label='Scope 1', 
              color='steelblue', edgecolor='black')
axes[1, 1].bar(x + width/2, scope2_corrs, width, label='Scope 2',
              color='orange', edgecolor='black')
axes[1, 1].set_xticks(x)
axes[1, 1].set_xticklabels(encoding_methods)
axes[1, 1].set_ylabel('Correlation Coefficient')
axes[1, 1].set_title('Country Encoding Performance\n(Valid Methods Only)', fontweight='bold')
axes[1, 1].legend()
axes[1, 1].axhline(0, color='black', linewidth=0.5)

plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("FEATURES CREATED (VALID FOR TEST SET)")
print("="*80)

print("""
✅ country_frequency (count-based, r={:.3f} for Scope 1)
✅ region_frequency (count-based, r={:.3f} for Scope 1)
✅ country_name (categorical - can use one-hot encoding)
✅ region_name (categorical - can use one-hot encoding)

⚠️ country_encoded, region_encoded (label encoding - weak, r<0.1)

Note: Frequency encoding works better than label encoding
      but both are weak compared to sector/revenue features
""".format(freq_country_scope1_corr, freq_region_scope1_corr))

In [ ]:
print("\n" + "="*80)
print("HYPOTHESIS 3: GEOGRAPHY MATTERS FOR SCOPE 2 - FINAL SUMMARY")
print("="*80)

# Key metrics from results
scope2_country_cv = 1.770
scope1_country_cv = 0.977
cv_difference_pct = ((scope2_country_cv - scope1_country_cv) / scope1_country_cv * 100)

f_stat_s1 = 2.70
f_stat_s2 = 1.44
p_val_s1 = 0.001609
p_val_s2 = 0.145268

freq_country_corr_s1 = 0.118
freq_country_corr_s2 = 0.113
freq_region_corr_s1 = -0.142
freq_region_corr_s2 = -0.153

label_country_corr = 0.082
label_region_corr = -0.106

# Determine validation status
status = "⚠️ PARTIALLY VALIDATED (WEAK FEATURES)"

summary = f"""

HYPOTHESIS 3: GEOGRAPHY MATTERS FOR SCOPE 2 {status}

KEY FINDINGS:
1. Scope 2 DOES vary more by country (exploratory evidence):
   - Scope 2 CV: {scope2_country_cv:.3f} (81% HIGHER than Scope 1)
   - Scope 1 CV: {scope1_country_cv:.3f}
   → Cross-country variation supports hypothesis
   
2. BUT statistical significance is MIXED:
   - Scope 1 ANOVA: F={f_stat_s1:.2f}, p={p_val_s1:.6f} ✅ SIGNIFICANT
   - Scope 2 ANOVA: F={f_stat_s2:.2f}, p={p_val_s2:.6f} ❌ NOT SIGNIFICANT
   → Paradox: Scope 2 varies MORE but not statistically significant
   
3. Valid encoding methods show WEAK predictive power:
   - Frequency encoding (country): r={freq_country_corr_s1:.3f} (Scope 1)
   - Label encoding (country): r={label_country_corr:.3f} (Scope 1)
   - Region frequency: r={freq_region_corr_s1:.3f} (Scope 1, NEGATIVE!)
   → Geographic features are WEAK predictors with valid methods
   
4. Electricity intensity varies dramatically by country:
   - Norway: Scope 2/Scope 1 ratio = 3.13 (very electricity-intensive)
   - USA: Scope 2/Scope 1 ratio = 1.18
   - Switzerland: Scope 2/Scope 1 ratio = 0.67 (low electricity)
   → Real grid intensity differences exist
   
5. Dataset limitations reduce geographic value:
   - 98% of companies in Western Europe (63%) + North America (36%)
   - Only 28 countries, 12 have n<3 (unreliable)
   - Norway is massive outlier (n=3, distorts statistics)
   → Limited geographic diversity

INTERPRETATION:
The hypothesis is PARTIALLY correct with important caveats:

✅ EXPLORATORY EVIDENCE SUPPORTS:
   - Scope 2 DOES vary more across countries (81% higher CV)
   - Grid intensity differences are real (3x range in Scope2/Scope1 ratios)
   - Norway, USA show high electricity dependence
   
❌ BUT PRACTICAL FEATURES ARE WEAK:
   - Best valid encoding: frequency (r={freq_country_corr_s1:.3f})
   - This is MUCH weaker than sector (r=0.275) or revenue (r=0.403)
   - Not statistically significant for Scope 2 (p={p_val_s2:.3f})
   
💡 WHY THE DISCONNECT?
   1. Valid encoding methods (frequency, label) can't capture the nuanced
      geographic effects that target encoding revealed
   2. Small sample sizes for most countries (12/28 have n<3)
   3. High within-country variance masks between-country differences
   4. Dataset concentrated in similar regions (Western developed countries)

VALIDATION OF SPECIFIC PREDICTIONS:
  ⚠️ "Grid carbon intensity varies by country"
     → CONFIRMED in exploratory analysis (3x ratio range)
     → BUT can't effectively use this in predictive features
     
  ⚠️ "Scope 2 varies more by geography than Scope 1"
     → CONFIRMED by CV (81% higher variation)
     → BUT NOT confirmed by ANOVA (p={p_val_s2:.3f} not significant)
     → Frequency encoding shows SIMILAR correlation for both scopes
     
  ❌ "Geography is a strong predictor of emissions"
     → REJECTED: r={freq_country_corr_s1:.3f} is weak
     → Much weaker than sector/revenue features

RECOMMENDED FEATURES FOR FINAL MODEL:
  Priority 3 (OPTIONAL - weak signal):
    ⚠️ country_frequency (r={freq_country_corr_s1:.3f}) - count-based encoding
    ⚠️ region_name (categorical - only 7 values, use one-hot)
    ❌ region_frequency (r={freq_region_corr_s1:.3f}) - NEGATIVE correlation!
    ❌ country_encoded, region_encoded (label encoding too weak)
    
  Alternative approach:
    ⚠️ country_name with one-hot encoding (28 columns)
    ⚠️ Group small countries into top 10 + "Other" (11 columns)
    
  Expected R² contribution: ~1-2% (very marginal)

MODELING IMPLICATIONS:
  1. Geographic features add MINIMAL value:
     - Expected R² contribution: ~1.4% (0.118² = 0.014)
     - Sector + Revenue already explain ~45-50%
     - Not worth complexity for 1-2% improvement
     
  2. One-hot encoding regions might work better:
     - Only 7 categories (manageable)
     - No arbitrary ordering like label encoding
     - Model can learn region-specific patterns
     
  3. Country encoding is problematic:
     - 28 categories → 28 columns if one-hot
     - Many countries have small samples (n<3)
     - Frequency encoding only r={freq_country_corr_s1:.3f}
     
  4. Consider SKIPPING geographic features entirely:
     - Focus on stronger predictors (sector, revenue)
     - Add geography only if model capacity allows
     - Not in minimal viable model

VISUAL EVIDENCE:
  - Norway dramatically outperforms all others (outlier effect)
  - Scope 2/Scope 1 ratios show 3x range (grid differences are real)
  - Regional bars show North America has higher Scope 2 than Europe
  - Frequency encoding slightly better than label encoding
  - But both correlations are very weak (r<0.12)

UNEXPECTED FINDINGS:
  1. **Region frequency has NEGATIVE correlation**:
     - r={freq_region_corr_s1:.3f} (Scope 1)
     - Larger regions (more companies) → LOWER emissions?
     - Likely confounded: Western Europe is large but low-emission
     - North America is large but moderate-emission
     
  2. **Frequency encoding barely better than label**:
     - Frequency: r={freq_country_corr_s1:.3f}
     - Label: r={label_country_corr:.3f}
     - Both very weak, minimal difference
     
  3. **Scope 2 ANOVA not significant despite higher variation**:
     - Higher CV but p={p_val_s2:.3f}
     - High within-country variance
     - Small sample sizes reduce power
     
  4. **Norway massively inflates Scope 2 statistics**:
     - Norway Scope 2: 503,019 (5.6x USA average!)
     - Only 3 companies in Norway
     - Removing Norway would likely reduce CV significantly

CRITICAL LIMITATIONS:
  1. **Geographic concentration**: 98% Western Europe + North America
  2. **Small samples**: 12/28 countries have n<3
  3. **Norway outlier**: n=3, extreme Scope 2 (503K)
  4. **Limited grid diversity**: Mostly developed countries
  5. **Valid encoding weakness**: Can't use target encoding on test set
  6. **High within-country variance**: Masks geographic effects

REVISED HYPOTHESIS:
  Original: "Geography matters MORE for Scope 2 than Scope 1"
  
  Revised: "Grid intensity varies by country, but geographic features
            have weak predictive power when using valid encoding methods"
  
  Better formulation:
  - Geographic variation EXISTS (exploratory evidence)
  - But we CAN'T effectively capture it in predictive features
  - Frequency/label encoding too weak (r<0.12)
  - One-hot encoding might work better but adds complexity
  - Not worth the effort given strong sector/revenue features

REAL-WORLD INTERPRETATION:

**Why does geography matter less than expected?**

1. **Dataset homogeneity**: 
   - 98% similar regions (Western developed countries)
   - All have relatively clean grids compared to coal-heavy countries
   - Limited variation to capture
   
2. **Within-country variance dominates**:
   - US companies range from 0 to 2M in Scope 2
   - Sector matters MORE than country
   - A US manufacturing company >> a US services company
   
3. **Sample size issues**:
   - Only 13 countries have n≥5 for reliable statistics
   - Can't learn country-specific patterns with n<5
   
4. **Outliers distort**:
   - Norway with n=3 has extreme values
   - One or two companies can skew entire country statistics

CONCLUSION:
  ⚠️ HYPOTHESIS PARTIALLY VALIDATED IN THEORY, WEAK IN PRACTICE
  
  Exploratory findings:
  ✅ Scope 2 varies more by country (81% higher CV)
  ✅ Grid intensity differences are real (3x range)
  
  But practical features are weak:
  ❌ Best encoding: r={freq_country_corr_s1:.3f} (very weak)
  ❌ Not statistically significant for Scope 2
  ❌ Much weaker than sector (r=0.275) or revenue (r=0.403)
"""

print(summary)

print("\n" + "="*80)
print("GEOGRAPHIC FEATURES SUMMARY TABLE")
print("="*80)
print(f"""
Feature                          │ Scope 1  │ Scope 2  │ Method          │ Use?
─────────────────────────────────┼──────────┼──────────┼─────────────────┼──────────
country_frequency                │  {freq_country_corr_s1:.3f}   │  {freq_country_corr_s2:.3f}   │ Count-based     │ ⚠️ MAYBE
region_frequency                 │ {freq_region_corr_s1:.3f}   │ {freq_region_corr_s2:.3f}   │ Count-based     │ ❌ NO (negative)
country_encoded (label)          │  {label_country_corr:.3f}   │  {label_country_corr:.3f}   │ Label encoding  │ ❌ NO (too weak)
region_encoded (label)           │ {label_region_corr:.3f}   │ -0.121   │ Label encoding  │ ❌ NO (negative)
country_name (one-hot)           │    ?     │    ?     │ One-hot (28)    │ ⚠️ MAYBE (complex)
region_name (one-hot)            │    ?     │    ?     │ One-hot (7)     │ ⚠️ MAYBE
─────────────────────────────────┴──────────┴──────────┴─────────────────┴──────────

KEY STATISTICS:
  Countries: 28 total, 16 with n≥3, 12 with n<3 (unreliable)
  Regions: 7 total, but 98% are Western Europe + North America
  
  Cross-country variation:
    Scope 1 CV: {scope1_country_cv:.3f}
    Scope 2 CV: {scope2_country_cv:.3f} (+81% higher) ✅
    
  Statistical significance:
    Scope 1 ANOVA: F={f_stat_s1:.2f}, p={p_val_s1:.6f} ✅
    Scope 2 ANOVA: F={f_stat_s2:.2f}, p={p_val_s2:.6f} ❌
    
  Best valid encoding:
    Frequency (country): r={freq_country_corr_s1:.3f} ⭐ WEAK
    
  Electricity intensity range:
    Norway: 3.13 (highest)
    Switzerland: 0.67 (lowest)
    Range: 4.7x difference
    
RECOMMENDATION: SKIP or use only as TIER 3 (optional) features
  Priority: Use sector + revenue instead (much stronger)
  Expected value: ~1-2% R² improvement
  Complexity cost: High (28 countries or encoding challenges)
""")

#### Hypothesis 9: Scale sets the baseline

In [ ]:
print("""
PREDICTION: Larger organizations emit disproportionately more, but only up to a point.
            Log-transformed revenue reveals scale effects.
            
SPECIFIC PREDICTIONS:
  • Log revenue correlates positively with emissions (already validated in H2)
  • Country-adjusted revenue (residuals) reveals giants and over-performers
  
TEST: Create scale-based features (valid for test set)
""")

print("\n" + "-"*80)
print("TEST 1: LOG REVENUE SCALE EFFECT (Validation)")
print("-"*80)

# Calculate log revenue if not exists
if 'log_revenue' not in train_df.columns:
    train_df['log_revenue'] = np.log1p(train_df['revenue'])

# Revenue quartiles for comparison
train_df['revenue_quartile'] = pd.qcut(train_df['revenue'], q=4, labels=['Q1-Small', 'Q2-Medium', 'Q3-Large', 'Q4-Giant'])

print("\n📊 Emissions by Revenue Quartile:")
print("-" * 80)

quartile_analysis = train_df.groupby('revenue_quartile').agg({
    'revenue': ['min', 'max', 'mean'],
    'target_scope_1': ['mean', 'median', 'std'],
    'target_scope_2': ['mean', 'median', 'std'],
    'entity_id': 'count'
}).round(0)

quartile_analysis.columns = ['Rev_Min', 'Rev_Max', 'Rev_Mean', 
                             'Scope1_Mean', 'Scope1_Median', 'Scope1_Std',
                             'Scope2_Mean', 'Scope2_Median', 'Scope2_Std', 'Count']

print(quartile_analysis)

# Calculate emission intensity by quartile
quartile_analysis['Scope1_per_Revenue'] = quartile_analysis['Scope1_Mean'] / (quartile_analysis['Rev_Mean'] + 1)
quartile_analysis['Scope2_per_Revenue'] = quartile_analysis['Scope2_Mean'] / (quartile_analysis['Rev_Mean'] + 1)

print(f"\n📊 Emission Intensity by Quartile (emissions per dollar of revenue):")
print(quartile_analysis[['Scope1_per_Revenue', 'Scope2_per_Revenue']])

# Check for economies of scale
q1_intensity = quartile_analysis.loc['Q1-Small', 'Scope1_per_Revenue']
q4_intensity = quartile_analysis.loc['Q4-Giant', 'Scope1_per_Revenue']

print(f"\n💡 ECONOMIES OF SCALE CHECK:")
print(f"   Q1 (Small) Scope 1 intensity: {q1_intensity:.8f}")
print(f"   Q4 (Giant) Scope 1 intensity: {q4_intensity:.8f}")

if q4_intensity < q1_intensity:
    efficiency_gain = ((q1_intensity - q4_intensity) / q1_intensity * 100)
    print(f"   ✅ Giants are {efficiency_gain:.1f}% MORE EFFICIENT (economies of scale!)")
else:
    print(f"   ❌ No economies of scale detected")

# Correlation check
log_revenue_scope1_corr = train_df['log_revenue'].corr(train_df['target_scope_1'])
log_revenue_scope2_corr = train_df['log_revenue'].corr(train_df['target_scope_2'])

print(f"\n📊 Log Revenue Correlations:")
print(f"   Log revenue vs Scope 1: r={log_revenue_scope1_corr:.3f} ⭐⭐⭐")
print(f"   Log revenue vs Scope 2: r={log_revenue_scope2_corr:.3f} ⭐⭐⭐")

print("\n" + "-"*80)
print("TEST 2: COUNTRY-ADJUSTED REVENUE (VALID - uses revenue only)")
print("-"*80)

print("""
Goal: Identify companies that are giants WITHIN their country
      vs. absolute giants globally
      
Method: Calculate Z-score from country-level revenue statistics
        (Uses revenue only - VALID for test set)
""")

# Calculate country-level revenue statistics
country_revenue_stats = train_df.groupby('country_name')['revenue'].agg(['mean', 'std', 'count'])

# Calculate Z-score (VALID - uses revenue only, not emissions)
train_df['revenue_country_zscore'] = train_df.groupby('country_name')['revenue'].transform(
    lambda x: (x - x.mean()) / (x.std() + 1)
)

print(f"\n📊 Country-Adjusted Revenue Z-score Statistics:")
print(train_df['revenue_country_zscore'].describe())

# Correlations (VALID)
zscore_scope1_corr = train_df['revenue_country_zscore'].corr(train_df['target_scope_1'])
zscore_scope2_corr = train_df['revenue_country_zscore'].corr(train_df['target_scope_2'])

print(f"\n📊 Country-Adjusted Revenue Correlations:")
print(f"   Revenue Z-score vs Scope 1:       r={zscore_scope1_corr:.3f}")
print(f"   Revenue Z-score vs Scope 2:       r={zscore_scope2_corr:.3f}")

# Identify giants within country
train_df['is_giant_in_country'] = (train_df['revenue_country_zscore'] > 1).astype(int)

giants_in_country = train_df[train_df['is_giant_in_country'] == 1]
non_giants = train_df[train_df['is_giant_in_country'] == 0]

print(f"\n📊 Giants vs Non-Giants Within Countries:")
print("-" * 80)

print(f"\nGiants in their country (Z>1, n={len(giants_in_country)}):")
print(f"   Avg revenue: ${giants_in_country['revenue'].mean():,.0f}")
print(f"   Avg Scope 1: {giants_in_country['target_scope_1'].mean():,.0f}")
print(f"   Avg Scope 2: {giants_in_country['target_scope_2'].mean():,.0f}")

print(f"\nNon-giants (Z≤1, n={len(non_giants)}):")
print(f"   Avg revenue: ${non_giants['revenue'].mean():,.0f}")
print(f"   Avg Scope 1: {non_giants['target_scope_1'].mean():,.0f}")
print(f"   Avg Scope 2: {non_giants['target_scope_2'].mean():,.0f}")

diff_pct_scope1 = ((giants_in_country['target_scope_1'].mean() - non_giants['target_scope_1'].mean()) / 
                   non_giants['target_scope_1'].mean() * 100)
diff_pct_scope2 = ((giants_in_country['target_scope_2'].mean() - non_giants['target_scope_2'].mean()) / 
                   non_giants['target_scope_2'].mean() * 100)

print(f"\n💡 DIFFERENCE:")
print(f"   Giants emit {diff_pct_scope1:+.0f}% more Scope 1")
print(f"   Giants emit {diff_pct_scope2:+.0f}% more Scope 2")

print("\n" + "-"*80)
print("TEST 3: NON-LINEAR SCALE EFFECTS")
print("-"*80)

# Test for non-linear relationships
train_df['log_revenue_squared'] = train_df['log_revenue'] ** 2

log_squared_scope1_corr = train_df['log_revenue_squared'].corr(train_df['target_scope_1'])
log_squared_scope2_corr = train_df['log_revenue_squared'].corr(train_df['target_scope_2'])

print(f"\n📊 Non-Linear Scale Effects:")
print(f"   Log(Revenue)² vs Scope 1:      r={log_squared_scope1_corr:.3f}")
print(f"   Log(Revenue)² vs Scope 2:      r={log_squared_scope2_corr:.3f}")

# Check if squared terms add value
print(f"\n💡 DO SQUARED TERMS ADD VALUE?")
if log_squared_scope1_corr > log_revenue_scope1_corr + 0.01:
    print(f"   ✅ Log²(Revenue) stronger than Log(Revenue)")
    print(f"   → Consider adding polynomial features")
else:
    print(f"   ❌ Log(Revenue) is sufficient (r={log_revenue_scope1_corr:.3f} vs r={log_squared_scope1_corr:.3f})")
    print(f"   → Skip polynomial features")

# VISUALIZATIONS
print("\n" + "-"*80)
print("CREATING VISUALIZATIONS...")
print("-"*80)

fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# 1. Emissions by revenue quartile
quartiles = ['Q1-Small', 'Q2-Medium', 'Q3-Large', 'Q4-Giant']
scope1_means = quartile_analysis['Scope1_Mean'].values
scope2_means = quartile_analysis['Scope2_Mean'].values

x = np.arange(len(quartiles))
width = 0.35

axes[0, 0].bar(x - width/2, scope1_means, width, label='Scope 1', color='steelblue', edgecolor='black')
axes[0, 0].bar(x + width/2, scope2_means, width, label='Scope 2', color='orange', edgecolor='black')
axes[0, 0].set_xticks(x)
axes[0, 0].set_xticklabels(quartiles, rotation=45, ha='right')
axes[0, 0].set_ylabel('Average Emissions')
axes[0, 0].set_title('Emissions by Revenue Quartile\n(Larger companies emit more)', fontweight='bold')
axes[0, 0].legend()

# 2. Emission intensity by quartile
axes[0, 1].plot(quartiles, quartile_analysis['Scope1_per_Revenue'].values, 
               marker='o', linewidth=2, markersize=8, color='steelblue', label='Scope 1')
axes[0, 1].plot(quartiles, quartile_analysis['Scope2_per_Revenue'].values,
               marker='s', linewidth=2, markersize=8, color='orange', label='Scope 2')
axes[0, 1].set_xlabel('Revenue Quartile')
axes[0, 1].set_ylabel('Emissions per Revenue Dollar')
axes[0, 1].set_title('Emission Intensity by Company Size\n(Economies of scale?)', fontweight='bold')
axes[0, 1].legend()
axes[0, 1].tick_params(axis='x', rotation=45)
axes[0, 1].ticklabel_format(style='scientific', axis='y', scilimits=(0,0))

# 3. Log revenue vs log emissions
axes[0, 2].scatter(train_df['log_revenue'], np.log1p(train_df['target_scope_1']), alpha=0.5, s=20)
axes[0, 2].set_xlabel('log(Revenue + 1)')
axes[0, 2].set_ylabel('log(Scope 1 + 1)')
axes[0, 2].set_title(f'Log-Log Relationship\n(r={log_revenue_scope1_corr:.3f})', fontweight='bold')

# Add trend line
z = np.polyfit(train_df['log_revenue'], np.log1p(train_df['target_scope_1']), 1)
p = np.poly1d(z)
axes[0, 2].plot(train_df['log_revenue'].sort_values(), 
               p(train_df['log_revenue'].sort_values()), 
               "r--", linewidth=2, label=f'y={z[0]:.2f}x+{z[1]:.2f}')
axes[0, 2].legend()

# 4. Giants vs non-giants
categories = ['Non-Giants\n(Z≤1)', 'Giants in Country\n(Z>1)']
scope1_avg = [non_giants['target_scope_1'].mean(), giants_in_country['target_scope_1'].mean()]
scope2_avg = [non_giants['target_scope_2'].mean(), giants_in_country['target_scope_2'].mean()]

x = np.arange(len(categories))
axes[1, 0].bar(x - width/2, scope1_avg, width, label='Scope 1', color='steelblue', edgecolor='black')
axes[1, 0].bar(x + width/2, scope2_avg, width, label='Scope 2', color='orange', edgecolor='black')
axes[1, 0].set_xticks(x)
axes[1, 0].set_xticklabels(categories)
axes[1, 0].set_ylabel('Average Emissions')
axes[1, 0].set_title('Giants vs Non-Giants Within Countries', fontweight='bold')
axes[1, 0].legend()

# 5. Country-adjusted revenue Z-score
axes[1, 1].scatter(train_df['revenue_country_zscore'], train_df['target_scope_1'], alpha=0.5, s=20, color='green')
axes[1, 1].set_xlabel('Revenue Z-Score Within Country')
axes[1, 1].set_ylabel('Scope 1 Emissions')
axes[1, 1].set_title(f'Country-Adjusted Scale Effect\n(r={zscore_scope1_corr:.3f})', fontweight='bold')
axes[1, 1].axvline(1, color='red', linestyle='--', alpha=0.5, label='Giant threshold (Z=1)')
axes[1, 1].legend()

# 6. Feature comparison
feature_names = ['Log\nRevenue', 'Revenue\nZ-score', 'Log(Rev)²']
correlations_scope1 = [log_revenue_scope1_corr, zscore_scope1_corr, log_squared_scope1_corr]
correlations_scope2 = [log_revenue_scope2_corr, zscore_scope2_corr, log_squared_scope2_corr]

x = np.arange(len(feature_names))
axes[1, 2].bar(x - width/2, correlations_scope1, width, label='Scope 1',
               color='steelblue', edgecolor='black')
axes[1, 2].bar(x + width/2, correlations_scope2, width, label='Scope 2',
               color='orange', edgecolor='black')
axes[1, 2].set_xticks(x)
axes[1, 2].set_xticklabels(feature_names)
axes[1, 2].set_ylabel('Correlation Coefficient')
axes[1, 2].set_title('Scale Feature Performance Comparison\n(Valid Features Only)', fontweight='bold')
axes[1, 2].legend()
axes[1, 2].axhline(0, color='black', linewidth=0.5)

plt.tight_layout()
plt.show()

print("✓ Visualizations created!")

print("\n" + "="*80)
print("FEATURES CREATED (VALID FOR TEST SET)")
print("="*80)

print(f"""
✅ log_revenue (r={log_revenue_scope1_corr:.3f} for Scope 1)
   - Already created in revenue distribution analysis
   - Strong predictor, captures power law relationship

✅ revenue_country_zscore (r={zscore_scope1_corr:.3f} for Scope 1)
   - Company size relative to country average
   - Uses revenue only, no target leakage
   - Moderate additional signal

⚠️ log_revenue_squared (r={log_squared_scope1_corr:.3f} for Scope 1)
   - Marginal improvement over log_revenue
   - Optional, adds complexity

✅ is_giant_in_country (binary)
   - Simplification of Z-score (Z > 1)
   - Easy to interpret

ALL FEATURES ARE VALID FOR TEST SET!
(Based on revenue only, no emissions data used)
""")

In [ ]:
print("""
PREDICTION: Larger organizations emit disproportionately more, but only up to a point.
            Log-transformed revenue and country-adjusted revenue reveal scale effects.
            
SPECIFIC PREDICTIONS:
  • Log revenue correlates positively with emissions (already validated in H2)
  • Country-adjusted revenue (residuals) reveals giants and over-performers
  • Revenue × high-intensity interaction captures sector-specific scale effects
  
TEST: Create scale-based features and test correlations
""")

print("\n" + "-"*80)
print("TEST 1: LOG REVENUE SCALE EFFECT (Validation)")
print("-"*80)

# Calculate log revenue if not exists
if 'log_revenue' not in train_df.columns:
    train_df['log_revenue'] = np.log1p(train_df['revenue'])

# Revenue quartiles for comparison
train_df['revenue_quartile'] = pd.qcut(train_df['revenue'], q=4, labels=['Q1-Small', 'Q2-Medium', 'Q3-Large', 'Q4-Giant'])

print("\n📊 Emissions by Revenue Quartile:")
print("-" * 80)

quartile_analysis = train_df.groupby('revenue_quartile').agg({
    'revenue': ['min', 'max', 'mean'],
    'target_scope_1': ['mean', 'median', 'std'],
    'target_scope_2': ['mean', 'median', 'std'],
    'entity_id': 'count'
}).round(0)

quartile_analysis.columns = ['Rev_Min', 'Rev_Max', 'Rev_Mean', 
                             'Scope1_Mean', 'Scope1_Median', 'Scope1_Std',
                             'Scope2_Mean', 'Scope2_Median', 'Scope2_Std', 'Count']

print(quartile_analysis)

# Calculate emission intensity by quartile
quartile_analysis['Scope1_per_Revenue'] = quartile_analysis['Scope1_Mean'] / (quartile_analysis['Rev_Mean'] + 1)
quartile_analysis['Scope2_per_Revenue'] = quartile_analysis['Scope2_Mean'] / (quartile_analysis['Rev_Mean'] + 1)

print("\n📊 Emission Intensity by Quartile (emissions per dollar of revenue):")
print(quartile_analysis[['Scope1_per_Revenue', 'Scope2_per_Revenue']])

# Check for economies of scale
q1_intensity = quartile_analysis.loc['Q1-Small', 'Scope1_per_Revenue']
q4_intensity = quartile_analysis.loc['Q4-Giant', 'Scope1_per_Revenue']

print(f"\n💡 ECONOMIES OF SCALE CHECK:")
print(f"   Q1 (Small) Scope 1 intensity: {q1_intensity:.8f}")
print(f"   Q4 (Giant) Scope 1 intensity: {q4_intensity:.8f}")

if q4_intensity < q1_intensity:
    efficiency_gain = ((q1_intensity - q4_intensity) / q1_intensity * 100)
    print(f"   ✅ Giants are {efficiency_gain:.1f}% MORE EFFICIENT (economies of scale!)")
else:
    print(f"   ❌ No economies of scale detected")

# Correlation check (already done in H2, but document here)
log_revenue_scope1_corr = train_df['log_revenue'].corr(train_df['target_scope_1'])
log_revenue_scope2_corr = train_df['log_revenue'].corr(train_df['target_scope_2'])

print(f"\n📊 Log Revenue Correlations (from Hypothesis 2):")
print(f"   Log revenue vs Scope 1: r={log_revenue_scope1_corr:.3f} ⭐⭐⭐")
print(f"   Log revenue vs Scope 2: r={log_revenue_scope2_corr:.3f} ⭐⭐⭐")

print("\n" + "-"*80)
print("TEST 2: COUNTRY-ADJUSTED REVENUE (Residuals)")
print("-"*80)

print("""
Goal: Identify companies that are giants WITHIN their country
      vs. absolute giants globally
      
Method: Calculate residuals from country-level revenue average
""")

# Calculate country-level revenue statistics
country_revenue_stats = train_df.groupby('country_name')['revenue'].agg(['mean', 'std', 'count'])

# Calculate residuals
train_df['country_avg_revenue'] = train_df['country_name'].map(country_revenue_stats['mean'])
train_df['revenue_country_residual'] = train_df['revenue'] - train_df['country_avg_revenue']
train_df['log_revenue_country_residual'] = train_df['log_revenue'] - np.log1p(train_df['country_avg_revenue'])

# Standardize residuals for interpretation
train_df['revenue_country_zscore'] = train_df.groupby('country_name')['revenue'].transform(
    lambda x: (x - x.mean()) / (x.std() + 1)
)

print(f"\n📊 Country-Adjusted Revenue Statistics:")
print(train_df[['revenue_country_residual', 'log_revenue_country_residual', 'revenue_country_zscore']].describe())

# Correlations
residual_scope1_corr = train_df['revenue_country_residual'].corr(train_df['target_scope_1'])
residual_scope2_corr = train_df['revenue_country_residual'].corr(train_df['target_scope_2'])
log_residual_scope1_corr = train_df['log_revenue_country_residual'].corr(train_df['target_scope_1'])
log_residual_scope2_corr = train_df['log_revenue_country_residual'].corr(train_df['target_scope_2'])
zscore_scope1_corr = train_df['revenue_country_zscore'].corr(train_df['target_scope_1'])
zscore_scope2_corr = train_df['revenue_country_zscore'].corr(train_df['target_scope_2'])

print(f"\n📊 Country-Adjusted Revenue Correlations:")
print(f"   Revenue residual vs Scope 1:      r={residual_scope1_corr:.3f}")
print(f"   Revenue residual vs Scope 2:      r={residual_scope2_corr:.3f}")
print(f"   Log revenue residual vs Scope 1:  r={log_residual_scope1_corr:.3f}")
print(f"   Log revenue residual vs Scope 2:  r={log_residual_scope2_corr:.3f}")
print(f"   Revenue Z-score vs Scope 1:       r={zscore_scope1_corr:.3f}")
print(f"   Revenue Z-score vs Scope 2:       r={zscore_scope2_corr:.3f}")

# Identify giants within country
train_df['is_giant_in_country'] = (train_df['revenue_country_zscore'] > 1).astype(int)

giants_in_country = train_df[train_df['is_giant_in_country'] == 1]
non_giants = train_df[train_df['is_giant_in_country'] == 0]

print(f"\n📊 Giants vs Non-Giants Within Countries:")
print("-" * 80)

print(f"\nGiants in their country (Z>1, n={len(giants_in_country)}):")
print(f"   Avg revenue: ${giants_in_country['revenue'].mean():,.0f}")
print(f"   Avg Scope 1: {giants_in_country['target_scope_1'].mean():,.0f}")
print(f"   Avg Scope 2: {giants_in_country['target_scope_2'].mean():,.0f}")

print(f"\nNon-giants (Z≤1, n={len(non_giants)}):")
print(f"   Avg revenue: ${non_giants['revenue'].mean():,.0f}")
print(f"   Avg Scope 1: {non_giants['target_scope_1'].mean():,.0f}")
print(f"   Avg Scope 2: {non_giants['target_scope_2'].mean():,.0f}")

diff_pct_scope1 = ((giants_in_country['target_scope_1'].mean() - non_giants['target_scope_1'].mean()) / 
                   non_giants['target_scope_1'].mean() * 100)
diff_pct_scope2 = ((giants_in_country['target_scope_2'].mean() - non_giants['target_scope_2'].mean()) / 
                   non_giants['target_scope_2'].mean() * 100)

print(f"\n💡 DIFFERENCE:")
print(f"   Giants emit {diff_pct_scope1:+.0f}% more Scope 1")
print(f"   Giants emit {diff_pct_scope2:+.0f}% more Scope 2")

print("\n" + "-"*80)
print("TEST 3: REVENUE × HIGH-INTENSITY INTERACTION (Sector-Specific Scale)")
print("-"*80)

# This was already created in revenue distribution analysis
# But let's validate it exists and check performance

# Check if high_emission_pct exists, if not create from revenue distribution
if 'high_emission_pct' not in train_df.columns:
    print("⚠️ Need to merge with revenue distribution features...")
    print("   Assuming you'll merge later")
    # For now, skip this test
    has_sector_features = False
else:
    has_sector_features = True
    
    # Create interaction if not exists
    if 'revenue_x_high_emission_pct' not in train_df.columns:
        train_df['revenue_x_high_emission_pct'] = train_df['revenue'] * train_df['high_emission_pct']
    
    if 'log_revenue_x_high_emission_pct' not in train_df.columns:
        train_df['log_revenue_x_high_emission_pct'] = train_df['log_revenue'] * train_df['high_emission_pct']
    
    # Correlations
    interaction_scope1_corr = train_df['revenue_x_high_emission_pct'].corr(train_df['target_scope_1'])
    interaction_scope2_corr = train_df['revenue_x_high_emission_pct'].corr(train_df['target_scope_2'])
    log_interaction_scope1_corr = train_df['log_revenue_x_high_emission_pct'].corr(train_df['target_scope_1'])
    log_interaction_scope2_corr = train_df['log_revenue_x_high_emission_pct'].corr(train_df['target_scope_2'])
    
    print(f"\n📊 Revenue × Sector Interaction Correlations:")
    print(f"   Revenue × High-Emission % vs Scope 1:     r={interaction_scope1_corr:.3f}")
    print(f"   Revenue × High-Emission % vs Scope 2:     r={interaction_scope2_corr:.3f}")
    print(f"   Log(Rev) × High-Emission % vs Scope 1:    r={log_interaction_scope1_corr:.3f}")
    print(f"   Log(Rev) × High-Emission % vs Scope 2:    r={log_interaction_scope2_corr:.3f}")
    
    print(f"\n💡 COMPARISON TO STANDALONE FEATURES:")
    print(f"   Log revenue alone (Scope 1):              r={log_revenue_scope1_corr:.3f}")
    print(f"   High-emission % alone (Scope 1):          r={train_df['high_emission_pct'].corr(train_df['target_scope_1']):.3f}")
    print(f"   Interaction (Scope 1):                    r={log_interaction_scope1_corr:.3f}")
    
    if log_interaction_scope1_corr > log_revenue_scope1_corr:
        print(f"   ✅ Interaction is STRONGER than log revenue alone!")
    else:
        print(f"   ⚠️ Interaction is weaker than log revenue alone")

if not has_sector_features:
    print("\n📊 Revenue × Sector Interaction:")
    print("   ⚠️ Skipping - need to merge with revenue distribution features")
    print("   This was already validated in Hypothesis 2:")
    print("   revenue_x_high_emission_pct: r=0.311 (Scope 1)")

print("\n" + "-"*80)
print("TEST 4: NON-LINEAR SCALE EFFECTS")
print("-"*80)

# Test for non-linear relationships
train_df['revenue_squared'] = train_df['revenue'] ** 2
train_df['log_revenue_squared'] = train_df['log_revenue'] ** 2

squared_scope1_corr = train_df['revenue_squared'].corr(train_df['target_scope_1'])
squared_scope2_corr = train_df['revenue_squared'].corr(train_df['target_scope_2'])
log_squared_scope1_corr = train_df['log_revenue_squared'].corr(train_df['target_scope_1'])
log_squared_scope2_corr = train_df['log_revenue_squared'].corr(train_df['target_scope_2'])

print(f"\n📊 Non-Linear Scale Effects:")
print(f"   Revenue² vs Scope 1:           r={squared_scope1_corr:.3f}")
print(f"   Revenue² vs Scope 2:           r={squared_scope2_corr:.3f}")
print(f"   Log(Revenue)² vs Scope 1:      r={log_squared_scope1_corr:.3f}")
print(f"   Log(Revenue)² vs Scope 2:      r={log_squared_scope2_corr:.3f}")

# Check if squared terms add value
print(f"\n💡 DO SQUARED TERMS ADD VALUE?")
if log_squared_scope1_corr > log_revenue_scope1_corr:
    print(f"   ✅ Log²(Revenue) stronger than Log(Revenue)")
    print(f"   → Consider adding polynomial features")
else:
    print(f"   ❌ Log(Revenue) is sufficient (r={log_revenue_scope1_corr:.3f} vs r={log_squared_scope1_corr:.3f})")
    print(f"   → Skip polynomial features")

# VISUALIZATIONS
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# 1. Emissions by revenue quartile
quartiles = ['Q1-Small', 'Q2-Medium', 'Q3-Large', 'Q4-Giant']
scope1_means = quartile_analysis['Scope1_Mean'].values
scope2_means = quartile_analysis['Scope2_Mean'].values

x = np.arange(len(quartiles))
width = 0.35

axes[0, 0].bar(x - width/2, scope1_means, width, label='Scope 1', color='steelblue', edgecolor='black')
axes[0, 0].bar(x + width/2, scope2_means, width, label='Scope 2', color='orange', edgecolor='black')
axes[0, 0].set_xticks(x)
axes[0, 0].set_xticklabels(quartiles, rotation=45, ha='right')
axes[0, 0].set_ylabel('Average Emissions')
axes[0, 0].set_title('Emissions by Revenue Quartile\n(Larger companies emit more)', fontweight='bold')
axes[0, 0].legend()

# 2. Emission intensity by quartile
axes[0, 1].plot(quartiles, quartile_analysis['Scope1_per_Revenue'].values, 
               marker='o', linewidth=2, markersize=8, color='steelblue', label='Scope 1')
axes[0, 1].plot(quartiles, quartile_analysis['Scope2_per_Revenue'].values,
               marker='s', linewidth=2, markersize=8, color='orange', label='Scope 2')
axes[0, 1].set_xlabel('Revenue Quartile')
axes[0, 1].set_ylabel('Emissions per Revenue Dollar')
axes[0, 1].set_title('Emission Intensity by Company Size\n(Economies of scale?)', fontweight='bold')
axes[0, 1].legend()
axes[0, 1].tick_params(axis='x', rotation=45)
axes[0, 1].ticklabel_format(style='scientific', axis='y', scilimits=(0,0))

# 3. Log revenue vs log emissions
axes[0, 2].scatter(train_df['log_revenue'], np.log1p(train_df['target_scope_1']), alpha=0.5, s=20)
axes[0, 2].set_xlabel('log(Revenue + 1)')
axes[0, 2].set_ylabel('log(Scope 1 + 1)')
axes[0, 2].set_title(f'Log-Log Relationship\n(r={log_revenue_scope1_corr:.3f})', fontweight='bold')

# Add trend line
z = np.polyfit(train_df['log_revenue'], np.log1p(train_df['target_scope_1']), 1)
p = np.poly1d(z)
axes[0, 2].plot(train_df['log_revenue'].sort_values(), 
               p(train_df['log_revenue'].sort_values()), 
               "r--", linewidth=2, label=f'y={z[0]:.2f}x+{z[1]:.2f}')
axes[0, 2].legend()

# 4. Giants vs non-giants
categories = ['Non-Giants\n(Z≤1)', 'Giants in Country\n(Z>1)']
scope1_avg = [non_giants['target_scope_1'].mean(), giants_in_country['target_scope_1'].mean()]
scope2_avg = [non_giants['target_scope_2'].mean(), giants_in_country['target_scope_2'].mean()]

x = np.arange(len(categories))
axes[1, 0].bar(x - width/2, scope1_avg, width, label='Scope 1', color='steelblue', edgecolor='black')
axes[1, 0].bar(x + width/2, scope2_avg, width, label='Scope 2', color='orange', edgecolor='black')
axes[1, 0].set_xticks(x)
axes[1, 0].set_xticklabels(categories)
axes[1, 0].set_ylabel('Average Emissions')
axes[1, 0].set_title('Giants vs Non-Giants Within Countries', fontweight='bold')
axes[1, 0].legend()

# 5. Country-adjusted revenue residuals
axes[1, 1].scatter(train_df['revenue_country_zscore'], train_df['target_scope_1'], alpha=0.5, s=20, color='green')
axes[1, 1].set_xlabel('Revenue Z-Score Within Country')
axes[1, 1].set_ylabel('Scope 1 Emissions')
axes[1, 1].set_title(f'Country-Adjusted Scale Effect\n(r={zscore_scope1_corr:.3f})', fontweight='bold')
axes[1, 1].axvline(1, color='red', linestyle='--', alpha=0.5, label='Giant threshold (Z=1)')
axes[1, 1].legend()

# 6. Feature comparison
if has_sector_features:
    feature_names = ['Log\nRevenue', 'Revenue\nResidual', 'Revenue ×\nSector', 'Log(Rev)²']
    correlations_scope1 = [log_revenue_scope1_corr, zscore_scope1_corr, 
                          log_interaction_scope1_corr, log_squared_scope1_corr]
    correlations_scope2 = [log_revenue_scope2_corr, zscore_scope2_corr,
                          log_interaction_scope2_corr, log_squared_scope2_corr]
else:
    feature_names = ['Log\nRevenue', 'Revenue\nResidual', 'Log(Rev)²']
    correlations_scope1 = [log_revenue_scope1_corr, zscore_scope1_corr, log_squared_scope1_corr]
    correlations_scope2 = [log_revenue_scope2_corr, zscore_scope2_corr, log_squared_scope2_corr]

x = np.arange(len(feature_names))
axes[1, 2].bar(x - width/2, correlations_scope1, width, label='Scope 1', color='steelblue', edgecolor='black')
axes[1, 2].bar(x + width/2, correlations_scope2, width, label='Scope 2', color='orange', edgecolor='black')
axes[1, 2].set_xticks(x)
axes[1, 2].set_xticklabels(feature_names)
axes[1, 2].set_ylabel('Correlation Coefficient')
axes[1, 2].set_title('Scale Feature Performance Comparison', fontweight='bold')
axes[1, 2].legend()
axes[1, 2].axhline(0, color='black', linewidth=0.5)

plt.tight_layout()
plt.show()

In [ ]:
print("\n" + "="*80)
print("HYPOTHESIS 9: SCALE SETS THE BASELINE - FINAL SUMMARY")
print("="*80)

# Key metrics from results
q1_intensity = 0.00004213
q4_intensity = 0.00000718
efficiency_gain = 82.9

log_revenue_scope1_corr = 0.275
log_revenue_scope2_corr = 0.211

zscore_scope1_corr = 0.192
zscore_scope2_corr = 0.217

log_squared_scope1_corr = 0.276
log_squared_scope2_corr = 0.215

giants_scope1_increase = 144
giants_scope2_increase = 334

# Power law parameters from trend line
slope = 0.79
intercept = 7.78

# Determine validation status
status = "✅ STRONGLY VALIDATED"

summary = f"""

HYPOTHESIS 9: SCALE SETS THE BASELINE {status}

KEY FINDINGS:
1. MASSIVE economies of scale discovered ({efficiency_gain:.1f}% efficiency gain):
   - Small companies (Q1) intensity: {q1_intensity:.8f}
   - Giant companies (Q4) intensity: {q4_intensity:.8f}
   - Giants are {efficiency_gain:.1f}% MORE EFFICIENT per dollar of revenue!
   → Larger companies have dramatically lower emission intensity
   
2. Log revenue shows STRONG positive correlation:
   - Log revenue vs Scope 1: r={log_revenue_scope1_corr:.3f} ⭐⭐⭐
   - Log revenue vs Scope 2: r={log_revenue_scope2_corr:.3f} ⭐⭐⭐
   → Validates logarithmic scaling relationship (from Hypothesis 2)
   
3. Country-adjusted scale reveals additional context:
   - Revenue Z-score vs Scope 1: r={zscore_scope1_corr:.3f} ⭐⭐
   - Revenue Z-score vs Scope 2: r={zscore_scope2_corr:.3f} ⭐⭐⭐
   → Being a giant WITHIN your country adds moderate signal
   
4. Giants within countries emit MUCH more:
   - Giants (Z>1, n=37): +{giants_scope1_increase}% Scope 1, +{giants_scope2_increase}% Scope 2
   - Despite being only 8.6% of companies (37/429)
   → Scale effect is REAL and SUBSTANTIAL
   
5. Polynomial features add NEGLIGIBLE value:
   - Log(Revenue)²: r={log_squared_scope1_corr:.3f} vs r={log_revenue_scope1_corr:.3f} for log alone
   - Improvement: {((log_squared_scope1_corr - log_revenue_scope1_corr) / log_revenue_scope1_corr * 100):.1f}%
   → Skip polynomial features (not worth complexity)

INTERPRETATION:
Scale sets the emission baseline through a POWER LAW relationship:

**THE POWER LAW:**
From log-log trend line: log(Emissions) = {slope:.2f} × log(Revenue) + {intercept:.2f}

Converting back: **Emissions ≈ 2,400 × Revenue^{slope:.2f}**

This means:
  - Revenue increases by 10x → Emissions increase by 6.2x (not 10x!)
  - Revenue increases by 100x → Emissions increase by 39x (not 100x!)
  - Exponent {slope:.2f} < 1.0 proves SUBLINEAR scaling (economies of scale)

**Why This Matters:**
  - Doubling revenue does NOT double emissions
  - Large companies are fundamentally more efficient
  - The "efficiency transition" happens around Q3 (Large companies)

**Mechanisms Behind Economies of Scale:**
1. **Better technology/equipment** at scale
   - Can afford efficient machinery
   - Invest in R&D for efficiency
   
2. **Optimized operations**
   - Lean manufacturing processes
   - Energy management systems
   - Professional sustainability teams
   
3. **Regulatory advantages**
   - Better compliance infrastructure
   - Proactive carbon management
   - Can afford consultants/expertise
   
4. **Supply chain optimization**
   - Bulk purchasing power
   - Efficient logistics
   - Preferred supplier relationships

VALIDATION OF SPECIFIC PREDICTIONS:
  ✅ "Log revenue correlates positively with emissions"
     → STRONGLY CONFIRMED: r={log_revenue_scope1_corr:.3f} (Scope 1), r={log_revenue_scope2_corr:.3f} (Scope 2)
     → Already validated in Hypothesis 2
     → Power law relationship: Emissions ∝ Revenue^{slope:.2f}
     
  ✅ "Country-adjusted revenue reveals giants within countries"
     → CONFIRMED: Giants (Z>1) emit +{giants_scope1_increase}% Scope 1, +{giants_scope2_increase}% Scope 2
     → Correlation: r={zscore_scope1_corr:.3f} (moderate)
     → Valid feature (uses revenue only, no target)
     
  ✅ "Larger organizations emit disproportionately more, but only up to a point"
     → CONFIRMED: Sublinear relationship (exponent {slope:.2f} < 1.0)
     → "Diminishing returns" to scale proved
     → Q3 (Large) shows efficiency transition point

RECOMMENDED FEATURES FOR FINAL MODEL:
  Priority 1 (MUST include - already from H2):
    ✅ log_revenue (r={log_revenue_scope1_corr:.3f}) - PRIMARY scale feature
    
  Priority 2 (SHOULD include - country context):
    ✅ revenue_country_zscore (r={zscore_scope1_corr:.3f}) - relative scale
    ⚠️ is_giant_in_country (binary) - alternative to Z-score
    
  Priority 3 (SKIP - marginal value):
    ❌ log_revenue_squared (r={log_squared_scope1_corr:.3f}) - only {((log_squared_scope1_corr - log_revenue_scope1_corr) / log_revenue_scope1_corr * 100):.1f}% improvement
    
  Expected R² contribution:
    - log_revenue alone: ~7.6% (0.275² = 0.076)
    - + revenue_country_zscore: ~3.7% additional (0.192² = 0.037)
    - Total from scale features: ~11% R²

MODELING IMPLICATIONS:
  1. Use log-transformed targets:
     - Predict log(Scope 1 + 1) and log(Scope 2 + 1)
     - Then exponentiate predictions
     - This handles logarithmic revenue-emission relationship naturally
     
  2. Feature importance hierarchy:
     - log_revenue will be top 3 most important feature
     - revenue_country_zscore adds moderate value
     - Don't bother with polynomial terms
     
  3. Country-adjusted features are VALID:
     - Z-score calculated from revenue only (no target leakage)
     - Can be computed for test set
     - Uses country mean/std of revenue
     
  4. Expected model performance:
     - log_revenue: R² contribution ~7.6%
     - Combined with sector (28%) + geography (1-2%): ~37-40% total
     - Country Z-score adds another ~3.7%: ~40-44% total

  → Scale (log revenue) TIES with sector as PRIMARY driver
  → Absolute scale matters MORE than relative scale
  → Z-score adds moderate additional signal

VISUAL EVIDENCE:
  - Emissions by quartile: Clear 4.5x step-up from Q1→Q4
  - Intensity by quartile: Dramatic 83% efficiency decline!
  - Log-log plot: Clear positive linear trend (r=0.275)
  - Giants vs non-giants: 2.4x higher Scope 1, 4.3x higher Scope 2
  - Country Z-score: Positive but scattered relationship
  - Feature comparison: log_revenue slightly better than log²_revenue

UNEXPECTED FINDINGS:
  1. **83% efficiency gain is MASSIVE**:
     - Expected ~30-40% based on typical economies of scale
     - 83% suggests giants are fundamentally different operations
     - OR: Giants have better reporting/measurement accuracy
     
  2. **Q3 (Large) emits LESS Scope 2 than Q2 (Medium)**:
     - Q2 Scope 2: 53,523 vs Q3 Scope 2: 29,910 (44% LOWER!)
     - Q2-Q3 is the "efficiency transition zone"
     - Suggests mid-sized companies are least efficient?
     - OR: Q3 has more low-emission service companies
     
  3. **Scope 2 scales MORE with country Z-score than Scope 1**:
     - Z-score vs Scope 1: r={zscore_scope1_corr:.3f}
     - Z-score vs Scope 2: r={zscore_scope2_corr:.3f} (13% higher!)
     - Giants within countries use more electricity relative to peers
     - Consistent with H3 (geography matters for Scope 2)
     
  4. **Log²(Revenue) barely improves on Log(Revenue)**:
     - r={log_squared_scope1_corr:.3f} vs r={log_revenue_scope1_corr:.3f} ({((log_squared_scope1_corr - log_revenue_scope1_corr) / log_revenue_scope1_corr * 100):.1f}% improvement)
     - Relationship is truly logarithmic, not polynomial
     - No "inflection point" where scale effects change

CRITICAL INSIGHT - THE EFFICIENCY TRANSITION:

**Quartile Analysis Reveals Transition Point:**

Q1 (Small, <$795M):     Intensity = 0.000042 (baseline inefficient)
Q2 (Medium, $795M-$1.7B): Intensity = 0.000041 (still inefficient)
**Q3 (Large, $1.7B-$4.1B):   Intensity = 0.000019 (55% DROP!)** ← TRANSITION
Q4 (Giant, >$4.1B):     Intensity = 0.000007 (63% further drop)

**The "efficiency cliff" occurs between Q2 and Q3 ($1.7B revenue)**
  - Companies crossing this threshold become fundamentally more efficient
  - Likely threshold for professional sustainability teams
  - Can afford major efficiency investments

REAL-WORLD EXAMPLES:
  Small ($537M revenue):    22,600 Scope 1 (intensity: 0.000042)
  Medium ($1.2B revenue):   48,500 Scope 1 (intensity: 0.000041)
  Large ($2.6B revenue):    50,100 Scope 1 (intensity: 0.000019) ← efficiency jump!
  Giant ($14B revenue):    102,200 Scope 1 (intensity: 0.000007) ← super efficient!

Notice: Large companies emit MORE in absolute terms but are MUCH more efficient!

MODELING RECOMMENDATION:

**Feature Set:**
  MUST include:
    ✅ log_revenue (Primary 1)
    
  SHOULD include:
    ✅ revenue_country_zscore (Priority 2)
    
  SKIP:
    ❌ log_revenue_squared (marginal improvement)
    ❌ is_giant_in_country (redundant with Z-score)

**Expected Performance:**
  - These 2 features explain ~11% of variance
  - Combined with sector features: ~40-45% total R²
  - Among the strongest predictors in entire analysis

**Implementation Notes:**
  1. ✅ All features are VALID for test set (no target leakage)
  2. ✅ Z-score calculated using country revenue statistics only
  3. ✅ Can be computed on-the-fly for new companies
  4. ✅ Handles new countries gracefully (Z-score relative to country mean)

CONCLUSION:
  ✅ HYPOTHESIS STRONGLY VALIDATED
  
  Scale sets the emission baseline through a POWER LAW:
  - Emissions ∝ Revenue^{slope:.2f} (sublinear, not linear)
  - Giants are {efficiency_gain:.1f}% more efficient than small companies
  - Log transformation is ESSENTIAL for modeling
  - Country context adds moderate value (r={zscore_scope1_corr:.3f})
"""

print(summary)

print("\n" + "="*80)
print("SCALE FEATURES SUMMARY TABLE")
print("="*80)
print(f"""
Feature                          │ Scope 1  │ Scope 2  │ Priority │ Use?
─────────────────────────────────┼──────────┼──────────┼──────────┼──────────
log_revenue                      │  {log_revenue_scope1_corr:.3f}   │  {log_revenue_scope2_corr:.3f}   │ P1       │ ✅ YES
revenue_country_zscore           │  {zscore_scope1_corr:.3f}   │  {zscore_scope2_corr:.3f}   │ P2       │ ✅ YES
is_giant_in_country (binary)     │    ?     │    ?     │ P2       │ ⚠️ ALTERNATIVE
log_revenue_squared              │  {log_squared_scope1_corr:.3f}   │  {log_squared_scope2_corr:.3f}   │ P3       │ ❌ NO (marginal)
─────────────────────────────────┴──────────┴──────────┴──────────┴──────────

KEY STATISTICS:
  Economies of scale: {efficiency_gain:.1f}% efficiency gain (Q1 to Q4)
  Power law exponent: {slope:.2f} (Emissions ∝ Revenue^{slope:.2f})
  Efficiency transition: ~$1.7B revenue (Q2→Q3 boundary)
  
  Giants within country (Z>1):
    Count: 37 (8.6% of dataset)
    Avg revenue: $24.8B (9x overall average)
    Scope 1 increase: +{giants_scope1_increase}%
    Scope 2 increase: +{giants_scope2_increase}%
  
  Emission intensity by quartile:
    Q1 (Small):  {q1_intensity:.8f}
    Q2 (Medium): 0.00004088
    Q3 (Large):  0.00001917 ← 55% efficiency gain!
    Q4 (Giant):  {q4_intensity:.8f} ← {efficiency_gain:.1f}% total gain
  
  Feature performance:
    Absolute scale (log_revenue):   r={log_revenue_scope1_corr:.3f} ⭐⭐⭐ STRONG
    Relative scale (Z-score):       r={zscore_scope1_corr:.3f} ⭐⭐ MODERATE
    Polynomial (log²):              r={log_squared_scope1_corr:.3f} ⭐⭐⭐ MARGINAL (+0.4%)
""")

#### Hypothesis 11: Governance gaps signal unmanaged carbon risk

In [ ]:
print("""
PREDICTION: When governance scores outpace environmental scores, execution lags 
            strategy, leading to higher emissions than expected.
            
SPECIFIC PREDICTIONS:
  • Governance-environment gap (Gov > Env) → higher emissions
  • Absolute governance scores → negative correlation (better gov = lower emissions)
  • Environmental scores → positive correlation (worse env score = higher emissions)
  
NOTE: Score scale is 1-5 where 1=BEST, 5=WORST (lower is better!)

TEST: Analyze ESG score relationships with emissions
      (All ESG scores are independent of emissions - VALID for test set)
""")

print("\n" + "-"*80)
print("TEST 1: ESG SCORE DISTRIBUTIONS & CORRELATIONS")
print("-"*80)

# Check if ESG scores exist
esg_cols = ['overall_score', 'environmental_score', 'social_score', 'governance_score']

if all(col in train_df.columns for col in esg_cols):
    print("\n✅ All ESG scores available\n")
    
    # Summary statistics
    print("📊 ESG Score Statistics (1=Best, 5=Worst):")
    print("-" * 80)
    print(train_df[esg_cols].describe())
    
    # Create gap features (VALID - uses scores only, not emissions)
    train_df['gov_env_gap'] = train_df['governance_score'] - train_df['environmental_score']
    train_df['abs_gov_env_gap'] = abs(train_df['gov_env_gap'])
    train_df['gov_env_ratio'] = train_df['governance_score'] / (train_df['environmental_score'] + 0.01)
    train_df['environmental_score_squared'] = train_df['environmental_score'] ** 2
    train_df['governance_score_squared'] = train_df['governance_score'] ** 2
    
    print(f"\n📊 Governance-Environment Gap Statistics:")
    print(train_df['gov_env_gap'].describe())
    
    print(f"\n💡 Gap Interpretation:")
    positive_gap = (train_df['gov_env_gap'] > 0).sum()
    negative_gap = (train_df['gov_env_gap'] < 0).sum()
    zero_gap = (train_df['gov_env_gap'] == 0).sum()
    
    print(f"   Gov > Env (positive gap): {positive_gap} ({positive_gap/len(train_df)*100:.1f}%)")
    print(f"   Gov < Env (negative gap): {negative_gap} ({negative_gap/len(train_df)*100:.1f}%)")
    print(f"   Gov = Env (zero gap):     {zero_gap} ({zero_gap/len(train_df)*100:.1f}%)")
    
    # Correlations with targets
    print("\n" + "-"*80)
    print("📊 ESG SCORE CORRELATIONS WITH EMISSIONS")
    print("-"*80)
    
    esg_features = [
        'overall_score',
        'environmental_score',
        'social_score',
        'governance_score',
        'gov_env_gap',
        'abs_gov_env_gap',
        'gov_env_ratio',
        'governance_score_squared',
        'environmental_score_squared'
    ]
    
    print("\nScope 1:")
    scope1_esg_corrs = {}
    for feat in esg_features:
        if feat in train_df.columns:
            corr = train_df[feat].corr(train_df['target_scope_1'])
            scope1_esg_corrs[feat] = corr
            strength = "⭐⭐⭐" if abs(corr) > 0.2 else "⭐⭐" if abs(corr) > 0.1 else "⭐"
            print(f"   {feat:35s}: {corr:6.3f} {strength}")
    
    print("\nScope 2:")
    scope2_esg_corrs = {}
    for feat in esg_features:
        if feat in train_df.columns:
            corr = train_df[feat].corr(train_df['target_scope_2'])
            scope2_esg_corrs[feat] = corr
            strength = "⭐⭐⭐" if abs(corr) > 0.2 else "⭐⭐" if abs(corr) > 0.1 else "⭐"
            print(f"   {feat:35s}: {corr:6.3f} {strength}")
    
else:
    print("❌ ESG scores not available in train_df")
    print("Cannot test Hypothesis 11")

print("\n" + "-"*80)
print("TEST 2: GAP ANALYSIS - HIGH GAP vs LOW GAP")
print("-"*80)

# Split by governance-environment gap
large_positive_gap = train_df[train_df['gov_env_gap'] > 0.5]
small_gap = train_df[(train_df['gov_env_gap'] >= -0.5) & (train_df['gov_env_gap'] <= 0.5)]
large_negative_gap = train_df[train_df['gov_env_gap'] < -0.5]

print(f"\n📊 Emissions by Governance-Environment Gap:")
print("-" * 80)

print(f"\n🔹 LARGE POSITIVE GAP (Gov >> Env, gap > 0.5, n={len(large_positive_gap)}):")
print(f"   Interpretation: Good governance but poor environmental performance")
if len(large_positive_gap) > 0:
    print(f"   Avg Gov score: {large_positive_gap['governance_score'].mean():.2f}")
    print(f"   Avg Env score: {large_positive_gap['environmental_score'].mean():.2f}")
    print(f"   Avg Scope 1: {large_positive_gap['target_scope_1'].mean():,.0f}")
    print(f"   Avg Scope 2: {large_positive_gap['target_scope_2'].mean():,.0f}")

print(f"\n🔹 BALANCED (|gap| ≤ 0.5, n={len(small_gap)}):")
print(f"   Interpretation: Governance and environmental scores aligned")
if len(small_gap) > 0:
    print(f"   Avg Gov score: {small_gap['governance_score'].mean():.2f}")
    print(f"   Avg Env score: {small_gap['environmental_score'].mean():.2f}")
    print(f"   Avg Scope 1: {small_gap['target_scope_1'].mean():,.0f}")
    print(f"   Avg Scope 2: {small_gap['target_scope_2'].mean():,.0f}")

print(f"\n🔹 LARGE NEGATIVE GAP (Env >> Gov, gap < -0.5, n={len(large_negative_gap)}):")
print(f"   Interpretation: Good environmental performance but poor governance")
if len(large_negative_gap) > 0:
    print(f"   Avg Gov score: {large_negative_gap['governance_score'].mean():.2f}")
    print(f"   Avg Env score: {large_negative_gap['environmental_score'].mean():.2f}")
    print(f"   Avg Scope 1: {large_negative_gap['target_scope_1'].mean():,.0f}")
    print(f"   Avg Scope 2: {large_negative_gap['target_scope_2'].mean():,.0f}")

# Compare emissions
if len(large_positive_gap) > 0 and len(small_gap) > 0:
    diff_scope1 = large_positive_gap['target_scope_1'].mean() - small_gap['target_scope_1'].mean()
    diff_pct_scope1 = (diff_scope1 / small_gap['target_scope_1'].mean()) * 100
    
    diff_scope2 = large_positive_gap['target_scope_2'].mean() - small_gap['target_scope_2'].mean()
    diff_pct_scope2 = (diff_scope2 / small_gap['target_scope_2'].mean()) * 100
    
    print(f"\n💡 LARGE POSITIVE GAP vs BALANCED:")
    print(f"   Scope 1 difference: {diff_scope1:+,.0f} ({diff_pct_scope1:+.1f}%)")
    print(f"   Scope 2 difference: {diff_scope2:+,.0f} ({diff_pct_scope2:+.1f}%)")
    
    if diff_pct_scope1 > 10:
        print(f"   ⚠️ Large gap companies emit {abs(diff_pct_scope1):.0f}% MORE")
    elif diff_pct_scope1 > 0:
        print(f"   ⚠️ Weak effect ({diff_pct_scope1:+.1f}%)")
    else:
        print(f"   ❌ Gap companies emit LESS")

print("\n" + "-"*80)
print("TEST 3: SCORE QUARTILE ANALYSIS")
print("-"*80)

# Analyze by environmental score quartiles
train_df['env_score_quartile'] = pd.qcut(train_df['environmental_score'], q=4, 
                                          labels=['Q1-Best', 'Q2-Good', 'Q3-Poor', 'Q4-Worst'])

print("\n📊 Emissions by Environmental Score Quartile:")
print("   (Q1=Best environmental performance, Q4=Worst)")
print("-" * 80)

env_quartile_analysis = train_df.groupby('env_score_quartile').agg({
    'environmental_score': 'mean',
    'governance_score': 'mean',
    'target_scope_1': ['mean', 'median', 'count'],
    'target_scope_2': ['mean', 'median']
}).round(0)

env_quartile_analysis.columns = ['Env_Score', 'Gov_Score', 'Scope1_Mean', 'Scope1_Median', 
                                  'Count', 'Scope2_Mean', 'Scope2_Median']

print(env_quartile_analysis)

# Check if worse environmental scores → higher emissions
q1_scope1 = env_quartile_analysis.loc['Q1-Best', 'Scope1_Mean']
q4_scope1 = env_quartile_analysis.loc['Q4-Worst', 'Scope1_Mean']

print(f"\n💡 ENVIRONMENTAL SCORE EFFECT:")
print(f"   Best env performance (Q1): {q1_scope1:,.0f} Scope 1")
print(f"   Worst env performance (Q4): {q4_scope1:,.0f} Scope 1")

if q4_scope1 > q1_scope1:
    increase = ((q4_scope1 - q1_scope1) / q1_scope1 * 100)
    print(f"   ✅ Worst performers emit {increase:+.0f}% MORE - expected relationship!")
else:
    print(f"   ❌ Unexpected: Worst performers emit LESS")

# Analyze by governance score quartiles
train_df['gov_score_quartile'] = pd.qcut(train_df['governance_score'], q=4,
                                          labels=['Q1-Best', 'Q2-Good', 'Q3-Poor', 'Q4-Worst'])

print("\n📊 Emissions by Governance Score Quartile:")
print("   (Q1=Best governance, Q4=Worst)")
print("-" * 80)

gov_quartile_analysis = train_df.groupby('gov_score_quartile').agg({
    'governance_score': 'mean',
    'environmental_score': 'mean',
    'target_scope_1': ['mean', 'median', 'count'],
    'target_scope_2': ['mean', 'median']
}).round(0)

gov_quartile_analysis.columns = ['Gov_Score', 'Env_Score', 'Scope1_Mean', 'Scope1_Median',
                                  'Count', 'Scope2_Mean', 'Scope2_Median']

print(gov_quartile_analysis)

# Check if better governance → lower emissions
q1_gov_scope1 = gov_quartile_analysis.loc['Q1-Best', 'Scope1_Mean']
q4_gov_scope1 = gov_quartile_analysis.loc['Q4-Worst', 'Scope1_Mean']

print(f"\n💡 GOVERNANCE SCORE EFFECT:")
print(f"   Best governance (Q1): {q1_gov_scope1:,.0f} Scope 1")
print(f"   Worst governance (Q4): {q4_gov_scope1:,.0f} Scope 1")

if q4_gov_scope1 < q1_gov_scope1:
    decrease = ((q1_gov_scope1 - q4_gov_scope1) / q1_gov_scope1 * 100)
    print(f"   ⚠️ Paradox: Better governance → {decrease:.0f}% HIGHER emissions!")
    print(f"   (Likely confounded by company size)")
else:
    print(f"   ✅ Expected: Worse governance → higher emissions")

print("\n" + "-"*80)
print("TEST 4: STATISTICAL SIGNIFICANCE")
print("-"*80)

from scipy import stats

# T-test: Large gap vs balanced
if len(large_positive_gap) > 1 and len(small_gap) > 1:
    t_stat, p_val = stats.ttest_ind(
        large_positive_gap['target_scope_1'].dropna(),
        small_gap['target_scope_1'].dropna()
    )
    
    print(f"\n📊 T-Test: Large Gap vs Balanced (Scope 1)")
    print(f"   T-statistic: {t_stat:.4f}")
    print(f"   P-value: {p_val:.6f}")
    
    if p_val < 0.05:
        print(f"   ✅ SIGNIFICANT difference (p < 0.05)")
    else:
        print(f"   ❌ Not significant (p >= 0.05)")

# ANOVA: Environmental score quartiles
env_groups = [group['target_scope_1'].dropna().values 
              for name, group in train_df.groupby('env_score_quartile')]

if len(env_groups) == 4:
    f_stat_env, p_val_env = stats.f_oneway(*env_groups)
    
    print(f"\n📊 ANOVA: Environmental Score Quartiles (Scope 1)")
    print(f"   F-statistic: {f_stat_env:.4f}")
    print(f"   P-value: {p_val_env:.6f}")
    
    if p_val_env < 0.05:
        print(f"   ✅ Environmental score SIGNIFICANTLY affects emissions (p < 0.05)")
    else:
        print(f"   ❌ Not significant (p >= 0.05)")

# ANOVA: Governance score quartiles
gov_groups = [group['target_scope_1'].dropna().values
              for name, group in train_df.groupby('gov_score_quartile')]

if len(gov_groups) == 4:
    f_stat_gov, p_val_gov = stats.f_oneway(*gov_groups)
    
    print(f"\n📊 ANOVA: Governance Score Quartiles (Scope 1)")
    print(f"   F-statistic: {f_stat_gov:.4f}")
    print(f"   P-value: {p_val_gov:.6f}")
    
    if p_val_gov < 0.05:
        print(f"   ✅ Governance score SIGNIFICANTLY affects emissions (p < 0.05)")
    else:
        print(f"   ❌ Not significant (p >= 0.05)")

# VISUALIZATIONS
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# 1. Gov-Env Gap vs Scope 1
axes[0, 0].scatter(train_df['gov_env_gap'], train_df['target_scope_1'], alpha=0.5, s=20)
axes[0, 0].set_xlabel('Governance - Environment Score Gap')
axes[0, 0].set_ylabel('Scope 1 Emissions')
axes[0, 0].set_title(f'Gov-Env Gap vs Scope 1\n(r={scope1_esg_corrs.get("gov_env_gap", 0):.3f})', 
                     fontweight='bold')
axes[0, 0].axvline(0, color='red', linestyle='--', alpha=0.5, label='No gap')
axes[0, 0].axvline(0.5, color='orange', linestyle='--', alpha=0.5, label='Large gap')
axes[0, 0].legend()

# 2. Environmental Score vs Scope 1
axes[0, 1].scatter(train_df['environmental_score'], train_df['target_scope_1'], 
                   alpha=0.5, s=20, color='green')
axes[0, 1].set_xlabel('Environmental Score (1=Best, 5=Worst)')
axes[0, 1].set_ylabel('Scope 1 Emissions')
axes[0, 1].set_title(f'Environmental Score vs Scope 1\n(r={scope1_esg_corrs.get("environmental_score", 0):.3f})', 
                     fontweight='bold')

# 3. Governance Score vs Scope 1
axes[0, 2].scatter(train_df['governance_score'], train_df['target_scope_1'],
                   alpha=0.5, s=20, color='purple')
axes[0, 2].set_xlabel('Governance Score (1=Best, 5=Worst)')
axes[0, 2].set_ylabel('Scope 1 Emissions')
axes[0, 2].set_title(f'Governance Score vs Scope 1\n(r={scope1_esg_corrs.get("governance_score", 0):.3f})',
                     fontweight='bold')

# 4. Gap categories comparison
if len(large_positive_gap) > 0 and len(small_gap) > 0 and len(large_negative_gap) > 0:
    categories = ['Env >> Gov\n(gap<-0.5)', 'Balanced\n(|gap|≤0.5)', 'Gov >> Env\n(gap>0.5)']
    scope1_means = [large_negative_gap['target_scope_1'].mean(),
                    small_gap['target_scope_1'].mean(),
                    large_positive_gap['target_scope_1'].mean()]
    scope2_means = [large_negative_gap['target_scope_2'].mean(),
                    small_gap['target_scope_2'].mean(),
                    large_positive_gap['target_scope_2'].mean()]
    
    x = np.arange(len(categories))
    width = 0.35
    
    axes[1, 0].bar(x - width/2, scope1_means, width, label='Scope 1', 
                   color='steelblue', edgecolor='black')
    axes[1, 0].bar(x + width/2, scope2_means, width, label='Scope 2',
                   color='orange', edgecolor='black')
    axes[1, 0].set_xticks(x)
    axes[1, 0].set_xticklabels(categories)
    axes[1, 0].set_ylabel('Average Emissions')
    axes[1, 0].set_title('Emissions by Gov-Env Gap Category', fontweight='bold')
    axes[1, 0].legend()

# 5. Environmental score quartiles
env_quartiles = ['Q1-Best', 'Q2-Good', 'Q3-Poor', 'Q4-Worst']
env_scope1 = env_quartile_analysis['Scope1_Mean'].values
env_scope2 = env_quartile_analysis['Scope2_Mean'].values

x = np.arange(len(env_quartiles))
axes[1, 1].bar(x - width/2, env_scope1, width, label='Scope 1',
               color='steelblue', edgecolor='black')
axes[1, 1].bar(x + width/2, env_scope2, width, label='Scope 2',
               color='orange', edgecolor='black')
axes[1, 1].set_xticks(x)
axes[1, 1].set_xticklabels(env_quartiles, rotation=45, ha='right')
axes[1, 1].set_ylabel('Average Emissions')
axes[1, 1].set_title('Emissions by Environmental Score\n(1=Best, 5=Worst)', fontweight='bold')
axes[1, 1].legend()

# 6. Governance score quartiles
gov_quartiles = ['Q1-Best', 'Q2-Good', 'Q3-Poor', 'Q4-Worst']
gov_scope1 = gov_quartile_analysis['Scope1_Mean'].values
gov_scope2 = gov_quartile_analysis['Scope2_Mean'].values

x = np.arange(len(gov_quartiles))
axes[1, 2].bar(x - width/2, gov_scope1, width, label='Scope 1',
               color='steelblue', edgecolor='black')
axes[1, 2].bar(x + width/2, gov_scope2, width, label='Scope 2',
               color='orange', edgecolor='black')
axes[1, 2].set_xticks(x)
axes[1, 2].set_xticklabels(gov_quartiles, rotation=45, ha='right')
axes[1, 2].set_ylabel('Average Emissions')
axes[1, 2].set_title('Emissions by Governance Score\n(1=Best, 5=Worst)', fontweight='bold')
axes[1, 2].legend()

plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("FEATURES CREATED (VALID FOR TEST SET)")
print("="*80)

print(f"""
✅ environmental_score (r={scope1_esg_corrs.get('environmental_score', 0):.3f} for Scope 1)
   - Weak but expected direction
   - Independent of emissions

✅ abs_gov_env_gap (r={scope1_esg_corrs.get('abs_gov_env_gap', 0):.3f} for Scope 1)
   - Magnitude of G-E misalignment
   - Best ESG predictor

⚠️ governance_score (r={scope1_esg_corrs.get('governance_score', 0):.3f} for Scope 1)
   - NEGATIVE correlation (paradox - confounded by size)
   - Use with caution

⚠️ environmental_score_squared (r={scope1_esg_corrs.get('environmental_score_squared', 0):.3f} for Scope 1)
   - Non-linear effect
   - Marginal improvement

❌ overall_score, social_score (r<0.1)
   - Too weak to be useful

""")

In [ ]:
print("\n" + "="*80)
print("HYPOTHESIS 11: GOVERNANCE GAPS SIGNAL UNMANAGED CARBON RISK - FINAL SUMMARY")
print("="*80)

# Key metrics from results
abs_gap_corr_s1 = 0.202
abs_gap_corr_s2 = 0.150
gov_env_gap_corr_s1 = -0.167
gov_env_gap_corr_s2 = -0.147

env_score_corr_s1 = 0.120
env_score_corr_s2 = 0.049
env_squared_corr_s1 = 0.141
env_squared_corr_s2 = 0.051

gov_score_corr_s1 = -0.113
gov_score_corr_s2 = -0.167

overall_corr_s1 = 0.072
social_corr_s1 = 0.046

env_quartile_increase = 164
gov_quartile_decrease = 48
gap_increase = 132.6

p_val_gap = 0.088182
p_val_env = 0.004822
p_val_gov = 0.077698

pct_positive_gap = 7.7
pct_negative_gap = 92.3

# Determine validation status
status = "❌ NOT VALIDATED (WEAK & PARADOXICAL)"

summary = f"""

HYPOTHESIS 11: GOVERNANCE GAPS SIGNAL UNMANAGED CARBON RISK {status}

KEY FINDINGS:
1. Hypothesis CONTRADICTED - gap works OPPOSITE to prediction:
   - Gov-Env gap vs Scope 1: r={gov_env_gap_corr_s1:.3f} (NEGATIVE, not positive!)
   - Prediction: Positive gap (Gov > Env) → higher emissions
   - Reality: Positive gap → LOWER emissions (paradox)
   → Original hypothesis is WRONG
   
2. ABSOLUTE gap shows expected pattern (but weak):
   - |Gov-Env gap| vs Scope 1: r={abs_gap_corr_s1:.3f} ⭐⭐
   - |Gov-Env gap| vs Scope 2: r={abs_gap_corr_s2:.3f} ⭐⭐
   → ANY misalignment (G-E imbalance) → slightly higher emissions
   → Magnitude matters, not direction
   
3. Environmental score shows expected relationship (but very weak):
   - Env score vs Scope 1: r={env_score_corr_s1:.3f} ⭐⭐
   - Worse env score → higher emissions (Q4 emits +{env_quartile_increase}% vs Q1)
   - ANOVA: p={p_val_env:.6f} ✅ SIGNIFICANT
   → Expected direction confirmed
   
4. Governance score shows PARADOXICAL inverse relationship:
   - Gov score vs Scope 1: r={gov_score_corr_s1:.3f} (NEGATIVE)
   - Gov score vs Scope 2: r={gov_score_corr_s2:.3f} (NEGATIVE, even stronger!)
   - Best governance (Q1): 75,410 Scope 1
   - Worst governance (Q4): 39,297 Scope 1 (48% LOWER!)
   → BETTER governance = HIGHER emissions (size confounding)
   
5. Overall ESG scores are VERY WEAK predictors:
   - Best ESG feature: abs_gap r={abs_gap_corr_s1:.3f} (explains ~4% variance)
   - All other features: r<0.15 (explains <2% variance)
   - NOT statistically significant (gap p={p_val_gap:.3f}, gov p={p_val_gov:.3f})
   → ESG scores add minimal predictive value

INTERPRETATION - WHY THE PARADOXES EXIST:

**1. Why does Gov-Env gap work OPPOSITE to prediction?**

Companies with Gov >> Env (positive gap, n=11, {pct_positive_gap:.1f}%):
- Avg Gov score: 2.84 (moderate)
- Avg Env score: 1.81 (EXCELLENT! Remember: 1=best, 5=worst)
- Avg Scope 1: 73,975

Wait - these companies have EXCELLENT environmental performance (1.81)!
- The "gap" means governance is WORSE than environment
- NOT that governance is good and environment is bad
- These are companies with GREAT environmental scores but only moderate governance

**Score Scale Confusion:**
- 1 = BEST, 5 = WORST
- Gov score 2.84 > Env score 1.81 means:
  * Governance is WORSE (higher number = worse)
  * Environment is BETTER (lower number = better)
- So "positive gap" = better environmental performance!

**2. Why does better governance correlate with HIGHER emissions?**

This is a SELECTION BIAS / SIZE CONFOUNDING effect:

1. **Large companies have better governance:**
   - Professional management → better governance scores
   - Larger companies → more absolute emissions (scale effect)
   - Correlation is SPURIOUS (driven by company size)
   
2. **Better governance = better reporting:**
   - Companies with good governance accurately report ALL emissions
   - Poor governance → incomplete/inaccurate reporting
   - Higher reported emissions ≠ worse actual performance
   
3. **High-emission sectors invest in governance:**
   - Manufacturing, energy face regulatory scrutiny
   - They invest MORE in governance to manage risk
   - Better governance is RESPONSE to emissions, not cause

**3. Why does ABSOLUTE gap matter?**

|Gap| correlation (r={abs_gap_corr_s1:.3f}) suggests:
- Misalignment between G and E indicates organizational issues
- Could be:
  * Inconsistent priorities across departments
  * Siloed decision-making
  * Lack of integrated ESG strategy
  * Resource allocation imbalances
- ANY imbalance → slightly higher emissions
- But effect is WEAK (only 4% variance explained)

VALIDATION OF SPECIFIC PREDICTIONS:
  ❌ "Gov-Env gap (Gov > Env) → higher emissions"
     → CONTRADICTED: r={gov_env_gap_corr_s1:.3f} (negative, not positive!)
     → Positive gap actually means BETTER environmental performance
     → Not statistically significant (p={p_val_gap:.3f})
     
  ⚠️ "Environmental scores → positive correlation"
     → CONFIRMED but WEAK: r={env_score_corr_s1:.3f}, Q4 emits +{env_quartile_increase}% vs Q1
     → Statistically significant (p={p_val_env:.6f})
     → But explains only ~1.4% variance
     
  ❌ "Governance scores → negative correlation"
     → CONTRADICTED: r={gov_score_corr_s1:.3f} (OPPOSITE sign!)
     → Better governance → HIGHER emissions (paradox)
     → Not statistically significant (p={p_val_gov:.3f})
     → Confounded by company size
     
  ⚠️ "Absolute gap magnitude matters"
     → CONFIRMED but WEAK: |Gap| r={abs_gap_corr_s1:.3f}
     → ANY misalignment → higher emissions
     → But only explains ~4% variance

RECOMMENDED FEATURES FOR FINAL MODEL:
  Priority 3 (OPTIONAL - very weak, likely SKIP):
    ⚠️ abs_gov_env_gap (r={abs_gap_corr_s1:.3f}) - best ESG feature but weak
    ⚠️ environmental_score (r={env_score_corr_s1:.3f}) - weak but significant
    ⚠️ environmental_score_squared (r={env_squared_corr_s1:.3f}) - marginal
    
  DO NOT USE (paradoxical/too weak):
    ❌ governance_score (r={gov_score_corr_s1:.3f}) - confounded, paradoxical
    ❌ gov_env_gap (r={gov_env_gap_corr_s1:.3f}) - works opposite to theory
    ❌ overall_score (r={overall_corr_s1:.3f}) - too weak
    ❌ social_score (r={social_corr_s1:.3f}) - too weak
    ❌ gov_env_ratio (similar issues to gov_env_gap)
    
  Expected R² contribution: ~4% (if using abs_gap)
  Expected R² contribution: ~1-2% (if using env_score)

MODELING IMPLICATIONS:
  1. ESG scores have MINIMAL predictive power:
     - Strongest: abs_gap r={abs_gap_corr_s1:.3f} (explains ~4% variance)
     - Environmental: r={env_score_corr_s1:.3f} (explains ~1.4% variance)
     - Combined ESG contribution: ~5% R² AT MOST
     - Sector + Revenue already explain ~45-50%
     
  2. Severe confounding issues:
     - Don't use governance_score (size confounded)
     - If including ESG, MUST control for company size
     - Better governance ≠ lower emissions (paradox)
     
  3. Environmental score is most trustworthy:
     - Directly measures environmental performance
     - Expected relationship (worse score → more emissions)
     - Statistically significant (p={p_val_env:.6f})
     - But still very weak (r={env_score_corr_s1:.3f})
     
  4. **RECOMMENDATION: SKIP ESG FEATURES ENTIRELY**
     - Add <5% marginal value
     - Sector + Revenue + Geography already explain 45-50%
     - Not worth complexity for weak signals
     - Focus on stronger predictors

VISUAL EVIDENCE:
  - Gov-Env gap scatter: NEGATIVE trend (opposite to hypothesis!)
  - Environmental score: Positive trend but VERY scattered (r=0.120)
  - Governance score: NEGATIVE trend (paradox confirmed)
  - Gap categories: Positive gap (Gov>>Env) emits MORE (but n=11 tiny sample!)
  - Env quartiles: Clear increase Q1→Q4 (as expected, +{env_quartile_increase}%)
  - Gov quartiles: DECREASE Q1→Q4 (paradox: -{gov_quartile_decrease}%)

UNEXPECTED FINDINGS:
  1. **{pct_negative_gap:.1f}% of companies have Env score WORSE than Gov score:**
     - Most companies: good governance, poor environmental performance
     - Only {pct_positive_gap:.1f}% have Gov > Env (positive gap)
     - Dataset heavily skewed toward poor environmental performers
     
  2. **Best governance quartile has HIGHEST emissions:**
     - Q1 (best gov): 75,410 Scope 1
     - Q4 (worst gov): 39,297 Scope 1
     - Nearly 2x difference in WRONG direction!
     - Clear size confounding effect
     
  3. **Scope 2 shows STRONGER governance effect:**
     - Gov score vs Scope 2: r={gov_score_corr_s2:.3f}
     - Gov score vs Scope 1: r={gov_score_corr_s1:.3f}
     - Better governance → MUCH lower Scope 2
     - Suggests governance affects energy purchasing decisions
     - But still paradoxical (better gov = higher reported emissions)
     
  4. **Environmental score² has HIGHER correlation than linear:**
     - Env² vs Scope 1: r={env_squared_corr_s1:.3f}
     - Env vs Scope 1: r={env_score_corr_s1:.3f}
     - Non-linear: very poor environmental scores → disproportionately high emissions
     - But still too weak to be useful (r<0.15)

  5. **Gap effect not statistically significant:**
     - T-test (large gap vs balanced): p={p_val_gap:.3f}
     - Despite 133% difference in means!
     - High variance and small sample (n=11 positive gap)
     - Can't reliably distinguish gap categories

CRITICAL LIMITATIONS:
  1. **Reverse causality:** Poor governance → high emissions? Or high emissions → better governance (to manage risk)?
  2. **Selection bias:** Good governance → better reporting accuracy
  3. **Size confounding:** Large companies have better governance AND more emissions
  4. **Small samples:** Only {pct_positive_gap:.1f}% have Gov > Env (n=33)
  5. **Score validity:** ESG scores are subjective, non-standardized across providers
  6. **Weak signals:** All correlations r<0.21 (very weak predictive power)

REVISED HYPOTHESIS:
  Original: "Gov-Env gap signals unmanaged carbon risk"
  
  Revised: "ESG scores have minimal predictive value for emissions;
            absolute G-E misalignment shows weak positive association"
  
  Better formulation:
  - ABSOLUTE gap (|Gov - Env|) → slightly higher emissions
  - Not about WHICH is better, but about ALIGNMENT
  - But effect is VERY WEAK (r={abs_gap_corr_s1:.3f})
  - Not worth including in model given stronger alternatives

REAL-WORLD INTERPRETATION:

**The 3 Gap Categories (Exploratory):**

1. **Env >> Gov (gap < -0.5, n=322, 75%)**:
   - Poor governance (2.25), VERY POOR environment (3.67)
   - Avg emissions: 62,262 Scope 1
   - Typical: Large manufacturing companies with high emissions
   - They KNOW they have problems (reporting them)
   
2. **Balanced (|gap| ≤ 0.5, n=96, 22%)**:
   - Governance and environmental performance aligned
   - Avg emissions: 31,800 Scope 1 (LOWEST!)
   - Companies with consistent ESG priorities
   
3. **Gov >> Env (gap > 0.5, n=11, 3%)**:
   - Moderate governance (2.84), EXCELLENT environment (1.81)
   - Avg emissions: 73,975 Scope 1
   - Rare: Companies with great environmental performance

**Balanced companies emit LEAST → Alignment may matter slightly**
But difference is not statistically significant (p={p_val_gap:.3f})

DATASET CHARACTERISTICS:
- Mean gap: -1.08 (most companies: Env worse than Gov)
- Only {pct_positive_gap:.1f}% have positive gap (Gov > Env)
- Environmental scores are generally POOR (mean=3.46, where 5=worst)
- Governance scores are generally MODERATE (mean=2.38)
- Most companies struggle with environmental performance more than governance

CONCLUSION:
  ❌ HYPOTHESIS NOT VALIDATED
  
  Original predictions WRONG:
  - Gov > Env gap does NOT → higher emissions (works opposite!)
  - Gap correlation has WRONG sign (r={gov_env_gap_corr_s1:.3f})
  - Governance paradox: better gov = higher emissions (confounded)
  
  What DOES work (but weak):
  ⚠️ Absolute gap magnitude → higher emissions (r={abs_gap_corr_s1:.3f})
  ⚠️ Environmental score → higher emissions (r={env_score_corr_s1:.3f})
  ❌ But both are TOO WEAK to be useful (<5% R²)
  
  **PRACTICAL RECOMMENDATION:**
  - **SKIP ESG FEATURES ENTIRELY**
  - They add <5% marginal R² improvement
  - Sector + Revenue + Geography explain 45-50% already
  - Not worth complexity given weak, paradoxical signals
  - Focus on validated hypotheses (H1, H2, H9, H10)
  - ESG scores are not good emission predictors
"""

print(summary)

print("\n" + "="*80)
print("ESG FEATURES SUMMARY TABLE")
print("="*80)
print(f"""
Feature                          │ Scope 1  │ Scope 2  │ Priority │ Use?
─────────────────────────────────┼──────────┼──────────┼──────────┼──────────
abs_gov_env_gap                  │  {abs_gap_corr_s1:.3f}   │  {abs_gap_corr_s2:.3f}   │ P3-SKIP  │ ❌ NO (too weak)
environmental_score              │  {env_score_corr_s1:.3f}   │  {env_score_corr_s2:.3f}   │ P3-SKIP  │ ❌ NO (too weak)
environmental_score_squared      │  {env_squared_corr_s1:.3f}   │  {env_squared_corr_s2:.3f}   │ P3-SKIP  │ ❌ NO (marginal)
governance_score                 │ {gov_score_corr_s1:.3f}   │ {gov_score_corr_s2:.3f}   │ SKIP     │ ❌ NO (paradox)
gov_env_gap                      │ {gov_env_gap_corr_s1:.3f}   │ {gov_env_gap_corr_s2:.3f}   │ SKIP     │ ❌ NO (wrong sign)
overall_score                    │  {overall_corr_s1:.3f}   │ -0.007   │ SKIP     │ ❌ NO (too weak)
social_score                     │  {social_corr_s1:.3f}   │  0.034   │ SKIP     │ ❌ NO (too weak)
─────────────────────────────────┴──────────┴──────────┴──────────┴──────────

KEY STATISTICS:
  Best ESG feature: abs_gov_env_gap (r={abs_gap_corr_s1:.3f})
  Expected R² contribution: ~4% (0.202² = 0.041)
  
  Gap distribution:
    Gov > Env (positive): {pct_positive_gap:.1f}% (n=33) ⚠️ TINY SAMPLE
    Balanced: 22.4% (n=96)
    Env > Gov (negative): {pct_negative_gap:.1f}% (n=322) - MAJORITY
    
  Statistical significance:
    Gap t-test: p={p_val_gap:.3f} ❌ NOT SIGNIFICANT
    Environmental ANOVA: p={p_val_env:.6f} ✅ SIGNIFICANT
    Governance ANOVA: p={p_val_gov:.3f} ❌ NOT SIGNIFICANT
    
  Emission differences (exploratory):
    Best env (Q1) → Worst env (Q4): +{env_quartile_increase}% ✅
    Best gov (Q1) → Worst gov (Q4): -{gov_quartile_decrease}% ❌ (paradox!)
    Balanced → Large gap: +{gap_increase:.1f}% (not significant)

⚠️ FINAL RECOMMENDATION: SKIP ALL ESG FEATURES
   - All features r<0.21 (very weak)
   - Governance paradox (confounded by size)
   - Gap works opposite to hypothesis
   - Not statistically reliable
   - Focus on sector + revenue + geography instead
""")

### Train Summary

In [ ]:
print("="*80)
print("CREATING FINAL FEATURE DATAFRAME FROM TRAIN_DF")
print("="*80)

print("""
Goal: Extract all validated features from train_df (Hypotheses 3, 9, 11)
Output: DataFrame ready for modeling
""")

print("\n" + "="*80)
print("STEP 1: CREATE ALL VALIDATED FEATURES")
print("="*80)

# Start with entity_id
train_features_final = train_df[['entity_id']].copy()

print("\n🔧 Creating features from Hypothesis 3 (Geography)...")

# Geographic features - Frequency encoding (VALID)
if 'country_frequency' not in train_df.columns:
    country_freq = train_df['country_name'].value_counts()
    train_df['country_frequency'] = train_df['country_name'].map(country_freq)
    print("   ✓ Created country_frequency")

if 'region_frequency' not in train_df.columns:
    region_freq = train_df['region_name'].value_counts()
    train_df['region_frequency'] = train_df['region_name'].map(region_freq)
    print("   ✓ Created region_frequency")

# Add to final features
train_features_final['country_frequency'] = train_df['country_frequency']
train_features_final['region_frequency'] = train_df['region_frequency']

# Categorical (optional - can use for one-hot encoding later)
train_features_final['country_name'] = train_df['country_name']
train_features_final['region_name'] = train_df['region_name']

print(f"   ✅ Added 4 geographic features")

print("\n🔧 Creating features from Hypothesis 9 (Scale)...")

# Scale features - Log revenue
if 'log_revenue' not in train_df.columns:
    train_df['log_revenue'] = np.log1p(train_df['revenue'])
    print("   ✓ Created log_revenue")

train_features_final['log_revenue'] = train_df['log_revenue']

# Country-adjusted scale (Z-score)
if 'revenue_country_zscore' not in train_df.columns:
    train_df['revenue_country_zscore'] = train_df.groupby('country_name')['revenue'].transform(
        lambda x: (x - x.mean()) / (x.std() + 1)
    )
    print("   ✓ Created revenue_country_zscore")

train_features_final['revenue_country_zscore'] = train_df['revenue_country_zscore']

# Binary giant indicator
if 'is_giant_in_country' not in train_df.columns:
    train_df['is_giant_in_country'] = (train_df['revenue_country_zscore'] > 1).astype(int)
    print("   ✓ Created is_giant_in_country")

train_features_final['is_giant_in_country'] = train_df['is_giant_in_country']

print(f"   ✅ Added 3 scale features")

print("\n🔧 Creating features from Hypothesis 11 (ESG - OPTIONAL)...")

# ESG features (weak, likely skip, but available if needed)

# Gov-Env gap
if 'gov_env_gap' not in train_df.columns:
    train_df['gov_env_gap'] = train_df['governance_score'] - train_df['environmental_score']
    print("   ✓ Created gov_env_gap")

if 'abs_gov_env_gap' not in train_df.columns:
    train_df['abs_gov_env_gap'] = abs(train_df['gov_env_gap'])
    print("   ✓ Created abs_gov_env_gap")

# ESG scores
if 'environmental_score_squared' not in train_df.columns:
    train_df['environmental_score_squared'] = train_df['environmental_score'] ** 2
    print("   ✓ Created environmental_score_squared")

# Add ESG features (marked as optional)
train_features_final['abs_gov_env_gap'] = train_df['abs_gov_env_gap']
train_features_final['environmental_score'] = train_df['environmental_score']
train_features_final['environmental_score_squared'] = train_df['environmental_score_squared']
train_features_final['governance_score'] = train_df['governance_score']
train_features_final['social_score'] = train_df['social_score']
train_features_final['overall_score'] = train_df['overall_score']

print(f"   ⚠️ Added 6 ESG features (WEAK - recommend skipping)")

print("\n" + "="*80)
print("STEP 2: FEATURE SUMMARY")
print("="*80)

print(f"\n📊 Final feature dataframe shape: {train_features_final.shape}")
print(f"   Rows (entities): {len(train_features_final)}")
print(f"   Columns (features + ID): {len(train_features_final.columns)}")

print("\n" + "-"*80)
print("FEATURE BREAKDOWN BY PRIORITY")
print("-"*80)

priority_1_features = [
    'log_revenue'
]

priority_2_features = [
    'country_frequency',
    'region_frequency',
    'revenue_country_zscore',
    'is_giant_in_country'
]

priority_3_categorical = [
    'country_name',
    'region_name'
]

priority_3_skip = [
    'abs_gov_env_gap',
    'environmental_score',
    'environmental_score_squared',
    'governance_score',
    'social_score',
    'overall_score'
]

print("\n✅ PRIORITY 1 (MUST INCLUDE - already from revenue distribution):")
for feat in priority_1_features:
    if feat in train_features_final.columns:
        print(f"   • {feat:40s} (r=0.275)")

print("\n✅ PRIORITY 2 (SHOULD INCLUDE - moderate value):")
for feat in priority_2_features:
    if feat in train_features_final.columns:
        dtype_str = str(train_features_final[feat].dtype)  # FIXED: Convert to string
        if feat == 'country_frequency':
            corr_info = "r=0.118"
        elif feat == 'region_frequency':
            corr_info = "r=-0.142 (skip)"
        elif feat == 'revenue_country_zscore':
            corr_info = "r=0.192"
        elif feat == 'is_giant_in_country':
            corr_info = "binary alternative"
        else:
            corr_info = ""
        print(f"   • {feat:40s} {dtype_str:12s} {corr_info}")

print("\n⚠️ PRIORITY 3 (OPTIONAL - categorical, need encoding):")
for feat in priority_3_categorical:
    if feat in train_features_final.columns:
        n_unique = train_features_final[feat].nunique()
        print(f"   • {feat:40s} ({n_unique} categories)")

print("\n❌ PRIORITY 3 (SKIP - too weak / paradoxical):")
for feat in priority_3_skip:
    if feat in train_features_final.columns:
        if feat == 'abs_gov_env_gap':
            corr_info = "r=0.202 (weak)"
        elif feat == 'environmental_score':
            corr_info = "r=0.120 (very weak)"
        elif feat == 'environmental_score_squared':
            corr_info = "r=0.141 (marginal)"
        elif feat == 'governance_score':
            corr_info = "r=-0.113 (paradox)"
        elif feat == 'social_score':
            corr_info = "r=0.046 (too weak)"
        elif feat == 'overall_score':
            corr_info = "r=0.072 (too weak)"
        else:
            corr_info = ""
        print(f"   • {feat:40s} {corr_info}")

print("\n" + "="*80)
print("STEP 3: DATA QUALITY CHECK")
print("="*80)

print(f"\n📊 Missing values:")
missing_summary = train_features_final.isnull().sum()
if missing_summary.sum() == 0:
    print("   ✅ No missing values!")
else:
    print(missing_summary[missing_summary > 0])

print(f"\n📊 Data types:")
print(train_features_final.dtypes.value_counts())

print(f"\n📊 Sample of first 5 rows:")
print(train_features_final.head())

print("\n" + "="*80)
print("STEP 4: RECOMMENDED FEATURE SUBSETS")
print("="*80)

print("\n💡 MINIMAL FEATURE SET (Recommended for initial modeling):")
minimal_features = [
    'entity_id',
    'log_revenue',
    'country_frequency',
    'revenue_country_zscore'
]
print(f"   Features: {minimal_features}")
print(f"   Count: {len(minimal_features)-1} features")
print(f"   Expected R²: ~10-12%")

print("\n💡 STANDARD FEATURE SET (Balanced performance/complexity):")
standard_features = [
    'entity_id',
    'log_revenue',
    'country_frequency',
    'revenue_country_zscore',
    'is_giant_in_country',
    'region_name'
]
print(f"   Features: {standard_features}")
print(f"   Count: {len(standard_features)-1} features + one-hot region (7 categories)")
print(f"   Expected R²: ~12-15%")

print("\n💡 FULL FEATURE SET (Maximum information, may overfit):")
full_features = [
    'entity_id',
    'log_revenue',
    'country_frequency',
    'revenue_country_zscore',
    'is_giant_in_country',
    'country_name',
    'region_name',
    'abs_gov_env_gap',
    'environmental_score'
]
print(f"   Features: {full_features}")
print(f"   Count: {len(full_features)-1} features + categorical encoding")
print(f"   Expected R²: ~15-18%")
print(f"   ⚠️ Warning: ESG features very weak, may not improve performance")

print("\n" + "="*80)
print("STEP 5: CREATE FEATURE SUBSETS")
print("="*80)

# Create the three subsets
train_features_minimal = train_features_final[minimal_features].copy()
train_features_standard = train_features_final[standard_features].copy()
train_features_full = train_features_final[full_features].copy()

print(f"\n✅ Created 3 feature subsets:")
print(f"   • train_features_minimal:  {train_features_minimal.shape}")
print(f"   • train_features_standard: {train_features_standard.shape}")
print(f"   • train_features_full:     {train_features_full.shape}")

print("\n" + "="*80)
print("✅ FEATURE DATAFRAMES READY!")
print("="*80)

print(f"""
Available DataFrames:

1. train_features_final ({train_features_final.shape})
   - All features from Hypotheses 3, 9, 11
   - {len(train_features_final.columns)-1} total features
   - Ready for modeling

2. train_features_minimal ({train_features_minimal.shape})
   - Best core features only
   - {len(train_features_minimal.columns)-1} features
   - Recommended for initial baseline

3. train_features_standard ({train_features_standard.shape})
   - Balanced feature set
   - {len(train_features_standard.columns)-1} features + region encoding
   - Recommended for production

4. train_features_full ({train_features_full.shape})
   - Maximum information
   - {len(train_features_full.columns)-1} features + categorical encoding
   - Use if model capacity allows
""")

print("\n" + "="*80)
print("FEATURE CORRELATION SUMMARY (from hypothesis testing)")
print("="*80)

correlation_summary = pd.DataFrame({
    'Feature': [
        'log_revenue',
        'revenue_country_zscore',
        'country_frequency',
        'region_frequency',
        'is_giant_in_country',
        'abs_gov_env_gap',
        'environmental_score',
        'environmental_score_squared',
        'governance_score',
        'social_score',
        'overall_score'
    ],
    'Scope_1_Corr': [0.275, 0.192, 0.118, -0.142, '?', 0.202, 0.120, 0.141, -0.113, 0.046, 0.072],
    'Scope_2_Corr': [0.211, 0.217, 0.113, -0.153, '?', 0.150, 0.049, 0.051, -0.167, 0.034, -0.007],
    'Priority': ['P1', 'P2', 'P2', 'SKIP', 'P2', 'P3-Skip', 'P3-Skip', 'P3-Skip', 'SKIP', 'SKIP', 'SKIP'],
    'Recommendation': [
        'MUST INCLUDE',
        'SHOULD INCLUDE',
        'SHOULD INCLUDE',
        'SKIP (negative)',
        'ALTERNATIVE to Z-score',
        'SKIP (too weak)',
        'SKIP (too weak)',
        'SKIP (marginal)',
        'SKIP (paradox)',
        'SKIP (too weak)',
        'SKIP (too weak)'
    ]
})

print("\n")
print(correlation_summary.to_string(index=False))

## Environmental Activity EDA & Feature Engineering
**For this portion of Environmental Activity analysis, please refer the details at "test notebooks/EnvAct2_EDA&Features.ipynb" file, we will import our final dataset we extract from the Environmental Activity table below for training Purpose.**

In [ ]:
environmental_activities_feature_df = pd.read_csv('../test notebooks/env_act_relatedFeatures.csv')
environmental_activities_feature_df.head()

In [ ]:
environmental_activities_feature_df.columns

## Sustainable Goal EDA & Feature Engineering
**For this portion please refer to "SDG.ipynb" file, we will import our final dataset we extract from the Environmental Activity table below for training Purpose.**

## Model Trainning

In [ ]:
# ============================================================================
# STEP 10b: APPLY DUAN'S SMEARING CORRECTION
# ============================================================================

print("\n" + "="*80)
print("STEP 10b: APPLYING DUAN'S SMEARING CORRECTION")
print("="*80)

def duans_smearing_correction(y_true_log, y_pred_log):
    """Apply Duan's smearing correction for log-transformed data"""
    residuals = y_true_log - y_pred_log
    smearing_factor = np.mean(np.expm1(residuals))
    return smearing_factor

# Calculate correction factors on training data
smearing_scope1 = duans_smearing_correction(y1_train, y1_train_pred)
smearing_scope2 = duans_smearing_correction(y2_train, y2_train_pred)

print(f"Scope 1 smearing factor: {smearing_scope1:.4f}")
print(f"Scope 2 smearing factor: {smearing_scope2:.4f}")

# Apply correction to validation predictions
y1_val_pred_orig_corrected = np.expm1(y1_val_pred) * smearing_scope1
y2_val_pred_orig_corrected = np.expm1(y2_val_pred) * smearing_scope2

print("\n📊 CORRECTED PERFORMANCE (ORIGINAL SCALE):")
print("-" * 80)
print(f"Scope 1 R² (corrected):  {r2_score(y1_val_orig, y1_val_pred_orig_corrected):.4f}")
print(f"Scope 2 R² (corrected):  {r2_score(y2_val_orig, y2_val_pred_orig_corrected):.4f}")

In [ ]:
print("="*80)
print("CORRECT MERGE: FIXING THE log_revenue DUPLICATE")
print("="*80)

print("""
PROBLEM IDENTIFIED:
===================

Revenue distribution features:
  • entity_id
  • log_revenue  ← DUPLICATE!
  • revenue_x_high_emission_pct
  • high_emission_pct
  • manufacturing_pct
  • is_manufacturing
  • dominant_nace_l1
  • is_mining
  • mining_pct
  • log_revenue_x_high_emission_pct
  • revenue_x_manufacturing_pct

Train features:
  • entity_id
  • log_revenue  ← DUPLICATE!
  • country_frequency
  • revenue_country_zscore
  • is_giant_in_country
  • region_name

SOLUTION: Drop log_revenue from train_features_standard before merge
          (Keep the one from revenue distribution)
""")

# ============================================================================
# STEP 1: CHECK CURRENT DATAFRAMES
# ============================================================================

print("\n" + "="*80)
print("STEP 1: VERIFY SOURCE DATAFRAMES")
print("="*80)

print(f"\n📊 final_revenue_features_df:")
print(f"   Shape: {final_revenue_features_df.shape}")
print(f"   Columns: {list(final_revenue_features_df.columns)}")

print(f"\n📊 train_features_standard:")
print(f"   Shape: {train_features_standard.shape}")
print(f"   Columns: {list(train_features_standard.columns)}")

# Identify overlap
overlap = set(final_revenue_features_df.columns) & set(train_features_standard.columns)
overlap.discard('entity_id')  # entity_id should overlap for merge

print(f"\n⚠️ Overlapping columns (besides entity_id): {overlap}")

# ============================================================================
# STEP 2: CLEAN MERGE - DROP DUPLICATE FROM TRAIN FEATURES
# ============================================================================

print("\n" + "="*80)
print("STEP 2: CLEAN MERGE")
print("="*80)

# Drop log_revenue from train_features_standard (keep revenue distribution version)
train_features_no_dup = train_features_standard.drop(columns=['log_revenue'], errors='ignore')

print(f"\n✅ Removed duplicate from train_features_standard:")
print(f"   Original columns: {list(train_features_standard.columns)}")
print(f"   After removing log_revenue: {list(train_features_no_dup.columns)}")

# Merge
combined_features_fixed = final_revenue_features_df.merge(
    train_features_no_dup,
    on='entity_id',
    how='inner',
    validate='1:1'
)

print(f"\n✅ Merge successful!")
print(f"   Revenue features: {final_revenue_features_df.shape}")
print(f"   Train features (no dup): {train_features_no_dup.shape}")
print(f"   Combined: {combined_features_fixed.shape}")

# Verify perfect merge
if len(combined_features_fixed) == len(final_revenue_features_df) == len(train_features_no_dup):
    print(f"\n✅ Perfect merge - all {len(combined_features_fixed)} entities matched!")
else:
    print(f"\n⚠️ Merge issue - check entity_id alignment")

# Check for any remaining duplicates
duplicate_cols = combined_features_fixed.columns[combined_features_fixed.columns.duplicated()].tolist()
if duplicate_cols:
    print(f"\n❌ Still have duplicate columns: {duplicate_cols}")
else:
    print(f"\n✅ No duplicate columns!")

print(f"\n📊 COMBINED FEATURES ({len(combined_features_fixed.columns)} columns):")
print(list(combined_features_fixed.columns))

# ============================================================================
# STEP 3: ADD TARGETS
# ============================================================================

print("\n" + "="*80)
print("STEP 3: ADD TARGETS")
print("="*80)

# Add targets
combined_features_fixed = combined_features_fixed.merge(
    train_df[['entity_id', 'target_scope_1', 'target_scope_2']],
    on='entity_id',
    how='left'
)

# Log-transform targets
combined_features_fixed['log_target_scope_1'] = np.log1p(combined_features_fixed['target_scope_1'])
combined_features_fixed['log_target_scope_2'] = np.log1p(combined_features_fixed['target_scope_2'])

print(f"\n✅ Targets added")
print(f"   Shape: {combined_features_fixed.shape}")

# ============================================================================
# STEP 4: ENCODE CATEGORICAL FEATURES
# ============================================================================

print("\n" + "="*80)
print("STEP 4: ONE-HOT ENCODE CATEGORICAL FEATURES")
print("="*80)

from sklearn.preprocessing import OneHotEncoder

# Identify categorical columns
categorical_cols = []

if 'region_name' in combined_features_fixed.columns:
    categorical_cols.append('region_name')
    print(f"   • region_name: {combined_features_fixed['region_name'].nunique()} categories")
    
if 'dominant_nace_l1' in combined_features_fixed.columns:
    categorical_cols.append('dominant_nace_l1')
    print(f"   • dominant_nace_l1: {combined_features_fixed['dominant_nace_l1'].nunique()} categories")

if categorical_cols:
    print(f"\n🔧 One-hot encoding: {categorical_cols}")
    
    encoder = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')
    encoded_cats = encoder.fit_transform(combined_features_fixed[categorical_cols])
    
    # Create DataFrame
    encoded_col_names = encoder.get_feature_names_out(categorical_cols)
    encoded_df = pd.DataFrame(
        encoded_cats,
        columns=encoded_col_names,
        index=combined_features_fixed.index
    )
    
    # Drop original categorical and add encoded
    combined_features_fixed = combined_features_fixed.drop(columns=categorical_cols).join(encoded_df)
    
    print(f"   ✅ Created {len(encoded_col_names)} dummy variables")
    print(f"   New shape: {combined_features_fixed.shape}")

# ============================================================================
# STEP 5: DEFINE FINAL FEATURE SET
# ============================================================================

print("\n" + "="*80)
print("STEP 5: DEFINE FEATURE SET FOR MODELING")
print("="*80)

# Exclude ID and targets
exclude_cols = ['entity_id', 'target_scope_1', 'target_scope_2', 
                'log_target_scope_1', 'log_target_scope_2']

# All feature columns
feature_cols_final = [col for col in combined_features_fixed.columns 
                      if col not in exclude_cols]

print(f"\n📊 TOTAL FEATURES: {len(feature_cols_final)}")

# Verify no duplicates
from collections import Counter
feature_counts = Counter(feature_cols_final)
duplicates = {col: count for col, count in feature_counts.items() if count > 1}

if duplicates:
    print(f"\n❌ ERROR: Found duplicates in feature list:")
    for col, count in duplicates.items():
        print(f"   • {col}: {count} times")
else:
    print(f"\n✅ NO DUPLICATES in feature list!")

# Categorize features for clarity
print("\n📊 FEATURE BREAKDOWN:")
print("-" * 80)

# Revenue & scale features
revenue_features = [col for col in feature_cols_final if 'revenue' in col.lower()]
print(f"\n1. REVENUE & SCALE ({len(revenue_features)} features):")
for feat in revenue_features:
    print(f"   • {feat}")

# Sector features
sector_features = [col for col in feature_cols_final if any(x in col.lower() 
                   for x in ['emission', 'manufacturing', 'mining', 'nace'])]
print(f"\n2. SECTOR COMPOSITION ({len(sector_features)} features):")
for feat in sector_features[:15]:  # Show first 15
    print(f"   • {feat}")
if len(sector_features) > 15:
    print(f"   ... and {len(sector_features)-15} more")

# Geographic features
geo_features = [col for col in feature_cols_final if any(x in col.lower() 
                for x in ['country', 'region'])]
print(f"\n3. GEOGRAPHIC ({len(geo_features)} features):")
for feat in geo_features:
    print(f"   • {feat}")

# Binary flags
binary_features = [col for col in feature_cols_final if col.startswith('is_')]
print(f"\n4. BINARY FLAGS ({len(binary_features)} features):")
for feat in binary_features:
    print(f"   • {feat}")

# Summary
print(f"\n" + "="*80)
print(f"FEATURE SUMMARY:")
print(f"  • Revenue/Scale:  {len(revenue_features)}")
print(f"  • Sector:         {len(sector_features)}")
print(f"  • Geographic:     {len(geo_features)}")
print(f"  • Binary flags:   {len(binary_features)}")
print(f"  • TOTAL:          {len(feature_cols_final)}")
print(f"="*80)

# ============================================================================
# STEP 6: PREPARE X AND Y
# ============================================================================

print("\n" + "="*80)
print("STEP 6: PREPARE X (FEATURES) AND Y (TARGETS)")
print("="*80)

# Features
X_final = combined_features_fixed[feature_cols_final].copy()

# Check for issues
print(f"\n📊 Data quality check:")
missing = X_final.isnull().sum().sum()
print(f"   Missing values: {missing}")

if missing > 0:
    print(f"   Columns with missing values:")
    missing_cols = X_final.isnull().sum()
    print(missing_cols[missing_cols > 0])
    print(f"\n   Filling with median...")
    X_final = X_final.fillna(X_final.median())

inf_count = np.isinf(X_final.values).sum()
print(f"   Infinite values: {inf_count}")

if inf_count > 0:
    print(f"   Replacing infinite values with median...")
    X_final = X_final.replace([np.inf, -np.inf], np.nan).fillna(X_final.median())

# Targets (log-transformed)
y_scope1_final = combined_features_fixed['log_target_scope_1'].copy()
y_scope2_final = combined_features_fixed['log_target_scope_2'].copy()

print(f"\n✅ Data prepared:")
print(f"   X shape: {X_final.shape}")
print(f"   y_scope1 shape: {y_scope1_final.shape}")
print(f"   y_scope2 shape: {y_scope2_final.shape}")
print(f"   Features: {len(feature_cols_final)} (NO DUPLICATES ✅)")

# ============================================================================
# STEP 7: TRAIN-VALIDATION SPLIT
# ============================================================================

print("\n" + "="*80)
print("STEP 7: SPLIT DATA (80% TRAIN, 20% VALIDATION)")
print("="*80)

from sklearn.model_selection import train_test_split

X_train_final, X_val_final, y1_train_final, y1_val_final, y2_train_final, y2_val_final = train_test_split(
    X_final, y_scope1_final, y_scope2_final,
    test_size=0.2,
    random_state=42
)

print(f"\n✅ Split complete:")
print(f"   Train: {X_train_final.shape[0]} samples × {X_train_final.shape[1]} features")
print(f"   Validation: {X_val_final.shape[0]} samples × {X_val_final.shape[1]} features")

# ============================================================================
# STEP 8: TRAIN RANDOM FOREST MODELS
# ============================================================================

print("\n" + "="*80)
print("STEP 8: TRAIN RANDOM FOREST MODELS (CLEAN - NO DUPLICATES)")
print("="*80)

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

print("\n🌲 Training Random Forest for Scope 1...")

rf_scope1_final = RandomForestRegressor(
    n_estimators=100,
    max_depth=20,
    min_samples_split=10,
    min_samples_leaf=5,
    max_features='sqrt',
    random_state=42,
    n_jobs=-1,
    verbose=0
)

rf_scope1_final.fit(X_train_final, y1_train_final)
print("   ✅ Scope 1 model trained!")

print("\n🌲 Training Random Forest for Scope 2...")

rf_scope2_final = RandomForestRegressor(
    n_estimators=100,
    max_depth=20,
    min_samples_split=10,
    min_samples_leaf=5,
    max_features='sqrt',
    random_state=42,
    n_jobs=-1,
    verbose=0
)

rf_scope2_final.fit(X_train_final, y2_train_final)
print("   ✅ Scope 2 model trained!")

# ============================================================================
# STEP 9: EVALUATE MODELS
# ============================================================================

print("\n" + "="*80)
print("STEP 9: EVALUATE MODEL PERFORMANCE")
print("="*80)

# Predictions
y1_train_pred_final = rf_scope1_final.predict(X_train_final)
y1_val_pred_final = rf_scope1_final.predict(X_val_final)

y2_train_pred_final = rf_scope2_final.predict(X_train_final)
y2_val_pred_final = rf_scope2_final.predict(X_val_final)

# Performance on log scale
print("\n📊 PERFORMANCE (LOG SCALE):")
print("="*80)
print("\nSCOPE 1:")
print(f"  Train R²:       {r2_score(y1_train_final, y1_train_pred_final):.4f}")
print(f"  Validation R²:  {r2_score(y1_val_final, y1_val_pred_final):.4f}")
print(f"  Train RMSE:     {np.sqrt(mean_squared_error(y1_train_final, y1_train_pred_final)):.4f}")
print(f"  Val RMSE:       {np.sqrt(mean_squared_error(y1_val_final, y1_val_pred_final)):.4f}")
print(f"  Val MAE:        {mean_absolute_error(y1_val_final, y1_val_pred_final):.4f}")

print("\nSCOPE 2:")
print(f"  Train R²:       {r2_score(y2_train_final, y2_train_pred_final):.4f}")
print(f"  Validation R²:  {r2_score(y2_val_final, y2_val_pred_final):.4f}")
print(f"  Train RMSE:     {np.sqrt(mean_squared_error(y2_train_final, y2_train_pred_final)):.4f}")
print(f"  Val RMSE:       {np.sqrt(mean_squared_error(y2_val_final, y2_val_pred_final)):.4f}")
print(f"  Val MAE:        {mean_absolute_error(y2_val_final, y2_val_pred_final):.4f}")

# Convert to original scale
y1_train_orig_final = np.expm1(y1_train_final)
y1_val_orig_final = np.expm1(y1_val_final)
y1_train_pred_orig_final = np.expm1(y1_train_pred_final)
y1_val_pred_orig_final = np.expm1(y1_val_pred_final)

y2_train_orig_final = np.expm1(y2_train_final)
y2_val_orig_final = np.expm1(y2_val_final)
y2_train_pred_orig_final = np.expm1(y2_train_pred_final)
y2_val_pred_orig_final = np.expm1(y2_val_pred_final)

print("\n📊 PERFORMANCE (ORIGINAL SCALE - NAIVE):")
print("="*80)
print("\nSCOPE 1:")
print(f"  Train R²:       {r2_score(y1_train_orig_final, y1_train_pred_orig_final):.4f}")
print(f"  Validation R²:  {r2_score(y1_val_orig_final, y1_val_pred_orig_final):.4f}")
print(f"  Train RMSE:     {np.sqrt(mean_squared_error(y1_train_orig_final, y1_train_pred_orig_final)):,.0f}")
print(f"  Val RMSE:       {np.sqrt(mean_squared_error(y1_val_orig_final, y1_val_pred_orig_final)):,.0f}")

print("\nSCOPE 2:")
print(f"  Train R²:       {r2_score(y2_train_orig_final, y2_train_pred_orig_final):.4f}")
print(f"  Validation R²:  {r2_score(y2_val_orig_final, y2_val_pred_orig_final):.4f}")
print(f"  Train RMSE:     {np.sqrt(mean_squared_error(y2_train_orig_final, y2_train_pred_orig_final)):,.0f}")
print(f"  Val RMSE:       {np.sqrt(mean_squared_error(y2_val_orig_final, y2_val_pred_orig_final)):,.0f}")

# ============================================================================
# STEP 10: FEATURE IMPORTANCE
# ============================================================================

print("\n" + "="*80)
print("STEP 10: FEATURE IMPORTANCE ANALYSIS")
print("="*80)

importance_scope1_final = pd.DataFrame({
    'feature': feature_cols_final,
    'importance': rf_scope1_final.feature_importances_
}).sort_values('importance', ascending=False)

print("\n📊 TOP 15 FEATURES FOR SCOPE 1:")
print("-" * 80)
print(importance_scope1_final.head(15).to_string(index=False))

importance_scope2_final = pd.DataFrame({
    'feature': feature_cols_final,
    'importance': rf_scope2_final.feature_importances_
}).sort_values('importance', ascending=False)

print("\n📊 TOP 15 FEATURES FOR SCOPE 2:")
print("-" * 80)
print(importance_scope2_final.head(15).to_string(index=False))

# ============================================================================
# FINAL SUMMARY
# ============================================================================

print("\n" + "="*80)
print("✅ CLEAN MODEL TRAINING COMPLETE!")
print("="*80)

print(f"""
FINAL SUMMARY:
==============

Data:
  ✅ Merged features properly (NO duplicates)
  ✅ {len(combined_features_fixed)} entities
  ✅ {len(feature_cols_final)} unique features
  
Features:
  • Revenue/Scale:  {len(revenue_features)}
  • Sector:         {len(sector_features)}
  • Geographic:     {len(geo_features)}
  • Binary flags:   {len(binary_features)}

Models Trained:
  ✅ Random Forest for Scope 1 (100 trees)
  ✅ Random Forest for Scope 2 (100 trees)

Performance (Validation, Log Scale):
  • Scope 1 R²: {r2_score(y1_val_final, y1_val_pred_final):.4f}
  • Scope 2 R²: {r2_score(y2_val_final, y2_val_pred_final):.4f}

Performance (Validation, Original Scale):
  • Scope 1 R²: {r2_score(y1_val_orig_final, y1_val_pred_orig_final):.4f}
  • Scope 2 R²: {r2_score(y2_val_orig_final, y2_val_pred_orig_final):.4f}

Top 3 Features (Scope 1):
  1. {importance_scope1_final.iloc[0]['feature']}: {importance_scope1_final.iloc[0]['importance']:.4f}
  2. {importance_scope1_final.iloc[1]['feature']}: {importance_scope1_final.iloc[1]['importance']:.4f}
  3. {importance_scope1_final.iloc[2]['feature']}: {importance_scope1_final.iloc[2]['importance']:.4f}

Saved Objects:
  • combined_features_fixed (complete dataset)
  • X_final, y_scope1_final, y_scope2_final (clean data)
  • rf_scope1_final, rf_scope2_final (trained models)
  • feature_cols_final (list of {len(feature_cols_final)} unique features)
  • encoder (for categorical encoding)

Next Steps:
  1. ✅ Duplicates fixed!
  2. 📊 Models trained with clean features
  3. 🚀 Ready for test predictions
  4. ⚡ Optional: Try hyperparameter tuning
  5. 🎯 Optional: Try ensemble methods
""")